# OSCAR-Ontology

In [346]:
import re

from datetime import datetime

from rdflib import Graph, RDF, URIRef, RDFS, OWL, Literal, Namespace, BNode, XSD
from rdflib.collection import Collection

import psycopg2

from SPARQLWrapper import SPARQLWrapper, TURTLE, RDFXML

from collections import defaultdict, Counter

from decimal import Decimal

from copy import deepcopy

from dateutil.parser import parse

import enchant
import nltk
from nltk.corpus import words

nltk.download('words')
english_words = set(words.words())
dict_en = enchant.Dict("en_US")


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\ilove\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


<ins><b>Nächste Schritte:</b></ins> 

- <b> Schlüssel suche in Daten:</b> 
    - Wenn eine Property sowas wie ID im Namen hat (und einzigartig ist) als Schlüsselaussuchen
    - Oder noch Besser URI nehmen, da wir von einzigartigkeit der Namen ausgehen können
- <b> Zwei Tabellen Generierungswege: </b>
    - Ontologie hat Individuen
    - Ontologie hat keine Individuen

<b> ALTE nächste Schritte: </b>
- <b> Reasoner </b> finden
- <b> Null Values verringern </b>:
    - Finde alle Attribute die wirklich zu einer Klasse gehören ohne Propertys!!
    - <s>Propertys: mit in Onto aufnehmen und schauen was ich damit machen kann</s>
    - Tabellen weiter aufteilen sinnvoll, wenn zu viele Null-Values?
    - Vorgehen, wenn keine Propertys vorhanden oder danach immer noch zu viele Null-Values
- <b> (Sub-)Properties</b> richtig ausnutzen
    - Reasoner finden/bauen der die findet und nutzbar macht
    - Wie könnten sie eingesetzt werden?
    - Bei Data Types überprüfen, ob die mit Propertys übereinstimmen. Also ob Domain und Range gleichen sind (sofern möglich)
    - Daraus können zusammengehörende Tabellen gebaut werden
    - <s>Wie funktioniert das genau mit der Vererbung </s>
    - Konzept der Vererbung richtig in RDB übersetzen
- <b> Oberklassen-Tabellen </b> generell sinnvoll? 
    - Dann hier hinter jeder ID Typ/Unterklasse des Individuals
    - vorher spezifizieren könnnen, ob man Oberklassen oder Subklassen Tabellen möchte
    - Ist es ab einer gewisssen Anzahl Subklassen evtl. generell sinnvoll Oberklassen zu nehmen? Evtl nur so lange wie nur Klassen ohne eigene Subklassen??
    - Auf Grundlage der Attribute bestimmen ob Klassen ähnliche Individuals haben. Bei ähnlichen sinnvoll zusammenzulegen bei unterschiedlichen eher nicht --> viele Null-Values
- Wie damit umgehen wenn eine <b> Tabelle nur wenige Attribute </b> hat?
- Wie damit umgehen wenn eine <b> Tabelle nur wenige Einträge </b> hat?
- Überprüfen ob <b>Tabellen-/Attributnamen aussagekraft</b> haben oder kryptisch/nur Zahlen sind??
- <b> zusammengehörende Infos </b> in Detail-Tabellen <b> als eine Zeile </b> anzeigen
- Was ist wenn es keinen <b> Schlüssel </b> gibt bzw. ein Attribut nicht als Schlüssel ausreicht?
    - Bei zwei oder mehr Attributen als Schlüssel, anpassen der Tabellen nötig, da alles auf ein Attribut ausgelegt ist
    - Propertys ausnutzen: Property verbindet Domain via Fremdschlüssel mit Range
- <b> Performance </b> verbessern
    - <s> Funktionen die Werte nicht ändern (z.B. get_class_attributes) einmal zentral aufrufen und dann nur noch auf Ergebnis zugreifen. </s>
- <b> "type"/Klassenname und "sub_class_of" als Attribute </b> zu Klassen hinzufügen?? 
- Kann man veralgemeiern welches Datenbanksystem genutzt wird?
- rdf oder turtel als Eingabe??
    - Wo gibt es mehr/weniger zu?
    - Macht es einen so großen Unterschied?
    - könnte ich schon ttl verwenden??
- Wie will ich evaluieren?
- Main-Funktion erstellen

\
<ins><b>Fertig:</b></ins> 
- <s> Pro Klasse alle Attribute bestimmen die nur einen Wert haben</s> ---> Abschnitt 7
- <s> Pro Klasse ein Attribut bestimmen das Einzigartig ist</s> ---> Abschnitt 8
- <s> Weitere Tabellen erstellen mit Schlüssel als Fremdschlüssel</s> ---> Abschnitt 8
- <s> Es fehlen weitere Klassen in der Oscar-Onto --> Hinzufügen wie bei AwardWinner</s> ---> Abschnitt 1
- <s> Einfügen der Individuals in aufgeteilte Tabellen
    - Alle Mehrfachwerte einfügen, d.h. Mehrere zeilen für ein Fremdschlüssel</s> ---> Abschnitt 9
- <s> in der Onto fehlt das label von Q150943 - Gene Hackman.</s> ---> Abschnitt 1 
- <s> def get_attributes_with_multi_values(...) aus Abschnitt 7 ziegt noch alle Attribute an und nicht nur die mit Mehrfachwerteb
    - entweder das hinbekommen, oder doch noch wieder was an def insert_individuals_into_tables(...) ändern, sodass alte Mehrfachwerte Def genutzt werden kann</s> ---> Abschnitt 6
- <s> Keine Detail-Tabelle erstellen, wenn keine Attribute mit mehrfachwerten vorhanden sind</s> ---> Abschnitt 8
- <s> über all defaultdict() durch reguläres dict ersetzen</s>
- <s> Finden der zu entfernenden Präfixe</s> ---> Abschnitt α
- <s> Attribute die alle/viele Detail Tabellen haben zusammenfügen? 
    - Als Oberklasse_Details Tabelle. Hier auch noch Typen/Klassen wo Individual her kommt als Attribut</s> ---> Abschnitt 12
- Null Values verringern:
    -  <s>Propertys: mit in Onto aufnehmen und schauen was ich damit machen kann</s> ---> Abschnitt 13
- <s> Datentypen der Attribute/Values (automatisch) bestimmen</s> ---> Abschnitt 14
- Performance verbessern
    - <s> Funktionen die Werte nicht ändern (z.B. get_class_attributes) einmal zentral aufrufen und dann nur noch auf Ergebnis zugreifen. </s> ---> Abschnitt Ω
- <s> <b> Beim Cleanen der Namen</b> Schlüsselwörter herraussuchen die nicht aufgeteilt werden soll. Also aus ID soll nicht i_d werden sondern nur id
    - Idee: Wörter wo alles groß geschrieben ist nicht auseinander nehmen sondern nur klein schreiben </s> ---> Abschnitt α
- <s> alles auf Turtle umbauen und schauen ob das bisherige dann immer noch geht. </s>
- <s> <b> FEHLER beheben:</b> wenn multiplevalue Attribute leer sind, werden singlevalue Attribute nicht berechnet </s> ---> Abschnitt 7

\
<ins><b>Won't Do:</b></ins> 
- <s> Ähnlichklingende Attribute zusammen fassen</s> ---> Macht wenig sinn, da auch Attribute die was ganz verschiedenes meinen ähnlich klingen können



\
<ins><b>Main-Funktion:</b></ins> \
Ausgaben:
- Welche Tabelle wurden mit welchen Attributen erstellt
- Was sind die Superklassen und was die zugehörigen Subklassen
- Anzahl Klassen, Attribte, Individuals
- Schlüssel/Fremdschlüssel 

Fragen/Eingaben: 
- Auf Grundlage welcher Klassen-Ebene sollen die Tabellen erstellt werden?
- Eingabe der .rdf Datei

Funktionalitäten:
- Aufruf der benötigten Funktionen
- überprüfen ob es Propertys gibt oder nicht. Je nach dem müssen unterschiedliche Wege gegangen werden


## α. remove_prefix und add_underscore_before_caps

In [501]:
def get_all_used_namespaces(ttl_file):
    """
    Scannt eine RDF-Datei und extrahiert alle verwendeten Namespace-URIs.

    :param ttl_file: Pfad zur RDF-Datei (XML oder Turtle)
    :return: Set mit eindeutigen Namespace-URIs
    """
    g = Graph()
    g.parse(ttl_file, format="turtle")  # Falls Turtle: format="turtle"

    used_namespaces = set()

    for s, p, o in g:
        for uri in (s, p, o):  # Überprüfe Subjekt, Prädikat und Objekt
            if isinstance(uri, URIRef):  # Nur URIs, keine Literale
                uri_str = str(uri)
                
                # Prüfe zuerst auf `#`, dann auf `/`
                if '#' in uri_str:
                    namespace = uri_str.rsplit('#', 1)[0] + '#'  # Alles bis zum letzten `#`
                elif '/' in uri_str:
                    namespace = uri_str.rsplit('/', 1)[0] + '/'  # Alles bis zum letzten `/`
                else:
                    namespace = uri_str  # Falls kein Trenner vorhanden

                used_namespaces.add(namespace)  # Namespace speichern

    return used_namespaces

# ttl_file = "oscars_expose_updated4.ttl"
ttl_file = "Wine_reasoning2.ttl"
PREFIXES_TO_REMOVE = get_all_used_namespaces(ttl_file)
# print(PREFIXES_TO_REMOVE)


def remove_prefix(value):
    """Entfernt definierte Präfixe aus einer URI."""
    value = str(value).strip()

    for prefix in PREFIXES_TO_REMOVE:
        if value.startswith(prefix):
            value = value[len(prefix):].strip()

    return value 


def add_short_prefix(value):
    """_summary_"""
    prefix = value.rsplit('/', 1)[-1].split('#')[0]
    
    return f"{prefix}_{remove_prefix(value)}"
 

def find_all_caps_words(value):
    """
    Gibt alle Substrings mit 2 oder mehr aufeinanderfolgenden Großbuchstaben zurück.
    Wenn auf den letzten Großbuchstaben direkt ein Kleinbuchstabe folgt,
    wird dieser letzte Großbuchstabe ausgeschlossen.
    """
    matches = []
    for m in re.finditer(r'[A-Z]{2,}', value):
        end = m.end()
        # Prüfe: Kommt direkt nach dem Match ein Kleinbuchstabe?
        if end < len(value) and value[end].islower():
            # Letzten Buchstaben entfernen
            matches.append(m.group(0)[:-1])
        else:
            matches.append(m.group(0))
    return matches


def split_around_keywords(value, keywords):
    """
    Zerteilt `text` so, dass jedes Vorkommen eines beliebigen Keywords
    als eigener Eintrag in der Liste erscheint. Vor und nach jedem Keyword wird getrennt.
    
    `keywords` ist eine Liste von Wörtern.
    """
    if not keywords:
        return value
    else:
        # Baue einen Regex-Ausdruck, der alle Schlüsselwörter gruppiert 
        pattern = '|'.join(re.escape(k) for k in keywords)
        parts = re.split(f'({pattern})', value)
        return [part.strip() for part in parts if part]  # leere Teile entfernen


def add_underscore_before_caps(keywords, split):
    """
    Ersetzt Leerzeichen durch `_`, fügt `_` vor Großbuchstaben hinzu (falls nicht bereits vorhanden),
    und wandelt den gesamten String in Kleinbuchstaben um.
    """
    if isinstance(split, str):
        split = [split]  # String zu Liste machen

    result = []

    for part in split:
        if part in keywords:
            part = part.lower()
            result.append(part)
        else:
            # part = part.strip()
            part = part.replace(" ", "_")
            # Dann `_` vor Großbuchstaben einfügen, falls nicht am Anfang oder bereits ein `_` davor steht
            part = re.sub(r'(?<!^)(?<!_)([A-Z])', r'_\1', part)  
            part = part.lower()  # Alles in Kleinbuchstaben umwandeln
            result.append(part)
    return '_'.join(result)


def clean_value(value):
    value = remove_prefix(value)
    keywords = find_all_caps_words(value)  # z.B. ['ID', 'HTML']
    parts = split_around_keywords(value, keywords)
    clean_value = add_underscore_before_caps(keywords, parts)
    clean_value = re.sub(r"[^\w]", "", clean_value)

    return clean_value


## Alle Klassen und Proeprties

In [507]:
def get_all_classes(graph):

    g = graph

    # Alle Klassen, die explizit als owl:Class oder rdfs:Class deklariert sind
    explicit_class = set(g.subjects(RDF.type, RDFS.Class)).union(
        g.subjects(RDF.type, OWL.Class)
    )
    # Klassen aus rdfs:domain und rdfs:range
    domain_range_class = set(g.objects(None, RDFS.domain)).union(
        g.objects(None, RDFS.range)
    )
    # Klassen, die als Ziel oder Start von rdfs:subClassOf auftauchen
    super_class = set(g.objects(None, RDFS.subClassOf)).union(
        g.subjects(RDFS.subClassOf, None)
    )
    # Klassen, die als Ziel oder Start von owl.equivalentClass auftauchen
    equivalent_class = set(g.objects(None, OWL.equivalentClass)).union(
        g.subjects(OWL.equivalentClass, None)
    )
    # Klassen die Ziel oder Start von owl:complementOf auftauchen
    complement_class = set(g.subjects(OWL.complementOf, None)).union(
        g.objects(None, OWL.complementOf)
    ) 
    # Klassen aus owl:someValuesFrom, owl:allValuesFrom, owl:onClass
    restriction_targets = set(g.objects(None, OWL.someValuesFrom)).union(
        g.objects(None, OWL.allValuesFrom),
        g.objects(None, OWL.onClass)
    )
    # # Klassen die durch Class Expessions definiert sind
    # cls_with_classexpressions = set(g.subjects(RDFS.subClassOf, None)).union(
    #     g.subjects(OWL.equivalentClass, None)
    # )

    # Klassen mit min einer Instanz
    cls_with_instances = {}
    for _, _, cls in g.triples((None, RDF.type, None)):
        if isinstance(cls, URIRef):
            cls_with_instances[cls] = cls_with_instances.get(cls, 0) + 1
    cls_with_inst = set(cls_with_instances.keys())

    # Klassen die innerhalb einer Liste stehen
    in_bnods = set(g.objects(None, RDF.first))
    cls_in_bnode = set()
    for cls in in_bnods:
        if cls not in g.subjects(RDF.type, None):
            cls_in_bnode.add(cls)   

    # Klassen als Start oder Ziel von owl:disjointWith und owl:disjointUnionOf
    disjoint_class = set(g.subjects(OWL.disjointWith, None)).union(
        g.objects(None, OWL.disjointWith),
        g.subjects(OWL.disjointUnionOf, None),
        g.objects(None, OWL.disjointUnionOf)
    )

    all_class = explicit_class.union(
        domain_range_class,
        super_class,
        equivalent_class,
        restriction_targets,
        complement_class,
        cls_with_inst,
        cls_in_bnode,
        disjoint_class
    )

    # filtert alle technischen Klassen raus
    TECHNICAL_NAMESPACES = (
        "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "http://www.w3.org/2000/01/rdf-schema#",
        "http://www.w3.org/2002/07/owl#",
        "http://www.w3.org/2001/XMLSchema#",
    )
    # entfernt alle technischen Klassen und alle Blind Nodes
    all_classes = set(cls for cls in all_class 
                      if not str(cls).startswith(TECHNICAL_NAMESPACES) 
                      and not isinstance(cls, BNode)
                      and cls not in set(g.subjects(RDF.type, RDFS.Datatype))
                    )
    return all_classes


ttl_file = "oscars_expose_updated4.ttl"
g = Graph()
g.parse(ttl_file, format="turtle")

for p in get_all_classes(g):
    print(p)

http://example.org/ontology#AwardWinner2
http://example.org/ontology#Person
http://example.org/ontology#AwardWinner3
http://example.org/ontology#Film3
http://example.org/ontology#Princess2
http://example.org/ontology#Director
http://example.org/ontology#Film5
http://example.org/ontology#Movie
http://example.org/ontology#Producer
http://example.org/ontology#MovieProduction
http://example.org/ontology#test2
http://example.org/ontology#Princess
http://example.org/ontology#Prinzessin
http://example.org/ontology#AwardWinner
http://example.org/ontology#Writer
http://example.org/ontology#Award
http://example.org/ontology#Movie2
http://example.org/ontology#Movies2010er
http://example.org/ontology#Movie3
http://example.org/ontology#Film2
http://example.org/ontology#Regessure
http://example.org/ontology#Film
http://example.org/ontology#OscarWinner


In [497]:
def get_all_properties(graph):

    g = graph

    used_prop = {}
    for _, p, _ in g:
        if isinstance(p, URIRef):
            used_prop[p] = used_prop.get(p, 0) + 1
    
    used_props = set(used_prop.keys())
    
    # alle Proeprties die Typisiert sind.
    explicit_props = set(g.subjects(RDF.type, OWL.ObjectProperty)).union(
        g.subjects(RDF.type, OWL.DatatypeProperty),
        # g.subjects(RDF.type, OWL.AnnotationProperty),
        g.subjects(RDF.type, RDF.Property)
    )
    # alle Properties die eine Domain oder Range haben.
    domain_range_props = set(g.subjects(RDFS.domain, None)).union(
        g.subjects(RDFS.range, None)
    )

    # TODO theoretisch fehlen noch die Proeprties die weder genutzt werden noch eine 
        # Typzuordnung oder Domain/Range haben.
    # TODO es Fehlen noch die Properties die nur in einer Restriction genutzt werden
    all_props = used_props.union(explicit_props, domain_range_props)

    # filtert alle technischen Properties raus
    TECHNICAL_NAMESPACES = (
        "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "http://www.w3.org/2000/01/rdf-schema#",
        "http://www.w3.org/2002/07/owl#",
        "http://www.w3.org/2001/XMLSchema#"
    )

    all_properties = set(prop for prop in all_props 
                         if not str(prop).startswith(TECHNICAL_NAMESPACES) 
                         and prop not in set(g.subjects(RDF.type, OWL.AnnotationProperty))
                        )


    return all_properties

# ttl_file = "oscars_expose_updated4.ttl"
# # ttl_file = "Wine_reasoning.ttl"
# g = Graph()
# g.parse(ttl_file, format="turtle")

# all_properties = list(get_all_properties(g))
# print(all_properties)
# for p in all_properties:
#     print(p)

In [508]:
def get_elements_by_group(graph):
    """_summary_

    :param g: _description_
    :type g: _type_
    """
    g = graph

    all_classes = get_all_classes(g)
    all_properties = get_all_properties(g)
    all_individuals = set()
    
    for s, o in g.subject_objects(RDF.type):
        if o in all_classes or o == OWL.NamedIndividual:
            all_individuals.add(s)

    
    elements_by_group = {
        "classes": list(all_classes),
        "properties": list(all_properties),
        "individuals": list(all_individuals)
    }


    return elements_by_group

ttl_file = "oscars_expose_updated4.ttl"
# ttl_file = "Wine_reasoning.ttl"
g = Graph()
g.parse(ttl_file, format="turtle")

get_elements_by_group(g)

{'classes': [rdflib.term.URIRef('http://example.org/ontology#AwardWinner2'),
  rdflib.term.URIRef('http://example.org/ontology#Person'),
  rdflib.term.URIRef('http://example.org/ontology#AwardWinner3'),
  rdflib.term.URIRef('http://example.org/ontology#Film3'),
  rdflib.term.URIRef('http://example.org/ontology#Princess2'),
  rdflib.term.URIRef('http://example.org/ontology#Director'),
  rdflib.term.URIRef('http://example.org/ontology#Film5'),
  rdflib.term.URIRef('http://example.org/ontology#Movie'),
  rdflib.term.URIRef('http://example.org/ontology#Producer'),
  rdflib.term.URIRef('http://example.org/ontology#MovieProduction'),
  rdflib.term.URIRef('http://example.org/ontology#test2'),
  rdflib.term.URIRef('http://example.org/ontology#Princess'),
  rdflib.term.URIRef('http://example.org/ontology#Prinzessin'),
  rdflib.term.URIRef('http://example.org/ontology#AwardWinner'),
  rdflib.term.URIRef('http://example.org/ontology#Writer'),
  rdflib.term.URIRef('http://example.org/ontology#Awar

## 3. Restrictionspaket

### Properties

Bestimmt für Proeprties alle Restrictionskonstrukte in dennen sie genutzt werden, die Klasse die daraus enstehen und welchen Wert die Property hat

In [350]:
def extract_property_restrictions(graph):
    """
    Extrahiert alle owl:hasValue-Verwendungen aus owl:Restriction-Knoten.
    Gibt für jede Property alle Klassen zurück, in denen sie per hasValue verwendet wurde,
    zusammen mit dem jeweiligen Wert.

    :param ttl_file: Pfad zur Turtle-Datei
    :return: Dictionary {Property: [{"class": ..., "predicate", ...", value": ...}, ...]}
    """

    g = graph

    result = {}

    predicates = [
        OWL.hasValue, 
        OWL.someValuesFrom, 
        OWL.allValuesFrom,
        OWL.cardinality, 
        OWL.minCardinality, 
        OWL.maxCardinality,
        OWL.qualifiedCardinality, 
        OWL.minQualifiedCardinality, 
        OWL.maxQualifiedCardinality,
        OWL.hasSelf
    ]

    restriction_location =  set(g.triples((None, OWL.equivalentClass, None))).union(
        g.triples((None, RDFS.subClassOf, None))
    )

    for cls, location, equivalent_class in restriction_location:
        # Fall 1: Direkte Restriction
        if (equivalent_class, RDF.type, OWL.Restriction) in g:
            prop = g.value(equivalent_class, OWL.onProperty)
            for pred in predicates:
                value = g.value(equivalent_class, pred)
                if prop and value:
                    entry = {
                        "definedClass": cls,
                        "predicate": pred,
                        "value": value,
                        "restrictionLocation": location, 
                        "logicalContainer": None
                    }
                    if pred in (OWL.qualifiedCardinality, OWL.minQualifiedCardinality, OWL.maxQualifiedCardinality):
                        qual_class = g.value(equivalent_class, OWL.onClass)
                        if qual_class:
                            entry["onClass"] = qual_class
                    result.setdefault(prop, []).append(entry)

        # Fall 2: Komplexe Klassen mit owl:intersectionOf, unionOf etc.
        for list_pred in [OWL.intersectionOf, OWL.unionOf, OWL.complementOf]:
            if (equivalent_class, list_pred, None) in g:
                collection = g.value(equivalent_class, list_pred)
                while collection:
                    node = g.value(collection, RDF.first)
                    collection = g.value(collection, RDF.rest)

                    if (node, RDF.type, OWL.Restriction) in g:
                        prop = g.value(node, OWL.onProperty)
                        for pred in predicates:
                            value = g.value(node, pred)
                            if prop and value:
                                entry = {
                                    "definedClass": cls,
                                    "predicate": pred,
                                    "value": value,
                                    "restrictionLocation": location, 
                                    "logicalContainer": list_pred
                                }
                                if pred in (OWL.qualifiedCardinality, OWL.minQualifiedCardinality, OWL.maxQualifiedCardinality):
                                    qual_class = g.value(node, OWL.onClass)
                                    if qual_class:
                                        entry["onClass"] = qual_class
                                result.setdefault(prop, []).append(entry)

    return result

# ttl_file = "oscars_expose_updated4.ttl"
# g = Graph()
# g.parse(ttl_file, format="turtle")

# test = extract_property_restrictions(g)

# for a, b in test.items():
#     if a == URIRef("http://example.org/ontology#min"):
#         print(a)
#         for x in b:
#             print(x)
#             for y, z in x.items():
#                 print(f"    {y} - {z}")

### Klassen

Findet alle Class Axiome zu einer angegebenen Klasse und sammelt alle Infos zu den genutzten Restrictions. \
Außerdem unterschiedet er ob es sich beim Objekt der subClassOf und equivalentClass Beziehung um eine echte Klasse oder ein BNode/Class Expression handelt. 

In [351]:
def extract_class_axioms(g, cls, relation):
    """
    Verallgemeinerte Extraktion von OWL-Klassenaussagen, z.B. subClassOf oder equivalentClass.
    Liefert sowohl explizite Klassen als auch OWL-Restriktionen (direkt und in RDF-Listen).
    
    :param g: rdflib.Graph
    :param cls: URIRef der Klasse
    :param relation: RDFS.subClassOf oder OWL.equivalentClass
    :return: [{ "type": "unionOf", "restrictions": [...] }, ...]
    """
    explicit_classes = set()
    restrictions = []

    if relation == OWL.disjointUnionOf:
        parts = list(Collection(g, g.value(cls, OWL.disjointUnionOf)))
        all_uri = all(isinstance(p, URIRef) for p in parts)
        is_class = []
        if all_uri:
            explicit_classes = parts
        else:
            for target in parts:
                if isinstance(target, BNode):
                    parse_complex_class_construct(g, target, restrictions)
                if isinstance(target, URIRef):
                    is_class.append(target)
            restrictions.append({"disjointUnionOfClasses": is_class})
    
    else:
        for target in g.objects(cls, relation):  
            if isinstance(target, URIRef):
                explicit_classes.add(target)           
            elif isinstance(target, BNode):
                parse_complex_class_construct(g, target, restrictions)

    return explicit_classes, restrictions


def parse_restriction(g, node):
    """
    Extrahiert OWL-Restriction-Details aus einem BNode.
    """
    restriction = {
        "owl:onProperty": g.value(node, OWL.onProperty),
        "owl:onClass": g.value(node, OWL.onClass),
        "owl:hasValue": g.value(node, OWL.hasValue),
        "owl:someValuesFrom": g.value(node, OWL.someValuesFrom),
        "owl:allValuesFrom": g.value(node, OWL.allValuesFrom),
        "owl:cardinality": g.value(node, OWL.cardinality),
        "owl:minCardinality": g.value(node, OWL.minCardinality),
        "owl:maxCardinality": g.value(node, OWL.maxCardinality),
        "owl:qualifiedCardinality": g.value(node, OWL.qualifiedCardinality),
        "owl:minQualifiedCardinality": g.value(node, OWL.minQualifiedCardinality),
        "owl:maxQualifiedCardinality": g.value(node, OWL.maxQualifiedCardinality),
    }
    return {k: v for k, v in restriction.items() if v}


def parse_complex_class_construct(g, target, restrictions):
    """_summary_

    :param g: _description_
    :type g: _type_
    :param target: _description_
    :type target: _type_
    :param restrictions: _description_
    :type restrictions: _type_
    :return: _description_
    :rtype: _type_
    """

    # Fall 1: direkte Restriction
    if (target, RDF.type, OWL.Restriction) in g:
        restrictions.append({
            "type": "singleRestriction",
            "restrictions": [parse_restriction(g, target)]
        })

    # Fall 2: unionOf / intersectionOf
    for list_pred in [OWL.unionOf, OWL.intersectionOf]:
        if (target, list_pred, None) in g:
            collection = g.value(target, list_pred)
            if isinstance(collection, BNode):
                restriction_list = {
                    "type": list_pred,  # "unionOf" oder "intersectionOf"
                    "classes": [],
                    "restrictions": []
                }
                for node in Collection(g, collection):
                    if isinstance(node, URIRef):
                        restriction_list["classes"].append(node)
                    elif (node, RDF.type, OWL.Restriction) in g:
                        restriction_list["restrictions"].append(parse_restriction(g, node))
                if restriction_list["classes"] or restriction_list["restrictions"]:
                    restrictions.append(restriction_list)

    return restrictions


# ttl_file = "oscars_expose_updated4.ttl"
# g = Graph()
# g.parse(ttl_file, format="turtle")

# cls = URIRef("http://example.org/ontology#Writer")
# for rel in [RDFS.subClassOf, OWL.equivalentClass, OWL.complementOf, OWL.disjointUnionOf]:
#     a, b = extract_class_axioms(g, cls, rel)
#     print(a,b,rel)

# for c in b:
#     print(type(c))

Sucht für alle subClass- und equivalentClass Restrictions einer Klasse die raus mit Cardinality-Angaben und grupiert diese nach den Properties

In [352]:
def extract_cardinality_constraints(sub_class_restrictions, equivalent_restrictions):
    """_summary_

    :param sub_class_restrictions: _description_
    :type sub_class_restrictions: _type_
    :param equivalent_restrictions: _description_
    :type equivalent_restrictions: _type_
    :return: _description_
    :rtype: _type_
    """
    cardinality_list = []
    result = {}

    axiom_blocks = (sub_class_restrictions or []) + (equivalent_restrictions or [])
    # print(axiom_blocks)

    for block in axiom_blocks:
        for r in block.get("restrictions", []):
            cardinality_keys = [
                "owl:cardinality", "owl:minCardinality", "owl:maxCardinality",
                "owl:qualifiedCardinality", "owl:minQualifiedCardinality", "owl:maxQualifiedCardinality"
            ]
            if any(k in r for k in cardinality_keys):
                cardinality_list.append(r)


    for entry in cardinality_list:
        prop = entry.get("owl:onProperty")
        if prop:
            if prop in result:
                result[prop].append(entry)
            else:
                result[prop] = [entry]
    return result


# ttl_file = "oscars_expose_updated4.ttl"

# g = Graph()
# g.parse(ttl_file, format="turtle")

# cls = URIRef("http://example.org/ontology#AwardWinner2")
# subClassOf, sub_class_restrictions = extract_class_axioms(g, cls, RDFS.subClassOf)
# equivalent_classes, equivalent_restrictions = extract_class_axioms(g, cls, OWL.equivalentClass)
# # print(sub_class_restrictions)
# cardinality_constraints = extract_cardinality_constraints(sub_class_restrictions, equivalent_restrictions)


# print(cardinality_constraints)
# for prop, card in cardinality_constraints.items():
#     print(prop)
#     print(f"    -{card}")




Clustert wo und wie eine Klasse in ClassExpressions verwendet wird.

In [353]:
def collect_class_usages(g):
    """
    Sammelt, wo und wie jede Klasse in OWL-Klassenaussagen verwendet wird.

    :param g: rdflib.Graph
    :param all_classes: Menge aller Klassen-URIs
    :return: Dict[class_uri] = [Verwendungsinformationen]
    """
    
    usage_map = {}

    for cls in get_all_classes(g):
        for rel in [RDFS.subClassOf, OWL.equivalentClass, OWL.complementOf, OWL.disjointUnionOf]:
            explicit, restrictions = extract_class_axioms(g, cls, rel)

            # explizite Zielklassen (z.B. ex:A subClassOf ex:B)
            for target in explicit:
                # print(target)
                if target not in usage_map:
                    usage_map[target] = []
                usage_map[target].append({
                    "usedIn": cls,
                    "relation": rel,
                    "context": "explicit"
                })

            for block in restrictions:
                # Klassen aus unionOf / intersectionOf
                for c in block.get("classes", []):
                    if isinstance(c, URIRef):
                        if c not in usage_map:
                            usage_map[c] = []
                        usage_map[c].append({
                            "usedIn": cls,
                            "relation": rel,
                            "context": block["type"]
                        })

                # Klassen aus Restriktionen (someValuesFrom, allValuesFrom, onClass)
                for r in block.get("restrictions", []):
                    for key in ["owl:someValuesFrom", "owl:allValuesFrom", "owl:onClass"]:
                        if key in r:
                            target_class = r[key]
                            if isinstance(target_class, URIRef):
                                if target_class not in usage_map:
                                    usage_map[target_class] = []
                                usage_map[target_class].append({
                                    "usedIn": cls,
                                    "relation": rel,
                                    "context": key
                                })
                
                # Klassen aus owl:disjointUnionOf
                for d in block.get("disjointUnionOfClasses", []):
                    if isinstance(d, URIRef):
                        if d not in usage_map:
                            usage_map[d] = []
                        usage_map[d].append({
                            "usedIn": cls,
                            "relation": rel
                        })


                    # Absicherung: Falls mal URIRefs direkt in "restrictions" auftauchen
                    if isinstance(r, URIRef):
                        if r not in usage_map:
                            usage_map[r] = []
                        usage_map[r].append({
                            "usedIn": cls,
                            "relation": rel,
                            "context": "directRestrictionClass"
                        })

    return usage_map

# ttl_file = "oscars_expose_updated4.ttl"
# g = Graph()
# g.parse(ttl_file, format="turtle")

# # explicit_class = set(g.subjects(RDF.type, RDFS.Class)).union(
# #     g.subjects(RDF.type, OWL.Class),
# #     URIRef("http://example.org/ontology#AwardWinner2")
# # )
# # # print(explicit_class)

# cls = URIRef("http://example.org/ontology#Writer")

# # all_class = all_classes()

# usages = collect_class_usages(g)

# # print(usages)
# for cls, uses in usages.items():
#     print(f"{cls} wird verwendet in:")
#     for u in uses:
#         print(f"  → {u}")

## 1. Basispaket

### R1 - Klassenabbildung


siehe <b> Tabellen Infos zusammentragen </b>

### R10 - Individuen-Initialisierung

siehe <b> SQL-Generierung - Einträge </b>

### R4 - Subklassenbeziehung (einfach)


für Variante 2 von R4 

In [354]:
def decide_wich_super_class_to_keep(graph, super_classes):
    """_summary_

    :param graph: _description_
    :type graph: _type_
    """
    g = graph

    best_super_class = None
    best_score = -1  # kleiner als jeder mögliche Score
    
    for super_class in super_classes:
        as_subject = len(list(g.subject_predicates(super_class)))
        as_object = len(list(g.predicate_objects(super_class)))
        score = as_subject + as_object

        if score > best_score:
            best_score = score
            best_super_class = super_class
        
    return best_super_class


# ttl_file = "oscars_expose_updated4.ttl"
# g = Graph()
# g.parse(ttl_file, format="turtle")

# cls = URIRef("http://example.org/ontology#Director")
# super_class, as_restriction = extract_class_axioms(g, cls, RDFS.subClassOf)

# decide_wich_super_class_to_keep(g, super_class)



### R6/R7 - Single Value DatatypeProperty/ObjectProperty


In [355]:
def get_singlevalue_properties(graph, prop):
    """_summary_

    :param graph: _description_
    :type graph: _type_
    """
    g = graph
    single_props = {}
    # single = True

    prop_subjects = list(g.subjects(prop)) 
            # ohne das list() (bzw. set()) kann ich g.subjets() (bzw. alle Generatoren) nur einmal benutzten
    if set(g.triples((prop, RDF.type, OWL.FunctionalProperty))):
        single_props[prop] = "functional"

    elif prop_subjects:
        single = True
        for s in prop_subjects:      
            if not len(list(g.objects(s, prop))) == 1:
                single = False
        if single:
            single_props[prop] = "empirically functional"

        

    return single_props



# ttl_file = "oscars_expose_updated4.ttl"
# g = Graph()
# g.parse(ttl_file, format="turtle")

# properties = get_all_properties(g)
# prop = URIRef("http://example.org/ontology#test123")
# for prop in properties:
#     single = get_singlevalue_properties(g, prop)
#     print(single)

# single

### R3 - Schlüsselwahl


In [356]:
def get_primary_key(graph):
    """_summary_

    :param graph: _description_
    :type graph: _type_
    :return: _description_
    :rtype: _type_
    """
    g = graph

    # primary_key = 
    has_individuals, all_uris_in_class_unique = ontology_has_individuals(g)
    
    if has_individuals:
        if all_uris_in_class_unique:
            primary_key = "uri"
        else:
            primary_key = "auto"
    else: 
        primary_key = "auto"
    
    return primary_key


def ontology_has_individuals(graph):
    """
    Gibt True zurück, wenn in der Ontologie mindestens ein Individuum existiert
    """
    g = graph

    total_number_class_individuals = Counter()
    unique_class_individuals = defaultdict(set)
    individuals_not_unique = {}


    has_instances = False
    all_uris_in_class_unique = True

    EXCLUDED_NAMESPACES = {str(OWL), str(RDFS), str(XSD)}

    for s, p, o in g.triples((None, RDF.type, None)):
        if not any(str(o).startswith(ns) for ns in EXCLUDED_NAMESPACES):
            total_number_class_individuals[o] += 1
            unique_class_individuals[o].add(s)
            has_instances = True
    
    for cls in total_number_class_individuals:
        total_number = total_number_class_individuals[cls]
        unique_number = len(unique_class_individuals[cls])
        # all_individuals_unique[cls] = total_number == unique_number 
        if not total_number == unique_number:
            all_uris_in_class_unique = False 
            individuals_not_unique[cls] = total_number != unique_number 

    if not all_uris_in_class_unique:
        return has_instances, all_uris_in_class_unique, individuals_not_unique
    return has_instances, all_uris_in_class_unique


# ttl_file = "oscars_expose_updated5.ttl"
# g = Graph()
# g.parse(ttl_file, format="turtle")


# cls = URIRef("http://example.org/ontology#Director")
# print(get_primary_key(g))

# ontology_has_individuals(g)


# count_instances_and_check_uniqueness(g)

### R11 - Annotations

fasst alle Annotations einer Klasse/Proeprty/Individuum zusammen

In [357]:
def get_annotaions(graph, element):
    """_summary_

    :param graph: _description_
    :type graph: _type_
    :param part: Class, Property or Individual
    :type part: _type_
    """
    g = graph

    annotations = {}

    labels = []
    comments = []
    see_also = []
    is_defined_by = []
    equivalent_name = []
    same_as = []
    custom_annotations = {}
    version_info = []
    prior_version = []
    backward_compatible_with = []
    incompatible_with = []
    deprecated = []

    for _, p, o in g.triples((element, None, None)):
        o_new = remove_prefix(o).replace("'", "''")
        if p == RDFS.label:
            labels.append(o_new)

        elif p == RDFS.comment:
            comments.append(o_new)

        elif p == RDFS.seeAlso:
            see_also.append(o_new)

        elif p == RDFS.isDefinedBy:
            is_defined_by.append(o_new)
    
        elif p == OWL.sameAs:
            same_as.append(o_new)
        
        elif g.value(p, RDF.type) == OWL.AnnotationProperty:
            custom_annotations.setdefault(p, []).append(o_new)
        # elif g.value(p, RDF.type) == OWL.AnnotationProperty:
        #     custom_annotations[p] = o_new
        
        elif p == OWL.versionInfo:
            version_info.append(str(o))

        elif p == OWL.priorVersion:
            prior_version.append(o_new)

        elif p == OWL.backwardCompatibleWith:
            backward_compatible_with.append(o_new)

        elif p == OWL.incompatibleWith:
            incompatible_with.append(o_new)

        elif p == OWL.deprecated:
            deprecated.append(str(o))


    annotations[element] = {
        "rdfs:label": labels if labels else None,
        "rdfs:comment": comments if comments else None,
        "rdfs:seeAlso": see_also if see_also else None,
        "rdfs:isDefinedBy": is_defined_by if is_defined_by else None,
        "owl:sameAs": same_as if same_as else None,
        "owl:versionInfo": version_info if version_info else None,
        "owl:priorVersion": prior_version if prior_version else None,
        "owl:backwardCompatibleWith": backward_compatible_with if backward_compatible_with else None,
        "owl:incompatibleWith": incompatible_with if incompatible_with else None,
        "owl:deprecated": deprecated if deprecated else None,
        "customAnnotations": custom_annotations if custom_annotations else None,
    }

    return annotations

# ttl_file = "oscars_expose_updated4.ttl"
# g = Graph()
# g.parse(ttl_file, format="turtle")

# prop = URIRef("http://example.org/ontology#wonAwardInYear")
# annotations = get_annotaions(g, prop)

# all_annotations = {}

# # for prop in all_properties():
# #     all_annotations

# for props, infos in annotations.items():
#     print(f"Property: {props}")
#     for label, info in infos.items():
#         print(f"     - {label}: {info}")

# print(annotations)
# # print(prop in all_classes())

### R9 - Datentyp-Inferenz

Bestimmt für eine Property Datentypen anhand bestimmter Keywords in der URI, in Labeln oder in Kommentaren. Und zählt wie oft diese jeweils ausgewehlt werden.

In [358]:
def infer_datatypes_by_keywords(graph, prop):
    """
    Liefert ein Dictionary mit XSD-Datentypen als Schlüssel und der Anzahl
    der erkannten Schlüsselwörter als Werte.
    
    :param prop: Der vollständige Property-URI oder -Name.
    :return: Dict {XSD.typ: count, ...}
    """
    g = graph
    
    keyword_counts = Counter()
    labels_comments = []

    for props, annotations in get_annotaions(g, prop).items():
        for keys, annotation in annotations.items(): 
            if annotation != None:
                if keys != "customAnnotations":
                    labels_comments.extend(annotation)
                if keys == "customAnnotations":
                    for custom_anno, value in annotation.items():
                        labels_comments.extend(value)
                        

    p_clean = remove_prefix(prop).lower()
    labels_comments.append(p_clean)

    for keywords in labels_comments:
        keywords = keywords.lower()

        if "date" in keywords:
            keyword_counts[XSD.date] += 1

        if "time" in keywords:
            keyword_counts[XSD.time] += 1

        if "year" in keywords:
            keyword_counts[XSD.gYear] += 1

        if "time" in keywords and "date" in keywords:
            keyword_counts[XSD.dateTime] += 1

        if any(k in keywords for k in ["id", "uri", "url", "link"]):
            keyword_counts[XSD.anyURI] += 1

        if any(k in keywords for k in ["count", "number", "total", "size"]):
            keyword_counts[XSD.integer] += 1

        if any(k in keywords for k in ["flag", "valid", "enabled"]):
            keyword_counts[XSD.boolean] += 1

        if any(k in keywords for k in ["price", "amount", "value", "score"]):
            keyword_counts[XSD.decimal] += 1

        if any(k in keywords for k in ["name", "label", "title", "code", "type", "category", "about"]):
            keyword_counts[XSD.string] += 1

    return keyword_counts

ttl_file = "oscars_expose_updated4.ttl"
g = Graph()
g.parse(ttl_file, format="turtle")
prop = URIRef("http://example.org/ontology#wonAwardInYear")
infer_datatypes_by_keywords(g, prop)



Counter({rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#gYear'): 4,
         rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#date'): 2})

Bestimmt den Datatype eines Objektswerts

In [359]:
def infer_datatypes_by_value(object):
    """
    Erkennung des Werts als logischer Typ (STRING, INT, etc.) in XSD Format

    :param object: Object Value
    :return: TYPE des Values als XSD Type
    """
    byValue = set()

    # Sprachliteral (z. B. "Beispiel"@de)
    if isinstance(object, Literal) and object.language:
        byValue.add(XSD.language)  

    o_clean = remove_prefix(object).strip()

    # Boolean (auch einfache Varianten wie 0/1, yes/no)
    if o_clean.lower() in {"true", "false", "1", "0", "yes", "no", "on", "off"}:
        byValue.add(XSD.boolean)

    # gYear (vierstellige Zahl und startet mit 19 oder 20)
    if re.fullmatch(r"(19|20)\d{2}", o_clean):
        byValue.add(XSD.gYear)

    # gYearMonth (z. B. 2023-05)
    if re.fullmatch(r"(19|20)\d{2}-\d{2}", o_clean):
        byValue.add(XSD.gYearMonth)

    # Integer (ganze Zahl mit optionalem Minus)
    if re.fullmatch(r"-?\d+", o_clean):
        byValue.add(XSD.integer)

    # Decimal (mit Punkt, optional negativ)
    elif re.fullmatch(r"-?\d+\.\d+", o_clean):
        byValue.add(XSD.decimal)

    # Double (Exponentialschreibweise z. B. 1.23e4)
    elif re.fullmatch(r"-?\d+\.\d+[eE][+-]?\d+", o_clean):
        byValue.add(XSD.double)

    # Time (z. B. 12:30 oder 12:30:00)
    if re.fullmatch(r"\d{2}:\d{2}(:\d{2}(\.\d{1,6})?)?", o_clean):
        byValue.add(XSD.time)

    # AnyURI (http/https-URL)
    if re.match(r"^https?://[^\s]+$", object):
        byValue.add(XSD.anyURI)
    
    # Duration (z. B. PT2H30M, P1Y)
    if re.fullmatch(r"P(T)?(\d+[YMDHMS])+", o_clean):
        byValue.add(XSD.duration)

    # Date / DateTime (z. B. 2023-01-01 oder 2023-01-01 12:30:00)
    dt = 0
    try:
        dt = parse(o_clean, fuzzy=False)  
    except Exception:
        dt = None
    if dt is not None:
        has_time = dt.hour != 0 or dt.minute != 0 or dt.second != 0
        if has_time:
            byValue.add(XSD.dateTime)     
        else: 
            byValue.add(XSD.date)

    # default Wert String      
    if not byValue:
        byValue.add(XSD.string) 

  
    return byValue
    


Bestimmt Datatypes der Individuen einer Property. Per Range, per Keywords, per Wert des Literals, per Wert des Objekts

In [360]:
def extract_property_datatypes(graph, prop, classExpressions):
    """_summary_

    :param graph: _description_
    :param prop: _description_
    :param classExpressions: _description_
    :return: _description_
    """
    g = graph
    usedInClassExpression = classExpressions
    # usedInClassExpression = extract_property_restrictions(g)

    datatypes = {}

    byLiteral = Counter()
    byValue = Counter()
    byRestriction = Counter()

    # Alle Objeket auf die die Property prop zeigt
    objects_referred_by_prop = set(o for _, _, o in g.triples((None, prop, None)))

    # by rdfs:range
        # bestimmt die Range der Property prop
    range_ = g.value(prop, RDFS.range)

    # by Keywords
        # überprüft die URI, Labels (rdfs:label/andere Label) der Property prop auf festgelegte Keywords
        # besonders mit Domainwissen zur Ontologie kann dies ein guter Weg sein einen Datatype zu bestimmen
    byKeywords = infer_datatypes_by_keywords(g, prop)  # bleibt ein Set

    # by Restrictions
        # Schaut ob bei someValuesFrom und allValuesFrom ein Datatype angegeben ist.
    if prop in usedInClassExpression:
        for class_expression in usedInClassExpression[prop]:
            if class_expression["predicate"] in (OWL.someValuesFrom, OWL.allValuesFrom, OWL.hasValue):
                value = class_expression["value"]
                if g.value(value, RDF.type) == RDFS.Datatype:
                    byRestriction[value] += 1
                if str(value).startswith(str(XSD)):
                    byRestriction[value] += 1
                if isinstance(value, Literal):
                    byRestriction[value.datatype] += 1
    

    for o in objects_referred_by_prop:
        # by Literal Datatype
            # Bestimmt den Typ von Literalen sofern diese auch wirklich einen Typ angehängt haben
        if isinstance(o, Literal): 
            if o.datatype is not None:
                byLiteral[o.datatype] += 1
            if o.language is not None:
                byLiteral[XSD.language] += 1

        # by Value
            # Bestimmt je Objekt anhand der tatsächlichen Werte passende Datentypen
        inferred = infer_datatypes_by_value(o)
        if isinstance(inferred, set):
            for dtype in inferred:
                byValue[dtype] += 1
        elif inferred:  # falls infer_datatypes_by_value nur einen Typ zurückgibt
            byValue[inferred] += 1


    # Add combined count
    combined = Counter()
    for sub_counter in [byLiteral, byValue, byRestriction, byKeywords]:
        combined.update(sub_counter)

    # Zähle Range, wenn vorhanden
    if range_:
        if str(range_).startswith(str(XSD)):
            combined[range_] += 1
        if g.value(range_, RDF.type) == RDFS.Datatype:
            combined[range_] += 1


    datatypes[prop] = {
        "byRange": range_ if g.value(prop, RDF.type) == OWL.DatatypeProperty else None,
        "byLiteral": byLiteral if byLiteral else None,
        "byValue": byValue if byValue else None,
        "byRestriction": byRestriction if byRestriction else None,
        "byKeyword": byKeywords if byKeywords else None,
        "combinedCount": combined if combined else None
    }

    return datatypes


# ttl_file = "oscars_expose_updated4.ttl"

# g = Graph()
# g.parse(ttl_file, format="turtle")

# classExpression = extract_property_restrictions(g)
# prop = URIRef("http://example.org/ontology#boxOffice")
# datatypes = extract_property_datatypes(g, prop, classExpression)

# # print(datatypes)
# for props, infos in datatypes.items():
#     print(f"Property: {props}")
#     for label, info in infos.items():
#         print(f"     - {label}: {info}")

# print(g.triples((None, OWL.EquivalentClasses, None)))

In [361]:
def extract_custom_datatypes(graph):
    """
    Extrahiert benutzerdefinierte rdfs:Datatype-Definitionen mit owl:withRestrictions.
    
    :param g: rdflib.Graph mit geladener Ontologie
    :return: dict { datatype_uri: { "base": XSD-Typ, "restrictions": { facet: value, ... } } }
    """
    g = graph

    datatypes = {}
    
    for datatype in g.subjects(RDF.type, RDFS.Datatype):
        base = g.value(datatype, OWL.onDatatype)
        restrictions = {}
        
        for rest_node in g.items(g.value(datatype, OWL.withRestrictions)):
            for facet, value in g.predicate_objects(rest_node):
                if isinstance(value, Literal):
                    restrictions[facet] = value
        
        datatypes[datatype] = {
            "base": base,
            "restrictions": restrictions
        }

    return datatypes

# ttl_file = "oscars_expose_updated4.ttl"
# g = Graph()
# g.parse(ttl_file, format="turtle")

# restrictions = extract_custom_datatypes(g)[URIRef('http://example.org/ontology#PositiveInt')]["restrictions"]
# print(extract_custom_datatypes(g))

Nimmt einen XSD Datatype und übersetzt ihn in einen SQL Datatype.

In [362]:
def map_xsd_to_sql(graph, prop, xsd_type):
    """_summary_

    :param xsd_type: _description_
    :type xsd_type: _type_
    """
    xsd_to_sql = {
    # aus byValue und byKeywords:
        XSD.string: "VARCHAR",
        XSD.boolean: "BOOLEAN",
        XSD.integer: "INTEGER",
        XSD.decimal: "NUMERIC",
        XSD.double: "DOUBLE PRECISION",
        XSD.date: "DATE",
        XSD.time: "TIME",
        XSD.dateTime: "TIMESTAMP",
        XSD.duration: "INTERVAL",
        XSD.gYear: "NUMERIC(4,0)",
        XSD.gYearMonth: "VARCHAR(7)",
        XSD.anyURI: "VARCHAR",
        XSD.language: "VARCHAR",
        
        # Zusätzliche mögliche Datentypen für byRange und byLiteral:
        XSD.positiveInteger: "INTEGER",
        XSD.positiveInt: "Test",
        XSD.positiveInteger: "INTEGER",
        XSD.nonNegativeInteger: "INTEGER",
        XSD.negativeInteger: "INTEGER",
        XSD.byte: "SMALLINT",
        XSD.short: "SMALLINT",
        XSD.int: "INTEGER",
        XSD.long: "BIGINT",
        XSD.unsignedByte: "SMALLINT",
        XSD.unsignedShort: "SMALINT",
        XSD.unsignedInt: "INTEGER",
        XSD.unsignedLong: "INTEGER",
        XSD.float: "REAL",
        XSD.token: "VARCHAR",
        XSD.normalizedString: "VARCHAR",
        XSD.hexBinary: "BYTEA",
        XSD.Name: "VARCHAR",
        XSD.NCName: "VARCHAR",
        XSD.NMTOKEN: "VARCHAR",
        XSD.gDay: "VARCHAR(7)",
        XSD.gMonth: "VARCHAR(4)",
        XSD.gMonthDay: "VARCHAR(8)"
    }

    if str(xsd_type).startswith(str(XSD)):
        sql_type = xsd_to_sql.get(xsd_type, "VARCHAR")
    
    else:
        sql_type = (xsd_type, "rdfs:Datatype")

    g = graph
    custom_datytypes = extract_custom_datatypes(g)

    if sql_type[1] == "rdfs:Datatype":
        custom_type = custom_datytypes[sql_type[0]]
        sql_type = { 
            "sql_datatype": xsd_to_sql.get(custom_type["base"], "VARCHAR"),
            "restrictions": custom_type["restrictions"] 
        }


    if sql_type == "VARCHAR":
        values =  g.objects(None, prop)
        max_len = max((len(value) for value in values), default=0)
        if max_len > 255:
            sql_type = "VARCHAR"
        elif max_len <= 50:
            sql_type = "VARCHAR(50)"
        elif max_len <= 100:
            sql_type = "VARCHAR(100)"
        else:
            sql_type = "VARCHAR(255)"


    if sql_type == "NUMERIC":
        values =  g.objects(None, prop)
        scale = set()
        before_decimal = set()
        for value in values:
            dec = Decimal(str(value))  # vermeidet float-Rundungsfehler
            sign, digits, exponent = dec.as_tuple()
            digits = list(digits)
            
            scale.add(-exponent if exponent < 0 else 0)
            before_decimal.add(len(digits) + exponent)
        s = max(scale)
        p = max(before_decimal) + s
        sql_type = (f"NUMERIC({p}, {s})")

    return sql_type

# ttl_file = "oscars_expose_updated4.ttl"

# g = Graph()
# g.parse(ttl_file, format="turtle")

# prop = URIRef("http://example.org/ontology#boxOffice")
# xsd_type = URIRef("http://example.org/ontology#PositiveInt")
# # xsd_type = URIRef("http://www.w3.org/2001/XMLSchema#string")

# (map_xsd_to_sql(g, prop, xsd_type))

Bestimmt für eine Proeprty aus allen gefundenen XSD Datatypen einen finalen XSD Datatype.

In [363]:
def recommended_datatype(graph, prop):
    """_summary_

    :param g: _description_
    :type g: _type_
    :param datatypes: _description_
    :type datatypes: _type_
    """
    g = graph
    usedInClassExpression = extract_property_restrictions(g)

    all_datatypes = extract_property_datatypes(g, prop, usedInClassExpression)

    for props, info in all_datatypes.items():
        combined = info["combinedCount"]
        byRange = info["byRange"]
        byLiteral = info["byLiteral"]
        byValue = info["byValue"]
        byRestriction = info["byRestriction"]
        byKeywords = info["byKeyword"]

    
    recommended = {}

    # byRange zuerst
    if byRange:
        recommended = {
            "XSDdatatype": byRange,
            "SQLdatatype": map_xsd_to_sql(graph, prop, byRange), 
            "source": "byRange"
        }
    
    # sonst andere Quellen
    else:
        for source_name, type_source in [
            ("byLiteral", byLiteral),
            ("byValue", byValue),
            ("byRestriction", byRestriction),
            ("byKeyword", byKeywords)
        ]:
            if type_source and isinstance(type_source, dict) and len(type_source) >= 2:
                top2 = Counter(type_source).most_common(2)
                if top2[0][1] == top2[1][1]:
                    top_combined = combined and Counter(combined).most_common(1)[0][0]
                    top2_keys = (top2[0][0], top2[1][0])
                    value = top_combined if top_combined in top2_keys else top2[0][0]
                    recommended = {
                        "XSDdatatype": value,
                        "SQLdatatype": map_xsd_to_sql(graph, prop, value),
                        "source": source_name
                    }
                    break
            elif type_source:
                value = Counter(type_source).most_common(1)[0][0]
                recommended = {
                    "XSDdatatype": value,
                    "SQLdatatype": map_xsd_to_sql(graph, prop, value),
                    "source": source_name
                }
                break

    if not recommended:
        recommended = {
            "XSDdatatype": "No datatype found",
            "SQLdatatype": "VARCHAR",
            "source": None
        }

    return recommended


# ttl_file = "oscars_expose_updated4.ttl"
# g = Graph()
# g.parse(ttl_file, format="turtle")

# prop = URIRef("http://example.org/ontology#boxOffice")
# properties = get_all_properties(g)

# recommend = recommended_datatype(g, prop)

# recommend
    

### R2/R8 - Klassen-Naming/Property-Naming

Hilfstabellen für get_sql_name()

In [ ]:
def is_known_word(word):
    word = word.lower()
    return (
        word in english_words or
        dict_en.check(word)
    )


def split_camel_case(label):
    return re.findall(r'[A-ZÄÖÜ]?[a-zäöüß]+|[A-ZÄÖÜ]+(?=[A-ZÄÖÜ]|$)', label)


def is_label_meaningful(label):
    parts = split_camel_case(clean_value(label))
    if not parts:
        return False
    real_words = [w for w in parts if is_known_word(w)]
    return len(real_words) / len(parts) >= 0.5

Bestimmt den Namen für eine Klasse oder Property die später in SQL als Tabellenname oder Attributname verwendet wird. \
Reinfolge: Preferred Custom-Label --> rdfs:label --> Custom-Label --> Cleand URI

In [490]:
def get_sql_name(graph, element):
    """_summary_

    :param g: _description_
    :type g: _type_
    :param element: _description_
    :type element: _type_
    """
    g = graph
    sql_name = {}

    if element is None:
        return {}

    for elem, annotations in get_annotaions(g, element).items():
        label = annotations["rdfs:label"]
        custom_annotations = annotations["customAnnotations"]
        uri = remove_prefix(element)

        if custom_annotations:
            for key in custom_annotations:
                clean_key = clean_value(key)
                value = list(custom_annotations[key])[0]
                if "label" in clean_key and any(p in clean_key for p in ["preferred", "pref", "pf", "prefer", "preferable", "fav", "favour", "favourite"]):
                    if value and is_label_meaningful(value) and len(clean_value(value)) <= 40:
                        sql_name[element] = {
                            "sql_name": value,
                            "source": "PreferredCustomLabel"
                        }
                        break

        if element not in sql_name and label and is_label_meaningful(label) and len(clean_value(label)) <= 40:
            sql_name[element] = {
                "sql_name": label[0],
                "source": "Label"
            }
        
        if element not in sql_name and custom_annotations:
            for key in custom_annotations:
                clean_key = clean_value(key)
                value = next(iter(custom_annotations[key]), None)
                if value and is_label_meaningful(value) and len(clean_value(value)) <= 40:
                    if "label" in clean_key:
                        sql_name[element] = {
                            "sql_name": value,
                            "source": "CustomLabel"
                        }
                        break

        if element not in sql_name:
            sql_name[element] = {
                "sql_name": clean_value(uri),
                "source": "URI"
            }


    match = re.match(r'^(\d+)(.*)', sql_name[element]["sql_name"])
    if match:
        nummer = match.group(1)  # die Zahlen am Anfang
        rest = match.group(2)    # der Rest des Strings
        sql_name[element]["sql_name"] = clean_value(rest + nummer)
    else:
        sql_name[element]["sql_name"] = clean_value(sql_name[element]["sql_name"])

    return sql_name



ttl_file = "oscars_expose_updated4.ttl"
g = Graph()
g.parse(ttl_file, format="turtle")

element = URIRef("http://example.org/ontology#directorID")
# element = URIRef("http://www.wikidata.org/entity/Q160060")

sql_name = get_sql_name(g, element)

sql_name


{rdflib.term.URIRef('http://example.org/ontology#directorID'): {'sql_name': 'wiki_data_id_of_a_director',
  'source': 'Label'}}

Prüft ob alle Klassen-, Property- und Instanznamen einzigartig sind.

In [474]:
def check_uniqueness_of_names(graph, all_classes, all_properties):
    """_summary_

    :param g: _description_
    :type g: _type_
    """
    g = graph

    all_table_names = {}

    for cls in all_classes:
        name_dict = get_sql_name(g, cls)[cls]
        class_name = name_dict["sql_name"]
        name_sorce = name_dict["source"]

        if all_table_names.get(class_name):
            all_table_names[class_name].append((cls, name_sorce))
        else:
            all_table_names[class_name] = [(cls, name_sorce)]
    

    return all_table_names

ttl_file = "oscars_expose_updated4.ttl"
g = Graph()
g.parse(ttl_file, format="turtle")

all_classes = get_all_classes(g)
all_properties = get_all_properties(g)

test = check_uniqueness_of_names(g, all_classes, all_properties)
# for a, b in test.items():
#     print(len(b))

test

{'award_winner2': [(rdflib.term.URIRef('http://example.org/ontology#AwardWinner2'),
   'URI')],
 'person': [(rdflib.term.URIRef('http://example.org/ontology#Person'), 'URI')],
 'award_winner3': [(rdflib.term.URIRef('http://example.org/ontology#AwardWinner3'),
   'URI')],
 'film3': [(rdflib.term.URIRef('http://example.org/ontology#Film3'), 'URI')],
 'princess2': [(rdflib.term.URIRef('http://example.org/ontology#Princess2'),
   'URI')],
 'director': [(rdflib.term.URIRef('http://example.org/ontology#Director'),
   'Label')],
 'film': [(rdflib.term.URIRef('http://example.org/ontology#Film5'),
   'PreferredCustomLabel'),
  (rdflib.term.URIRef('http://example.org/ontology#Film2'),
   'PreferredCustomLabel'),
  (rdflib.term.URIRef('http://example.org/ontology#Film'), 'URI')],
 'movie123': [(rdflib.term.URIRef('http://example.org/ontology#Movie'),
   'PreferredCustomLabel')],
 'producer': [(rdflib.term.URIRef('http://example.org/ontology#Producer'),
   'URI')],
 'movie_production': [(rdflib.te

In [489]:
def get_all_lable_proeprties(graph):
    """_summary_

    :param g: _description_
    :type g: _type_
    """
    g = graph

    rdfs_label = [RDFS.label]
    preferred_label = []
    other_label = []

    all_label = []

    preferred_keywords = ["preferred", "pref", "pf", "prefer", "preferable", "fav", "favour", "favourite"]

    for s in g.subjects(RDF.type, OWL.AnnotationProperty):
        if "label" in s.lower() and any(p in s.lower() for p in preferred_keywords):
            preferred_label.append(s)
        if "label" in s.lower() and not any(p in s.lower() for p in preferred_keywords):
            other_label.append(s)
    
    all_label.extend(preferred_label)
    all_label.extend(rdfs_label)
    all_label.extend(other_label)

    return all_label



ttl_file = "oscars_expose_updated4.ttl"
# ttl_file = "Wine_reasoning.ttl"
g = Graph()
g.parse(ttl_file, format="turtle")

get_all_lable_proeprties(g)

[rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label')]

In [510]:
def generate_unique_names(g, elements_by_group, label_predicates=None):
    """
    Erzeugt ein zentrales Namensregister für Klassen, Properties und Individuen
    mit eindeutigen Namen und Duplikatbehandlung.
    
    :param g: rdflib.Graph
    :param elements_by_group: Dict mit Gruppen: {"classes": [...], "properties": [...], "individuals": [...]}
    :param preferred_label_predicates: Liste von Prädikaten für Labels nach Priorität
    :return: Dict mit eindeutigen Namen pro Gruppe
    """
    # if preferred_label_predicates is None:
    #     preferred_label_predicates = [
    #         URIRef("http://www.w3.org/2004/02/skos/core#prefLabel"), 
    #         RDFS.label
    #     ]
    
    # def remove_prefix(uri: URIRef) -> str:
    #     """Gibt den letzten Teil der URI zurück."""
    #     uri_str = str(uri)
    #     return uri_str.split("#")[-1].split("/")[-1]

    def generate_candidate_labels(uri):
        """Gibt alle möglichen Kandidatennamen nach Priorität zurück."""
        labels = []
        for pred in label_predicates:
            for lbl in g.objects(uri, pred):
                if is_label_meaningful(lbl) and len(clean_value(lbl)) <= 40:
                    match = re.match(r'^(\d+)(.*)', lbl)
                    if match:
                        nummer = match.group(1)  # die Zahlen am Anfang
                        rest = match.group(2)    # der Rest des Strings
                        labels.append(clean_value(rest + nummer))
                    else:
                        labels.append(clean_value(lbl))       
        # Fallback auf URI-Fragment
        labels.append(clean_value(uri))
        return labels

    # Hauptspeicher für Namen
    names = {group: {} for group in elements_by_group}
    used_names = {group: set() for group in elements_by_group}

    for group, elements in elements_by_group.items():
        for uri in elements:
            # Alle Namenskandidaten sammeln
            if group != "individuals":
                candidates = generate_candidate_labels(uri)
            else:
                candidates = [remove_prefix(uri)]

            # Wähle ersten eindeutigen Kandidaten
            final_name = None
            for candidate in candidates:
                if candidate not in used_names[group]:
                    final_name = candidate
                    break

            # Wenn immer noch kein eindeutiger Name gefunden
            if final_name is None:
                prefix_name = add_short_prefix(uri)
                if prefix_name not in used_names[group]:
                    final_name = prefix_name
                else: 
                    suffix = 1
                    while f"{prefix_name}_{suffix}" in used_names[group]:
                        suffix += 1
                    final_name = f"{prefix_name}_{suffix}"

            # Speichern
            names[group][uri] = final_name
            used_names[group].add(final_name)

    return names


# ttl_file = "oscars_expose_updated4.ttl"
ttl_file = "Wine_reasoning2.ttl"
g = Graph()
g.parse(ttl_file, format="turtle")

elements_by_group = get_elements_by_group(g)
lable_predicates = get_all_lable_proeprties(g)

generate_unique_names(g, elements_by_group, lable_predicates)

{'classes': {rdflib.term.URIRef('http://www.w3.org/TR/2003/PR-owl-guide-20031209/wine#RedBordeaux'): 'red_bordeaux',
  rdflib.term.URIRef('http://www.w3.org/TR/2003/PR-owl-guide-20031209/wine#WineFlavor'): 'wine_flavor',
  rdflib.term.URIRef('http://www.w3.org/TR/2003/PR-owl-guide-20031209/wine#DryWine'): 'dry_wine',
  rdflib.term.URIRef('http://www.w3.org/TR/2003/PR-owl-guide-20031209/food#NonSpicyRedMeat'): 'non_spicy_red_meat',
  rdflib.term.URIRef('http://www.w3.org/TR/2003/PR-owl-guide-20031209/food#Fish'): 'fish',
  rdflib.term.URIRef('http://www.w3.org/TR/2003/PR-owl-guide-20031209/food#Shellfish'): 'shellfish',
  rdflib.term.URIRef('http://www.w3.org/TR/2003/PR-owl-guide-20031209/food#Fruit'): 'fruit',
  rdflib.term.URIRef('http://www.w3.org/TR/2003/PR-owl-guide-20031209/food#SpicyRedMeat'): 'spicy_red_meat',
  rdflib.term.URIRef('http://www.w3.org/TR/2003/PR-owl-guide-20031209/food#EdibleThing'): 'edible_thing',
  rdflib.term.URIRef('http://www.w3.org/TR/2003/PR-owl-guide-2003

### R5 - Äquivalenzklassenbeziehung (einfach)


Hilfsfunktionen \
Kombiniert mehrere gleiche Dictionaries.


In [366]:
def combine_annotation_values(annotation_dict):
    combined = {}

    for anno_dict in annotation_dict:
        for cls, annotations in anno_dict.items():
            for prop, value in annotations.items():
                if value is None:
                    continue

                # Listen zusammenführen
                if isinstance(value, list):
                    combined.setdefault(prop, [])
                    for val in value:
                        if val not in combined[prop]:
                            combined[prop].append(f"{val} ({remove_prefix(cls)})")
                
                elif isinstance(value, dict):
                    combined.setdefault(prop, {})  # Stelle sicher, dass ein dict existiert

                    for subkey, subval in value.items():
                        subval_str = ", ".join(subval) if isinstance(subval, list) else str(subval)
                        annotated_val = f"{subval_str} ({remove_prefix(cls)})"

                        combined[prop].setdefault(subkey, [])
                        if annotated_val not in combined[prop][subkey]:
                            combined[prop][subkey].append(annotated_val)

                # Bool: True gewinnt
                elif isinstance(value, bool):
                    combined[prop] = combined.get(prop, False) or value

                # Alle anderen Werte (None, string, URIRefs, etc.)
                else:
                    if prop not in combined:
                        combined[prop] = value
                    elif combined[prop] is None and value is not None:
                        combined[prop] = value
                    # sonst: existierender Wert bleibt, None wird ignoriert
    return combined

# defi = [{URIRef('http://example.org/ontology#Film5'): {'rdfs:label': None, 'rdfs:comment': ['EIn Film5 Kommentar'], 'rdfs:seeAlso': None, 'rdfs:isDefinedBy': None, 'owl:sameAs': None, 'owl:versionInfo': None, 'owl:priorVersion': None, 'owl:backwardCompatibleWith': None, 'owl:incompatibleWith': None, 'owl:deprecated': None, 'customAnnotations': {URIRef('http://example.org/ontology#labelPref'): ['Filme 5']}}}, {URIRef('http://example.org/ontology#Director'): {'rdfs:label': ['Director'], 'rdfs:comment': ['Macht einen Film'], 'rdfs:seeAlso': None, 'rdfs:isDefinedBy': None, 'owl:sameAs': None, 'owl:versionInfo': None, 'owl:priorVersion': None, 'owl:backwardCompatibleWith': None, 'owl:incompatibleWith': None, 'owl:deprecated': None, 'customAnnotations': {URIRef('http://example.org/ontology#otherLabel'): ['Director']}}}]
# combine_annotation_values(defi)

In [368]:
def get_annotations_of_equivalent_classes(graph, equivalent_class_group):
    """_summary_

    :param graph: _description_
    :type graph: _type_
    :param prop: _description_
    :type prop: _type_
    """
    g = graph

    # cls_annotations = get_annotaions(g, cls)

    equivalent_annotations_combined = {}

    for cls, equi_group in equivalent_class_group.items():
        equivalent_annotations = []
        for equivalent_class in equi_group:
            equivalent_class_annotations = get_annotaions(g, equivalent_class)
            equivalent_annotations.append(equivalent_class_annotations) 

        equivalent_annotations_combined[cls] = combine_annotation_values(equivalent_annotations)

    return equivalent_annotations_combined

    

# ttl_file = "oscars_expose_updated4.ttl"
# g = Graph()
# g.parse(ttl_file, format="turtle")

# cls = URIRef("http://example.org/ontology#Prinzessin")

# a = build_equivalent_class_map(g)[1]
# a

# get_annotations_of_equivalent_classes(g, a)



Kombiniert Äquivalenzklassennamen zu einem

In [ ]:
def get_equivalent_sql_name(graph, equivalent_class):
    g = graph

    names = ["equivalent_table"]
    
    for equivalent_class in equivalent_class:
        equivalent_class_name = get_sql_name(g, equivalent_class)[equivalent_class]["sql_name"]
        names.append(equivalent_class_name)
    
    # names.append(cls_name)

    return "_".join(names)
    

# ttl_file = "oscars_expose_updated4.ttl"
# g = Graph()
# g.parse(ttl_file, format="turtle")

# cls = URIRef("http://example.org/ontology#Director")
# cls = {URIRef('http://example.org/ontology#Regessure'), URIRef('http://example.org/ontology#Director'), URIRef('http://example.org/ontology#Movie'), URIRef('http://example.org/ontology#Film5'), URIRef('http://example.org/ontology#Writer'), URIRef('http://example.org/ontology#Film2')}

# get_equivalent_sql_name(g, cls)

In [429]:
def build_equivalent_class_map(graph):
    """_summary_

    :param graph: _description_
    :type graph: _type_
    """
    g = graph
    
    class_groups = []
    equivalent_class_group = {}

    visited = set()
    for cls, o in g.subject_objects(OWL.equivalentClass):
        if isinstance(o, BNode): continue
        if cls in visited: continue
        group = set()
        stack = [cls]
        while stack:
            current = stack.pop()
            if current in group: continue
            group.add(current)
            visited.add(current)
            for neighbor in g.objects(current, OWL.equivalentClass):
                if isinstance(neighbor, URIRef): stack.append(neighbor)
            for neighbor in g.subjects(OWL.equivalentClass, current):
                if isinstance(neighbor, URIRef): stack.append(neighbor)
        if group:
            # print(type(group))
            class_groups.append(group)
            for gr in group:
                equivalent_class_group[gr] = group

    equivalent_class_map = {}

    # #### NOTE Variante 3/4 von R5 NOTE ####
    # for group in class_groups:
    #     if len(group) == 2:
    #         for cls in group:
    #             equivalent_class_map[cls] = [equivalent_class for equivalent_class in group if equivalent_class != cls][0]
    #     else:
    #         representative = sorted(group, key=lambda x: len(get_sql_name(g, x)[x]["sql_name"]))[0]
    #         for cls in group:
    #             equivalent_class_map[cls] = representative
    # #### NOTE Variante 3/4 von R5 NOTE ####


    #### NOTE Variante 5 von R5 NOTE ####
    max_length = (max(len(g) for g in class_groups) if class_groups else 0)
    if max_length <= 2:
        for group in class_groups:
            for cls in group:
                equivalent_class_map[cls] = get_equivalent_sql_name(g, group)
    else:
        for group in class_groups:
            representative = sorted(group, key=lambda x: len(get_sql_name(g, x)[x]["sql_name"]))[0]
            for cls in group:
                equivalent_class_map[cls] = representative
    #### NOTE Variante 5 von R5 NOTE ####

    
    equivalent_class_annotations = get_annotations_of_equivalent_classes(g, equivalent_class_group)

    return equivalent_class_map, equivalent_class_group, equivalent_class_annotations

ttl_file = "oscars_expose_updated4.ttl"
# ttl_file = "test_reasoning.ttl"
g = Graph()
g.parse(ttl_file, format="turtle")

cls = URIRef('http://example.org/ontology#Princess')
a = build_equivalent_class_map(g)[2]
a
# if cls in a.keys():
#     print(True)

{rdflib.term.URIRef('http://example.org/ontology#Film5'): {'rdfs:comment': ['EIn Film5 Kommentar (Film5)',
   'alle Filme mit min einem Oscar für Best Picture, Acting, Writing oder Directing (Movie)',
   'Macht einen Film (Director)',
   'Regessure wird glaube ich vollkommen anderes geschrieben (Regessure)'],
  'customAnnotations': {rdflib.term.URIRef('http://example.org/ontology#labelPref'): ['Fil (Film5)',
    'Movie123 (Movie)'],
   rdflib.term.URIRef('http://example.org/ontology#preferredLabel'): ['Movie321 (Movie)'],
   rdflib.term.URIRef('http://example.org/ontology#otherLabel'): ['Director (Director)',
    'BlaBlaBla (Regessure)']},
  'rdfs:label': ['Movie (Movie)', 'Director (Director)']},
 rdflib.term.URIRef('http://example.org/ontology#Movie'): {'rdfs:comment': ['EIn Film5 Kommentar (Film5)',
   'alle Filme mit min einem Oscar für Best Picture, Acting, Writing oder Directing (Movie)',
   'Macht einen Film (Director)',
   'Regessure wird glaube ich vollkommen anderes geschrieb

## Propety Infos

Bestimmt alle Properties, zugehörige Infos und groupiert sie wahlweise nach der Domain oder Range.

In [370]:
def extract_properties(graph, group_by):
    """
    Extrahiert Properties mit verschiedenen Infos aus einer RDF-Datei.
    Optional gruppiert nach Domain-Klasse.

    :param ttl_file: Pfad zur RDF-Datei (Turtle)
    :param group_by: Wenn "domain": Grupierung nach nach Domain-Klasse
                     Wenn "range": Grupierung nach nach Range-Klasse
                     Ansonsten: Keine Grupierung, nur Properties mit Infos
    :return: Wenn "domain": {Domain-Klasse: {Property: {"range": ..., "label": ..., "used":..., ...}}}
             Wenn "range": {Range-Klasse: {Property: {"domain": ..., "label": ..., "used":..., ...}}}
             Ansonsten: {Property: {"domain": ..., "range": ..., "label": ..., "used":..., ...}}
    """
    g = graph 
    # g = Graph()
    # g.parse(ttl_file, format="turtle")

    all_properties = get_all_properties(g)

    used_in_class_expression = extract_property_restrictions(g)

    used_in_intersection = set()
    for _, _, intersection_node in g.triples((None, OWL.intersectionOf, None)):
        if (intersection_node, RDF.first, None) in g:
            members = Collection(g, intersection_node)
            for member in members:
                if (member, RDF.type, OWL.Restriction) in g:
                    prop = g.value(member, OWL.onProperty)
                    if prop:
                        used_in_intersection.add(prop)

    # single_value_properties = get_singlevalue_properties(g, all_properties)

    prop_infos = {}

    # Alle Properties und ihre Infos sammeln
    for prop in all_properties:
    #Grundlegende Infos
        domain = g.value(prop, RDFS.domain)
        range_ = g.value(prop, RDFS.range)
        for prop, name in get_sql_name(g, prop).items():
            sql_name = name
        label = g.value(prop, RDFS.label)
        other_labels = []
        for _, p, o in g.triples((prop, None, None)):
            if "label" in clean_value(p):
                other_labels.append(remove_prefix(o))
        comment = g.value(prop, RDFS.comment)
        annotations = get_annotaions(g, prop)
        type_ = g.value(prop, RDF.type)
            # TODO nimmt immer den ersten Typ, sollte in der Regel zwar richtig sein, aber trotzdem
        used_in_classes = set(
            cls
            for s, _, _ in g.triples((None, prop, None))
            for cls in g.objects(s, RDF.type)
        )
        
        usage_count_by_class = {}
        for s, p, _ in g.triples((None, prop, None)):
            for cls in g.objects(s, RDF.type):
                if cls not in usage_count_by_class:
                    usage_count_by_class[cls] = 1
                else:
                    usage_count_by_class[cls] += 1

# TODO TODO TODO  
        # triple_count = used_prop.get(prop, 0)
            # entspricht auch der Anzahl aller Subjetzte bzw Objekte 
# TODO TODO TODO

        triples = set(g.triples((None, prop, None)))
            # gibt alle unterscheidlicher Triple zurück
            # total und distinct macht keinen Unterschied, da in RDF ein Graph die Menge von Tripeln ist
            # len(triples) entspricht auch der Anzahl aller Subjetzte bzw Objekte  
        distinct_subjects = set(s for s, _, _ in g.triples((None, prop, None)))
            # gibt alle unterschiedlichen Subjekte der Property zurück  
        distinct_objects = set(o for _, _, o in g.triples((None, prop, None)))
            # gibt alle unterschiedlichen Objekte der Propertie zurück 
        has_datatypes = extract_property_datatypes(g, prop, used_in_class_expression)
        recommended_datatypes = recommended_datatype(g, prop)
        single_valued = get_singlevalue_properties(g, prop)
        # single_valued = False
        # if prop in single_value_properties.keys():
        #     single_valued = True
        
        # multi_valued = False
        # for s in set(g.subjects(prop)):
        #     count = sum(1 for _ in g.objects(s, prop))
        #     if count > 1:
        #         multi_valued = True
        
    # Struktur & Hierarchie
        super_property_of = set(g.subjects(RDFS.subPropertyOf, prop))
        sub_property_of = set(g.objects(prop, RDFS.subPropertyOf))
        disjoint_property = set(g.subjects(prop, OWL.propertyDisjointWith))
        equivalent_property = set(g.subjects(prop, OWL.equivalentProperty))
        prop_chain = g.value(prop, OWL.propertyChainAxiom)
    # Eigenschaften
        inverse_of_subject = set(g.objects(prop, OWL.inverseOf))
        inverse_of_object = set(g.subjects(OWL.inverseOf, prop))
        inverse_of = inverse_of_subject.union(inverse_of_object)
        has_types = set(g.objects(prop, RDF.type))
        one_of = g.value(prop, OWL.oneOf)
        has_keys = []
        for s, _, o in g.triples((None, OWL.hasKey, None)):
            keys = list(Collection(g, o))
            if prop in keys:
                has_keys.append({
                "inKeyForClass": s,
                "key": keys
                })
    # Restrictions
        has_predicates = set()
        if prop in used_in_class_expression.keys():
            for predicates in used_in_class_expression[prop]:
                has_predicates.add(predicates["predicate"])
        has_container = set()
        if prop in used_in_class_expression.keys():
            for container in used_in_class_expression[prop]:
                has_container.add(container["logicalContainer"])
        has_location = set()
        if prop in used_in_class_expression.keys():
            for location in used_in_class_expression[prop]:
                has_location.add(location["restrictionLocation"])    


        prop_infos[prop] = {
        # Grundlegende Infos
            "rdf:type": type_ if type_ else None,
            "rdfs:domain": domain if domain else None,
            "rdfs:range": range_ if range_ else None,
            "sqlName": sql_name,
            # "rdfs:label": label if isinstance(label, Literal) else None,
            # "attributeName": clean_value(prop),
            
# TODO TODO TODO
            # "tripleCount": triple_count ,
# TODO TODO TODO

            "distinctSubjectCount": len(distinct_subjects),
            "distinctObjectCount": len(distinct_objects), 
            # "rdfs:comment": comment if isinstance(comment, Literal) else None, 

# TODO TODO TODO
            # "isUsed": prop in used_props,
                # TODO Schaut nicht in Restrictions nach.
# TODO TODO TODO

            "possibleDataTypes": has_datatypes[prop], 
            "recommendedDataType": recommended_datatypes ,
            "isSingleValued": single_valued[prop] if single_valued else False,
            "isMultiValued": True if not single_valued else False, 
                # TODO will ich wirklich alle außer funktionale Props als multivalued?

# TODO TODO TODO
            # "explicitProperty": prop in explicit_props,
# TODO TODO TODO            

            "usedByClasses": usage_count_by_class if usage_count_by_class else None,  
        # Struktur & Hierarchie
            "rdfs:subPropertyOf": [p for p in sub_property_of] if sub_property_of else None,
            "superPropertyOf": [p for p in super_property_of] if super_property_of else None,
            "owl:propertyDisjointWith": [p for p in disjoint_property] if disjoint_property else None,
            "owl:equivalentProperty": [p for p in equivalent_property] if equivalent_property else None, 
            "owl:propertyChainAxiom": list(Collection(g, prop_chain)) if prop_chain else None,
        # Eigenschaften
            "owl:inverseOf": [p for p in inverse_of] if inverse_of else None,
            "inverseOfAsSubject": [p for p in inverse_of_subject] if inverse_of_subject else None,
            "inverseOfAsObject": [p for p in inverse_of_object] if inverse_of_object else None, 
            "owl:FunctionalProperty": OWL.FunctionalProperty in has_types,
            "owl:InverseFunctional": OWL.InverseFunctionalProperty in has_types, 
            "owl:SymmetricProperty": OWL.SymmetricProperty in has_types,
            "owl:AsymmetricProperty": OWL.AsymmetricProperty in has_types,
            "owl:ReflexivProperty": OWL.ReflexiveProperty in has_types,
            "owl:IrreflexivProperty": OWL.IrreflexiveProperty in has_types,
            "owl:DeprecatedProperty": OWL.DeprecatedProperty in has_types,
            "FKCandidate": (range_, RDF.type, OWL.Class) in g or (range_, RDF.type, RDFS.Class) in g,
            "owl:oneOf": list(Collection(g, one_of)) if one_of else None,
            "owl:hasKey": has_keys if has_keys else None,
        # Restrictions
            "usedInClassExpression": used_in_class_expression[prop] if prop in used_in_class_expression.keys() else None, 
            "usedInClassExpressionCount": len(used_in_class_expression[prop]) if prop in  used_in_class_expression else 0,
            "usedInEqivalentClassDef": OWL.equivalentClass in has_location,
            "usedInSubClassOfDef": RDFS.subClassOf in has_location,
            "owl:hasValue": OWL.hasValue in has_predicates,
            "owl:someValuesFrom": OWL.someValuesFrom in has_predicates,
            "owl:allValuesFrom": OWL.allValuesFrom in has_predicates,
            "owl:hasSelf": OWL.hasSelf in has_predicates,
            "owl:minCardinality": OWL.minCardinality in has_predicates,
            "owl:maxCardinality": OWL.maxCardinality in has_predicates,
            "owl:cardinality": OWL.cardinality in has_predicates,
            "owl:minQualifiedCardinality": OWL.minQualifiedCardinality in has_predicates,
            "owl:maxQualifiedCardinality": OWL.maxQualifiedCardinality in has_predicates,
            "owl:qualifiedCardinality": OWL.qualifiedCardinality in has_predicates,
            "owl:intersectionOf": OWL.intersectionOf in has_container,
            "owl:unionOf": OWL.unionOf in has_container,
            "owl:complementOf": OWL.complementOf in has_container,
        # Annotations
            "annotations": annotations
        }   
        
    if group_by == "domain":
        # Gruppierung nach Domain-Klassen
        grouped_result_domain = {}
        for prop, info in prop_infos.items():
            domain_class = info["rdfs:domain"]
            if domain_class:
                grouped_result_domain.setdefault(domain_class, {})[prop] = {
                # Grundlegende Infos
                    "rdf:type": info["rdf:type"],
                    # "rdfs:domain": info["rdfs:domain"],
                    "rdfs:range": info["rdfs:range"],
                    "sqlName": info["sqlName"],
                    # "rdfs:label": info["rdfs:label"],
                    # "attributeName": info["attributeName"],

                    # TODO TODO TODO
                    # "tripleCount": info["tripleCount"],
                    # TODO TODO TODO

                    "distinctSubjectCount": info["distinctSubjectCount"],
                    "distinctObjectCount": info["distinctObjectCount"],
                    # "rdfs:comment": info["rdfs:comment"],

                    # TODO TODO TODO
                    # "isUsed": info["isUsed"],
                    # TODO TODO TODO

                    "possibleDataTypes": info["possibleDataTypes"],
                    "recommendedDataType": info["recommendedDataType"],
                    "isSingleValued": info["isSingleValued"],
                    "isMultiValued": info["isMultiValued"],
                    # TODO will ich wirklich alle außer funktionale Props als multivalued?

                    # TODO TODO TODO
                    # "explicitProperty": info["explicitProperty"],
                    # TODO TODO TODO

                    "usedByClasses": info["usedByClasses"],

                # Struktur & Hierarchie
                    "rdfs:subPropertyOf": info["rdfs:subPropertyOf"],
                    "superPropertyOf": info["superPropertyOf"],
                    "owl:propertyDisjointWith": info["owl:propertyDisjointWith"],
                    "owl:equivalentProperty": info["owl:equivalentProperty"],
                    "owl:propertyChainAxiom": info["owl:propertyChainAxiom"],

                # Eigenschaften
                    "owl:inverseOf": info["owl:inverseOf"],
                    "inverseOfAsSubject": info["inverseOfAsSubject"],
                    "inverseOfAsObject": info["inverseOfAsObject"],
                    "owl:FunctionalProperty": info["owl:FunctionalProperty"],
                    "owl:InverseFunctional": info["owl:InverseFunctional"],
                    "owl:SymmetricProperty": info["owl:SymmetricProperty"],
                    "owl:AsymmetricProperty": info["owl:AsymmetricProperty"],
                    "owl:ReflexivProperty": info["owl:ReflexivProperty"],
                    "owl:IrreflexivProperty": info["owl:IrreflexivProperty"],
                    "owl:DeprecatedProperty": info["owl:DeprecatedProperty"],
                    "FKCandidate": info["FKCandidate"],
                    "owl:oneOf": info["owl:oneOf"],
                    "owl:hasKey": info["owl:hasKey"],

                # Restrictions
                    "usedInClassExpression": info["usedInClassExpression"],
                    "usedInClassExpressionCount": info["usedInClassExpressionCount"],
                    "usedInEqivalentClassDef": info["usedInEqivalentClassDef"],
                    "usedInSubClassOfDef": info["usedInSubClassOfDef"],
                    "owl:hasValue": info["owl:hasValue"],
                    "owl:someValuesFrom": info["owl:someValuesFrom"],
                    "owl:allValuesFrom": info["owl:allValuesFrom"],
                    "owl:hasSelf": info["owl:hasSelf"],
                    "owl:minCardinality": info["owl:minCardinality"],
                    "owl:maxCardinality": info["owl:maxCardinality"],
                    "owl:cardinality": info["owl:cardinality"],
                    "owl:minQualifiedCardinality": info["owl:minQualifiedCardinality"],
                    "owl:maxQualifiedCardinality": info["owl:maxQualifiedCardinality"],
                    "owl:qualifiedCardinality": info["owl:qualifiedCardinality"],
                    "owl:intersectionOf": info["owl:intersectionOf"],
                    "owl:unionOf": info["owl:unionOf"],
                    "owl:complementOf": info["owl:complementOf"],

                # Annotations
                    "annotations": info["annotations"]
                }

        return grouped_result_domain

    if group_by == "range":
        # Gruppierung nach Range-Klassen
        grouped_result_domain = {}
        for prop, info in prop_infos.items():
            domain_class = info["rdfs:range"]
            if domain_class:
                grouped_result_domain.setdefault(domain_class, {})[prop] = {
            # Grundlegende Infos
                "rdf:type": info["rdf:type"],
                "rdfs:domain": info["rdfs:domain"],
                "rdfs:label": info["rdfs:label"],
                "cleandName": info["cleandName"],
                "tripleCount": info["tripleCount"],
                "distinctSubjectCount": info["distinctSubjectCount"],
                "distinctObjectCount": info["distinctObjectCount"],
                "rdfs:comment": info["rdfs:comment"],
                "isUsed": info["isUsed"],
                "possibleDataTypes": info["possibleDataTypes"],
                # TODO "recommendedDataType"
                # "recommendedDataType": info["recommendedDataType"],
                "isMultiValued": info["isMultiValued"],
                    # TODO will ich wirklich alle außer funktionale Props als multivalued?
                "explicitProperty": info["explicitProperty"],
                "usedByClasses": info["usedByClasses"],
            # Struktur & Hierarchie
                "rdfs:subPropertyOf": info["rdfs:subPropertyOf"],
                "superPropertyOf": info["superPropertyOf"],
                "owl:propertyDisjointWith": info["owl:propertyDisjointWith"],
                "owl:equivalentProperty": info["owl:equivalentProperty"],
                "owl:propertyChainAxiom": info["owl:propertyChainAxiom"],
            # Eigenschaften
                "owl:inverseOf": info["owl:inverseOf"],
                "inverseOfAsStart": info["inverseOfAsStart"],
                "inverseOfAsTarget": info["inverseOfAsTarget"],
                "owl:FunctionalProperty": info["owl:FunctionalProperty"],
                "owl:InverseFunctional": info["owl:InverseFunctional"],
                "owl:SymmetricProperty": info["owl:SymmetricProperty"],
                "owl:AsymmetricProperty": info["owl:AsymmetricProperty"],
                "owl:ReflexivProperty": info["owl:ReflexivProperty"],
                "owl:IrreflexivProperty": info["owl:IrreflexivProperty"],
                "owl:DeprecatedProperty": info["owl:DeprecatedProperty"],
                "FKCandidate": info["FKCandidate"],
                "owl:oneOf": info["owl:oneOf"],
            # Restrictions
                "usedInClassExpression": info["usedInClassExpression"],
                "usedInClassExpressionCount": info["usedInClassExpressionCount"],
                "usedInEqivalentClassDef": info["usedInEqivalentClassDef"],
                "usedInSubClassOfDef": info["usedInSubClassOfDef"],
                "owl:hasValue": info["owl:hasValue"],
                "owl:someValuesFrom": info["owl:someValuesFrom"],
                "owl:allValuesFrom": info["owl:allValuesFrom"],
                "owl:hasSelf": info["owl:hasSelf"],
                "owl:minCardinality": info["owl:minCardinality"],
                "owl:maxCardinality": info["owl:maxCardinality"],
                "owl:cardinality": info["owl:cardinality"],
                "owl:minQualifiedCardinality": info["owl:minQualifiedCardinality"],
                "owl:maxQualifiedCardinality": info["owl:maxQualifiedCardinality"],
                "owl:qualifiedCardinality": info["owl:qualifiedCardinality"],
                "owl:intersectionOf": info["owl:intersectionOf"],
                "owl:unionOf": info["owl:unionOf"],
                "owl:complementOf": info["owl:complementOf"]
                }

        return grouped_result_domain
    
    else:
        # Keine Gruppierung - nur Properties mit Infos
        return prop_infos


ttl_file = "oscars_6_movies_updated.ttl"
g = Graph()
g.parse(ttl_file, format="turtle")

properties = extract_properties(g, "properties")
# domain = extract_properties(g, "domain")
# ranges = extract_properties(g, "range")

# print(properties)

# for cls, props in domain.items():
#     for prop in domain[cls].keys():
#         print(prop)
# # for domain, props in domain.items():
# #     print(f"Domain Klasse: {domain}")
# #     for prop, infos in props.items():
# #         print(f"   - {prop}")
# #         for label, info in infos.items():
# #             print(f"        {label} : {info}")
    
# for cls, props in ranges.items():
#     print(f"Range Klasse: {cls}")
#     for prop in props:
#         print(f"   - {prop}")

for props, infos in properties.items():
    print(f"Property: {props}")
    for label, info in infos.items():
        print(f"     - {label}: {info}")


Property: http://example.org/ontology#participatedIn
     - rdf:type: http://www.w3.org/2002/07/owl#ObjectProperty
     - rdfs:domain: http://example.org/ontology#CastAndCrew
     - rdfs:range: http://example.org/ontology#Movie
     - sqlName: {'sql_name': 'participated_in', 'source': 'Label'}
     - distinctSubjectCount: 0
     - distinctObjectCount: 0
     - possibleDataTypes: {'byRange': None, 'byLiteral': None, 'byValue': None, 'byRestriction': None, 'byKeyword': None, 'combinedCount': None}
     - recommendedDataType: {'XSDdatatype': 'No datatype found', 'SQLdatatype': 'VARCHAR', 'source': None}
     - isSingleValued: False
     - isMultiValued: True
     - usedByClasses: None
     - rdfs:subPropertyOf: None
     - superPropertyOf: [rdflib.term.URIRef('http://example.org/ontology#actedIn'), rdflib.term.URIRef('http://example.org/ontology#produced'), rdflib.term.URIRef('http://example.org/ontology#directed'), rdflib.term.URIRef('http://example.org/ontology#wrote')]
     - owl:prope

## Klassen Infos

Bestimmt alle Klassen und zugehörige Infos

In [371]:
def extract_classes(graph, cls):
    """
    Extrahiert Klassen mit verschiedenen Infos aus einer RDF-Datei.

    :param ttl_file: Pfad zur RDF-Datei (Turtle)
    :return: {Class: {"label": ..., "comment":..., "used":..., ...}}
    """
    g = graph
    # g.parse(ttl_file, format="turtle")


    all_classes = get_all_classes(g)

    used_in_class_axioms = collect_class_usages(g)

    reference_props = [OWL.equivalentClass, RDFS.subClassOf, OWL.complementOf, OWL.disjointUnionOf]

    cls_infos = {}

    # Alle Klassen und ihre Infos sammeln
    # for cls in all_classes:

# Grundlegende Infos
    for cls, name in get_sql_name(g, cls).items():
        sql_name = name
    label = g.value(cls, RDFS.label)
    other_labels = []
    for _, p, _ in g.triples((cls, None, None)):
        if "label" in clean_value(p) and not RDFS.label:
            other_labels.append(p)
    comment = g.value(cls, RDFS.comment)

    type2 = set(g.objects(cls, RDF.type))
    if not type2 and any(g.value(cls, p) for p in reference_props):
        type2 = set(g.value(g.value(cls, p), RDF.type) for p in reference_props if g.value(cls, p))     
    type_ = (next(iter(type2)) if len(type2) == 1 else type2)

# TODO TODO TODO
    # number_of_instances = cls_with_instances.get(cls, 0)
# TODO TODO TODO
    
    # uri_as_pk = False
    # all_instances = set(g.subjects(RDF.type, cls))
    # if len(all_instances) == number_of_instances and all_instances:
    #     uri_as_pk = True
    # if not all_instances:
    #     uri_as_pk = "no Instances"
    primary_key = get_primary_key(g)
    
    reference_count = sum(    
        1 for s, p, o in g.triples((None, None, cls))
        if isinstance(cls, URIRef)
        )
# TODO Was genau zählt das und macht das überhaupt Sinn 
# Hierarchie
    super_class_of = set(g.subjects(RDFS.subClassOf, cls))
# TODO hier werden nur die direkten Subklassen angezeigt, ich will aber alle
    sub_class_of, sub_class_restrictions = extract_class_axioms(g, cls, RDFS.subClassOf)
# TODO wenn subClassOf aus mehreren einzel Restrictions besteht ist das gleich wie intersectionOf über die Restrictions
# TODO hier werden nur die direkten Superklassen angezeigt, ich will aber alle 
    super_class_to_keep = decide_wich_super_class_to_keep(g, cls)
    equivalent_class_subject, equivalent_restrictions = extract_class_axioms(g, cls, OWL.equivalentClass)
    equivalent_class_object = set(g.subjects(OWL.equivalentClass, cls))
    equivalent_class = equivalent_class_subject.union(equivalent_class_object)    
    complement_of_subject, complement_restriction = extract_class_axioms(g, cls, OWL.complementOf)
    complement_of_object = set(g.subjects(OWL.complementOf, cls))
    complement_of = complement_of_subject.union(complement_of_object)
# Struktur
    disjoint_class_subject = set(g.objects(cls, OWL.disjointWith))
    disjoint_class_object = set(g.subjects(OWL.disjointWith, cls))
    disjoint_class = disjoint_class_subject.union(disjoint_class_object)
    disjoint_union, disjoint_union_restriction = extract_class_axioms(g, cls, OWL.disjointUnionOf)
    has_key = g.value(cls, OWL.hasKey)
    one_of = g.value(cls, OWL.oneOf)
    cardinality_summary = extract_cardinality_constraints(sub_class_restrictions, equivalent_restrictions) 
    all_disjoint_class = []
    for s, p, o in g.triples((None, RDF.type, OWL.AllDisjointClasses)):
        member_list = list(Collection(g, g.value(s, OWL.members)))
        if cls in member_list:
            tst = [x for x in member_list if x != cls]
            all_disjoint_class.append(tst)
    
# Verwendung in Restrictions
    in_restrictions = []
    # used_in_class_axioms = collect_class_usages(g, cls)
    if cls in used_in_class_axioms.keys():
        for block in used_in_class_axioms[cls]:
            in_restrictions.append(block)     
# TODO in Restrictions könnte sowas wie allValuesFrom etc auch ein BNode sein!!
# als Domain oder Range
    as_domain_of = set(g.subjects(RDFS.domain, cls))
    as_range_of = set(g.subjects(RDFS.range, cls))
# Annotationen
    see_also = set(g.objects(cls, RDFS.seeAlso))
    defined_by = set(g.objects(cls, RDFS.isDefinedBy))
    annotations = get_annotaions(g, cls)
    # equivalent_annotations = get_annotations_of_equivalent_classes(g, cls)
    # equivalent_name = get_equivalent_sql_name(g, cls)
    equivalent_group_name, equivalent_group, equivalent_class_annotations = build_equivalent_class_map(g)
    equivalent_group_name = equivalent_group_name.get(cls)
    equivalent_group = equivalent_group.get(cls)
    equivalent_class_annotations = equivalent_class_annotations.get(cls)
    


    cls_infos[cls] = {
    # Grundlegende Infos
        "rdf:type": type_ if type_ else None,
        "sqlName": sql_name,
        "simpleClass": False if any([equivalent_restrictions, sub_class_restrictions, complement_restriction, disjoint_union_restriction]) else True, 
        # "rdfs:label": label if label else None,
        # "otherlabels": other_labels if other_labels else None, 
        # "tableName": clean_value(cls),

# TODO TODO TODO 
        # "inferred": cls not in explicit_class, # evlt nocch zählen woe oft das vorkommt
        # "hasInstances": cls in cls_with_instances,
        # "numberOfInstances": number_of_instances if number_of_instances else 0,
# TODO TODO TODO

        # "URIasPK": uri_as_pk, 
        "primaryKey": primary_key,
        "owl:DeprecatedClass": OWL.DeprecatedClass in set(g.objects(cls, RDF.type)),
        "usedAsType": (None, RDF.type, cls) in g,

# TODO TODO TODO            
        # "isPureTBox": number_of_instances == 0 and not ((None, RDF.type, cls) in g),
# TODO TODO TODO     
    
        "totalReferenceCount": sum(1 for _ in g.triples((None, None, cls))), 
        "axiomCount": sum(1 for p in reference_props if (cls, p, None) in g),
    # Hierarchie 
        "isTopClass": not sub_class_of,
        "isLeafClass": not super_class_of,
        "rdfs:subClassOf": sub_class_of if sub_class_of else None,
        "superClassOf": [c for c in super_class_of] if super_class_of else None, 
        "superClassToKeep": super_class_to_keep if super_class_to_keep else None,
        "owl:equivalentClass": equivalent_class if equivalent_class else None,
        "equivalentClassAsSubject": [c for c in equivalent_class_subject] if equivalent_class_subject else None,
        "equivalentClassAsObject": [c for c in equivalent_class_object] if equivalent_class_object else None,
        "complementOfSubject": [c for c in complement_of_subject] if complement_of_subject else None,
    # Struktur
        "owl:disjointClasses": [c for c in disjoint_class] if disjoint_class else None,
        "owl:disjointClassesAsSubject": [c for c in disjoint_class_subject] if disjoint_class_subject else None,
        "owl:disjointClassesAsObject": [c for c in disjoint_class_object] if disjoint_class_object else None,
        "owl:AllDisjointClasses": all_disjoint_class if all_disjoint_class else None,
        "owl:disjointUnionOf": disjoint_union if disjoint_union else None,
        "owl:complementOf": [c for c in complement_of] if complement_of else None,
        "owl:hasKey": list(Collection(g, has_key)) if isinstance(has_key, BNode) else None,
        "owl:oneOf": list(Collection(g, one_of)) if isinstance(one_of, BNode) else None,
        "cardinalitySummary": cardinality_summary if cardinality_summary else None,
    # Definiert durch in Restrictions
        "subClassRestrictions": sub_class_restrictions if sub_class_restrictions else None,
        "equivalentClassRestrictions": equivalent_restrictions if equivalent_restrictions else None,
        "complementOfRestriction": complement_restriction if complement_restriction else None,
        "disjointUnionOfRestriction": disjoint_union_restriction if disjoint_union_restriction else None,
        "usedInRestrictions": in_restrictions if in_restrictions else None,
    # als Domain oder Range
        "asDomainOf": [c for c in as_domain_of] if as_domain_of else None,
        "asRangeOf": [c for c in as_range_of] if as_range_of else None, 
    # Annotationen
        # "rdfs:comment": comment if comment else None,
        # "rdfs:seeAlso": [info for info in see_also] if see_also else None,
        # "rdfs:isDefinedBy": [info for info in defined_by] if defined_by else None,
        "annotations": annotations,
        "equivalentClassAnnotations": equivalent_class_annotations if equivalent_class_annotations else None,
        "equivalentTableName": equivalent_group_name if equivalent_group_name else None,
        "equivalentGroup": equivalent_group if equivalent_group else None,

# TODO TODO TODO
        "implementedAs": ()
# TODO TODO TODO
    }

    return cls_infos




ttl_file = "oscars_expose_updated4.ttl"
g = Graph()
g.parse(ttl_file, format="turtle")

cls = URIRef("http://example.org/ontology#Person")

classes = extract_classes(g, cls)

for cls, infos in classes.items():
    print(f"Klasse: {cls}")
    for label, info in infos.items():
        print(f"        {label} : {info}")

# all_class = get_all_classes(g)

# for cls in all_class:
#     info = extract_classes(g, cls)
#     if info[cls]["simpleClass"]:
#         print(cls)
#     # print(info[cls]["simpleClass"])


Klasse: http://example.org/ontology#Person
        rdf:type : None
        sqlName : {'sql_name': 'person', 'source': 'URI'}
        simpleClass : True
        primaryKey : uri
        owl:DeprecatedClass : False
        usedAsType : False
        totalReferenceCount : 2
        axiomCount : 0
        isTopClass : True
        isLeafClass : True
        rdfs:subClassOf : None
        superClassOf : None
        superClassToKeep : h
        owl:equivalentClass : None
        equivalentClassAsSubject : None
        equivalentClassAsObject : None
        complementOfSubject : None
        owl:disjointClasses : None
        owl:disjointClassesAsSubject : None
        owl:disjointClassesAsObject : None
        owl:AllDisjointClasses : None
        owl:disjointUnionOf : None
        owl:complementOf : None
        owl:hasKey : None
        owl:oneOf : None
        cardinalitySummary : None
        subClassRestrictions : None
        equivalentClassRestrictions : None
        complementOfRest

## SQL Generierung

### Hilfsfunktionen

In [372]:
def annotation_sql(info_dict, annotation_key, element, comment_list):

    annotations = (
        info_dict[annotation_key] if annotation_key == "equivalentClassAnnotations"
        else info_dict[annotation_key][element]
    )

    for key, values in annotations.items():
        if values in [True, False]:
            continue
        elif key != "customAnnotations" and values is not None:
            for val in values:
                comment_list.append(f"> {key}: {val}")
        elif key == "customAnnotations" and values is not None:
            for costum_anno, val in values.items():
                for v in val:
                    comment_list.append(f"> {remove_prefix(costum_anno)}: {v}")

    return comment_list

# ttl_file = "oscars_expose_updated4.ttl"
# g = Graph()
# g.parse(ttl_file, format="turtle")

# annotation_key = "equivalentClassAnnotations"
# comment_list = []
# cls = URIRef("http://example.org/ontology#Prinzessin")
# class_info = extract_classes(g, cls)[cls]

# annotation_sql(class_info, annotation_key, cls, comment_list)

### Trigger

SubClassOf - Trigger für Version 1 von R4

In [419]:
def trigger_definition_sub_classes(sub_class_name, super_classes, equivalent_class_mapping):
    """_summary_

    :param sup_class: _description_
    :type sup_class: _type_
    :param super_classes: _description_
    :type super_classes: _type_
    """
    trigger_definition = []


    trigger_definition_sub = {
        "triggerName": f"trg_check_super_classes_{sub_class_name}",
        "pointInTime": "BEFORE",
        "event": "INSERT", 
        "target": sub_class_name,
        "validity": "ROW",
        "function": "check_super_class_tables()"
    }
    trigger_definition.append(trigger_definition_sub)

    seen_super_class = []

    for super_class in super_classes:

        #### NOTE Variante 5 von R5 NOTE ####
        equi_super_class = equivalent_class_mapping.get(super_class)
        
        if equi_super_class not in seen_super_class:
            seen_super_class.append(equi_super_class)
            
            if super_class in equivalent_class_mapping.keys():
                    super_class_name = get_sql_name(g, equi_super_class)[equi_super_class]["sql_name"]
            else:    
                super_class_name = get_sql_name(g, super_class)[super_class]["sql_name"]
        
            trigger_definition_super = {
                "triggerName": f"trg_check_sub_classes_{super_class_name}",
                "pointInTime": "BEFORE",
                "event": "DELETE", 
                "target": super_class_name,
                "validity": "ROW",
                "function": "check_sub_class_tables()"
            }
            trigger_definition.append(trigger_definition_super)
        #### NOTE Variante 5 von R5 NOTE ####

        # #### NOTE Variante 3/4 von R5 NOTE ####
        # super_class_name = get_sql_name(g, super_class)[super_class]["sql_name"]
    
        # trigger_definition_super = {
        #     "triggerName": f"trg_check_sub_classes_{super_class_name}",
        #     "pointInTime": "BEFORE",
        #     "event": "DELETE", 
        #     "target": super_class_name,
        #     "validity": "ROW",
        #     "function": "check_sub_class_tables()"
        # }
        # trigger_definition.append(trigger_definition_super)
        # #### NOTE Variante 3/4 von R5 NOTE ####

    return trigger_definition

# sub_class = URIRef("http://example.org/ontology#Director")
# subcls_info = extract_classes(g, sub_class)[sub_class]
# subcls_name = subcls_info["sqlName"]["sql_name"]
# super_class = subcls_info["rdfs:subClassOf"]
# equi_map = build_equivalent_class_map(g)[0]

# trigger_definition_sub_classes(subcls_name, super_class, equi_map)


Trigger Funktionen für Version 1 von R4

In [374]:
def sub_class_trigger_functions():

    check_super_class_tables = """
    CREATE OR REPLACE FUNCTION check_super_class_tables()
    RETURNS TRIGGER AS $$
    DECLARE
        super TEXT;
        query TEXT;
        found INT;
    BEGIN
        FOR super IN
            SELECT super_class_label 
            FROM sub_class_table 
            WHERE sub_class_label = TG_TABLE_NAME
        LOOP
            query := format('SELECT COUNT(*) FROM %I WHERE id = $1', super);
            EXECUTE query INTO found USING NEW.id;
            IF found = 0 THEN
            RAISE EXCEPTION 'Instance % is missing in super class %', NEW.id, super;
            END IF;
        END LOOP;
        RETURN NEW;
    END;
    $$ LANGUAGE plpgsql;
    """

    check_sub_class_tables = """
    CREATE OR REPLACE FUNCTION check_sub_class_tables()
    RETURNS TRIGGER AS $$
    DECLARE
        sub TEXT;
        query TEXT;
        found INT;
    BEGIN
        FOR sub IN
            SELECT sub_class_label 
            FROM sub_class_table 
            WHERE super_class_label = TG_TABLE_NAME
        LOOP
            query := format('SELECT COUNT(*) FROM %I WHERE id = $1', sub);
            EXECUTE query INTO found USING OLD.id;
            IF found > 0 THEN
            RAISE EXCEPTION 'Instance % still exists in sub class % and must not be deleted.', OLD.id, sub;
            END IF;
        END LOOP;
        RETURN OLD;
    END;
    $$ LANGUAGE plpgsql;
    """

    return check_super_class_tables + "\n" + check_sub_class_tables


EquivalentClass - Trigger für Version 4 von R5

In [376]:
def trigger_definition_equivalent_class(equivalent_class_group):
    """_summary_

    :param equivalent_class_group: _description_
    :type equivalent_class_group: _type_
    """
    trigger_definition = []

    # for cls, group in equivalent_class_group.items():

    for cls in equivalent_class_group:
        cls = get_sql_name(g, cls)[cls]["sql_name"]
        trigger_definition_insert = {
            "triggerName": f"trg_equi_insert_{cls}",
            "pointInTime": "AFTER",
            "event": "INSERT", 
            "target": cls,
            "validity": "ROW",
            "function": "insert_into_equivalnet_classes()"
        }
        trigger_definition.append(trigger_definition_insert)

        trigger_definition_delete = {
            "triggerName": f"trg_equi_delete_{cls}",
            "pointInTime": "AFTER",
            "event": "DELETE", 
            "target": cls,
            "validity": "ROW",
            "function": "delete_from_equivalent_classes()"
        }
        trigger_definition.append(trigger_definition_delete)

    return trigger_definition

equivalent_class_map, equivalent_class_group, equivalent_annotations = build_equivalent_class_map(g)

cls = URIRef("http://example.org/ontology#Prinzessin")
trigger_definition_equivalent_class(equivalent_class_group[cls])

[{'triggerName': 'trg_equi_insert_princess',
  'pointInTime': 'AFTER',
  'event': 'INSERT',
  'target': 'princess',
  'validity': 'ROW',
  'function': 'insert_into_equivalnet_classes()'},
 {'triggerName': 'trg_equi_delete_princess',
  'pointInTime': 'AFTER',
  'event': 'DELETE',
  'target': 'princess',
  'validity': 'ROW',
  'function': 'delete_from_equivalent_classes()'},
 {'triggerName': 'trg_equi_insert_prinzessin',
  'pointInTime': 'AFTER',
  'event': 'INSERT',
  'target': 'prinzessin',
  'validity': 'ROW',
  'function': 'insert_into_equivalnet_classes()'},
 {'triggerName': 'trg_equi_delete_prinzessin',
  'pointInTime': 'AFTER',
  'event': 'DELETE',
  'target': 'prinzessin',
  'validity': 'ROW',
  'function': 'delete_from_equivalent_classes()'},
 {'triggerName': 'trg_equi_insert_princess2',
  'pointInTime': 'AFTER',
  'event': 'INSERT',
  'target': 'princess2',
  'validity': 'ROW',
  'function': 'insert_into_equivalnet_classes()'},
 {'triggerName': 'trg_equi_delete_princess2',
  'p

In [ ]:
def equivalent_class_trigger_function():
    """_summary_
    """

    insert_into_equivalnet_classes = """
    CREATE OR REPLACE FUNCTION insert_into_equivalnet_classes()
    RETURNS TRIGGER AS $$
    DECLARE
        equivalent_class_uri TEXT;
        equivalent_table TEXT;
    BEGIN
        SELECT equivalent_with_class INTO equivalent_class_uri
        FROM classes
        WHERE associated_table = TG_TABLE_NAME;

        FOR equivalent_table IN
            SELECT associated_table
            FROM classes
            WHERE equivalent_with_class = equivalent_class_uri
        LOOP
            EXECUTE format('INSERT INTO %I (id) VALUES ($1)', equivalent_table)
            USING NEW.id;
        END LOOP;

        RETURN NEW;
    END;
    $$ LANGUAGE plpgsql;
    """

    delete_from_equivalent_class = """    
    CREATE OR REPLACE FUNCTION delete_from_equivalent_classes()
    RETURNS TRIGGER AS $$
    DECLARE
        equivalent_class_uri TEXT;
        equivalent_table TEXT;
    BEGIN
        SELECT equivalent_with_class INTO equivalent_class_uri
        FROM classes
        WHERE associated_table = TG_TABLE_NAME;

        FOR equivalent_table IN
            SELECT associated_table
            FROM classes
            WHERE equivalent_with_class = equivalent_class_uri
        LOOP
            EXECUTE format('DELETE FROM %I WHERE id = $1', equivalent_table)
            USING OLD.id;
        END LOOP;

        RETURN OLD;
    END;
    $$ LANGUAGE plpgsql;
    """

    return insert_into_equivalnet_classes + "\n" + delete_from_equivalent_class

### Tabellen + Attribute

Eine Tabelle mit allen Klassen

In [385]:
def table_definition_all_classes_table():
    """_summary_

    :param graph: _description_
    :type graph: _type_
    :param all_classes: _description_
    :type all_classes: _type_
    """
    columns = []
    equivalent_class_key_defi = None

    columns.append({
        "attributeName": "class_uri",
        "datatype": "VARCHAR",
        "primaryKey": True,
    })
    RDFS.label
    column_names = ["associated_table", "equivalent_with_class", "implemented_as"]
    
    column_comments = {
        "implemented_as": "How was the class initially implemented: Not at all, as a table, or as a view."
    }

    for name in column_names:     
        column = {
            "attributeName": name,
            "datatype": "VARCHAR",
            "comment": column_comments.get(name),
        }
        columns.append(column)
    
    table_definition = {
        "tableName": "classes",
        "tableAnnotations": ["All classes from the ontology."],
        "columns": columns,
    }

    equivalent_class_key_defi = [{
        "foreign_key_table": "classes",
        "foreign_key_column": "equivalent_with_class",
        "target_table": "classes",
        "target_column": "class_uri",
    }]
    return table_definition, equivalent_class_key_defi
    
# ttl_file = "oscars_expose_updated4.ttl"
# g = Graph()
# g.parse(ttl_file, format="turtle")

# table_definition_all_classes_table()

Fasst für eine Klasse alle Kommentare und Attribute (mit benötigten Infos) zusammen. \
<b> Für Variante 3/4 von R5 </b>

In [ ]:
# def table_definition_class(graph, cls, cls_mapping, prop_mapping):
#     """_summary_

#     :param ttl_file: _description_
#     :type ttl_file: _type_
#     """
#     g = graph

#     table_definition = {}

#     columns = []
#     table_comments = []

#     foreign_key_definition = [] 
#     trigger_definition = []

#     sub_class_trigger_definition = None
#     equivalent_class_trigger_definition = None

#     equivalent_class_map = build_equivalent_class_map(g)[0]

#     all_classes = get_all_classes(g)
#     cls_info = extract_classes(g, cls)[cls]

#     if cls_info["simpleClass"] and cls not in cls_mapping.keys():
#         equivalent_classes = cls_info["equivalentGroup"]

#         table_name = cls_info["sqlName"]["sql_name"]

#         super_classes = cls_info["rdfs:subClassOf"]

#         if equivalent_classes and not any(cls in cls_mapping for cls in equivalent_classes):
#             equivalent_class_trigger_definition = trigger_definition_equivalent_class(equivalent_classes)
#             trigger_definition.extend(equivalent_class_trigger_definition)
        
#         all_properties = get_all_properties(g)
#         prop_info = extract_properties(g, "domain")

#         id_type = cls_info["primaryKey"]

#         pk_column = {
#             "attributeName": "id", 
#             "primaryKey": True,
#         }

#         if id_type == "uri":
#             pk_column["datatype"] = "VARCHAR"
#             pk_column["comment"] = "Primary Key: Unique URI from the original Ontology"
#         else:
#             pk_column["datatype"] = "SERIAL"
#             pk_column["comment"] = "Primary Key: Automatic ID"

#         columns.append(pk_column)
        
#         columns.append({
#             "attributeName": "label",
#             "datatype": "VARCHAR",
#             "comment": "rdfs:Label",
#         })

#         # #### NOTE SubClassOf Behandlung von Variante 2 NOTE ####
#         # super_class = decide_wich_super_class_to_keep(g, super_classes)
#         # if super_class:
#         #     # super_class_name = get_sql_name(g, super_class)[super_class]["sql_name"]
#         #     super_class_name = get_sql_name(g, super_class)[super_class]["sql_name"]
#         #     sub_class_key_definition = {
#         #         "foreign_key_table": table_name,
#         #         "foreign_key_column": "id",
#         #         "target_table": super_class_name,
#         #         "target_column": "id",
#         #     }
#         # #### NOTE SubClassOf Behandlung von Variante 2 NOTE ####

#         #### NOTE SubClassOf Behandlung von Variante 1 NOTE ####
#         number_of_super_classes = len(super_classes) if super_classes else 0
#         if super_classes and number_of_super_classes == 1:
#             super_class = next(iter(super_classes))            
#             super_class_name = get_sql_name(g, super_class)[super_class]["sql_name"]        
#             sub_class_key = {
#                 "foreign_key_table": table_name,
#                 "foreign_key_column": "id",
#                 "target_table": super_class_name,
#                 "target_column": "id",
#             }
#             foreign_key_definition.append(sub_class_key)
#         elif super_classes:
#             sub_class_trigger_definition = trigger_definition_sub_classes(table_name, super_classes, equivalent_class_map)
#             trigger_definition.extend(sub_class_trigger_definition)
#         #### NOTE SubClassOf Behandlung von Variante 1 NOTE ####
       
#         if prop_info.get(cls):
#             for prop, infos in prop_info[cls].items():
#                 if not infos.get("isSingleValued"):
#                     continue
                
#                 prop_name = infos["sqlName"]["sql_name"]
#                 datatype = infos["recommendedDataType"]["SQLdatatype"]

#                 column_comments = annotation_sql(infos, "annotations", prop, [])
            
#                 column_info = {
#                     "attributeName": prop_name, 
#                     "attributeAnnotations": column_comments,
#                     "singleValued": f"functionalProperty: {infos['isSingleValued']}",  
#                 }
                
#                 if isinstance(datatype, str):
#                     column_info["datatype"] = datatype
#                 elif isinstance(datatype, dict):
#                     column_info["datatype"] = datatype["sql_datatype"]
#                     column_info["check_constraints"] = [
#                         {"restriction": r, "value": v}
#                         for r, v in datatype["restrictions"].items()
#                     ]
#                 columns.append(column_info)

#                 prop_mapping[prop] = {
#                     "name": prop_name, 
#                     "domain": table_name
#                     }

#                 range_class = infos["rdfs:range"]
#                 if range_class in all_classes:
#                     range_class_name = get_sql_name(g, range_class)[range_class]["sql_name"]
#                     foreign_key_definition.append({
#                         "foreign_key_table": table_name,
#                         "foreign_key_column": prop_name,
#                         "target_table": range_class_name,
#                         "target_column": "id",
#                     })      
        
#         table_comments = annotation_sql(cls_info, "annotations", cls, table_comments)       
       
#         table_definition = {
#             "tableName": table_name,
#             "tableAnnotations": table_comments if table_comments else None, 
#             "columns": columns
#         }  

#         cls_mapping[cls] = table_name 


#     return table_definition, cls_mapping, prop_mapping, foreign_key_definition, trigger_definition


    
# # ttl_file = "oscars_6_movies_updated.ttl"
# ttl_file = "oscars_expose_updated4.ttl"
# g = Graph()
# g.parse(ttl_file, format="turtle")

# all_classes = get_all_classes(g)

# cls = URIRef("http://example.org/ontology#Director")
# cls_mapping = {}
# prop_mapping = {}
# defi_table = []
# defi_trigger = []
# defi_fk = []

# for cls in all_classes:
#     table_defi, cls_map, prop_map, foreign_key_defi, trigger_defi = table_definition_class(g, cls, cls_mapping, prop_mapping)
#     if table_defi:
#         # sql_table.append(sql_create_table(defi_table))
#         defi_table.append(table_defi)
#     if trigger_defi:
#         defi_trigger.extend(trigger_defi)
#     if foreign_key_defi:
#         defi_fk.extend(foreign_key_defi)

# defi_table

# equi_mapping = build_equivalent_class_map(g)[0]
# equi_mapping
# cls_map
# prop_map
# sub_class_trigger_defi
# foreign_key_defi

Fasst für eine Klasse alle Kommentare und Attribute (mit benötigten Infos) zusammen. \
<b> Für Variante 5 von R5 </b>

In [420]:
def table_definition_class(graph, cls, cls_mapping, prop_mapping):
    """_summary_

    :param ttl_file: _description_
    :type ttl_file: _type_
    """
    g = graph

    table_definition = {}

    columns = []
    table_comments = []

    foreign_key_definition = [] 
    trigger_definition = []

    sub_class_trigger_definition = None
    equivalent_class_trigger_definition = None

    equivalent_class_map = build_equivalent_class_map(g)[0]

    all_classes = get_all_classes(g)
    cls_info = extract_classes(g, cls)[cls]
    
    classes = set()
    super_classes = set()
    

    if cls_info["simpleClass"] and cls not in cls_mapping.keys():
        equivalent_classes = cls_info["equivalentGroup"]       
        if equivalent_classes:
            classes = set(equivalent_classes)
            table_name = cls_info["equivalentTableName"]

            if isinstance(table_name, URIRef):
                table_name = get_sql_name(g, table_name)[table_name]["sql_name"]
            
            for equivalent_class in equivalent_classes:
                equivalent_class_info = extract_classes(g, equivalent_class)[equivalent_class]
                if equivalent_class_info["rdfs:subClassOf"]:
                    super_classes = super_classes.union(equivalent_class_info["rdfs:subClassOf"])
        else:
            table_name = cls_info["sqlName"]["sql_name"]
            classes.add(cls)
            if cls_info["rdfs:subClassOf"]:
                super_classes = super_classes.union(cls_info["rdfs:subClassOf"])
        
        all_properties = get_all_properties(g)
        prop_info = extract_properties(g, "domain")

        id_type = cls_info["primaryKey"]

        pk_column = {
            "attributeName": "id", 
            "primaryKey": True,
        }

        if id_type == "uri":
            pk_column["datatype"] = "VARCHAR"
            pk_column["comment"] = "Primary Key: Unique URI from the original Ontology"
        else:
            pk_column["datatype"] = "SERIAL"
            pk_column["comment"] = "Primary Key: Automatic ID"

        columns.append(pk_column)
        
        columns.append({
            "attributeName": "label",
            "datatype": "VARCHAR",
            "comment": "rdfs:Label",
        })

        #### NOTE SubClassOf Behandlung von Variante 2 NOTE ####
        super_class = decide_wich_super_class_to_keep(g, super_classes)
        if super_class:         
            if super_class in equivalent_class_map.keys():
                equi_super_class = equivalent_class_map.get(super_class)
                super_class_name = get_sql_name(g, equi_super_class)[equi_super_class]["sql_name"]
            else:    
                super_class_name = get_sql_name(g, super_class)[super_class]["sql_name"]
            
            super_class_name = get_sql_name(g, super_class)[super_class]["sql_name"]
            sub_class_key_definition = {
                "foreign_key_table": table_name,
                "foreign_key_column": "id",
                "target_table": super_class_name,
                "target_column": "id",
            }
            foreign_key_definition.append(sub_class_key_definition)
        #### NOTE SubClassOf Behandlung von Variante 2 NOTE ####

        # #### NOTE SubClassOf Behandlung von Variante 1 NOTE ####
        # number_of_super_classes = len(super_classes) if super_classes else 0
        # if super_classes and number_of_super_classes == 1:
        #     super_class = next(iter(super_classes))
    
        #     if super_class in equivalent_class_map.keys():
        #         equi_super_class = equivalent_class_map.get(super_class)
        #         super_class_name = get_sql_name(g, equi_super_class)[equi_super_class]["sql_name"]
        #     else:    
        #         super_class_name = get_sql_name(g, super_class)[super_class]["sql_name"]
            
        #     sub_class_key = {
        #         "foreign_key_table": table_name,
        #         "foreign_key_column": "id",
        #         "target_table": super_class_name,
        #         "target_column": "id",
        #     }
        #     foreign_key_definition.append(sub_class_key)
        # elif super_classes:
        #     sub_class_trigger_definition = trigger_definition_sub_classes(table_name, super_classes, equivalent_class_map)
        #     trigger_definition.extend(sub_class_trigger_definition)
        # #### NOTE SubClassOf Behandlung von Variante 1 NOTE ####

        for clss in classes:
            if not prop_info.get(clss):
                continue

            for prop, infos in prop_info[clss].items():
                if not infos.get("isSingleValued"):
                    continue

                prop_name = infos["sqlName"]["sql_name"]
                datatype = infos["recommendedDataType"]["SQLdatatype"]

                column_comments = annotation_sql(infos, "annotations", prop, [])
            
                column_info = {
                    "attributeName": prop_name, 
                    "attributeAnnotations": column_comments,
                    "singleValued": f"functionalProperty: {infos['isSingleValued']}",  
                }

                if isinstance(datatype, str):
                    column_info["datatype"] = datatype
                elif isinstance(datatype, dict):
                    column_info["datatype"] = datatype["sql_datatype"]
                    column_info["check_constraints"] = [
                        {"restriction": r, "value": v}
                        for r, v in datatype["restrictions"].items()
                    ]
                    
                columns.append(column_info)

                prop_mapping[prop] = {
                    "name": prop_name, 
                    "domain": table_name
                    }

                range_class = infos["rdfs:range"]
                if range_class in all_classes:                   
                    equi_range_class = equivalent_class_map.get(range_class, range_class)
                    range_class_name = get_sql_name(g, equi_range_class)[equi_range_class]["sql_name"]
                
                    foreign_key_definition.append({
                        "foreign_key_table": table_name,
                        "foreign_key_column": prop_name,
                        "target_table": range_class_name,
                        "target_column": "id",
                    })

        if equivalent_classes:
            table_comments.append(
                f"Combined table for equivalent classes " 
                f"{', '.join(remove_prefix(equivalent_class) for equivalent_class in equivalent_classes)}"
                # f" and {remove_prefix(cls)}"
            )
            if cls_info["equivalentClassAnnotations"]:
                table_comments = annotation_sql(cls_info, "equivalentClassAnnotations", cls, table_comments)
        else:
            table_comments = annotation_sql(cls_info, "annotations", cls, table_comments)
              
        table_definition = {
            "tableName": table_name,
            "tableAnnotations": table_comments if table_comments else None, 
            "columns": columns
        }  

        for clss in classes:
            cls_mapping[clss] = table_name
        
    return table_definition, cls_mapping, prop_mapping, foreign_key_definition, trigger_definition


    
# # ttl_file = "oscars_6_movies_updated.ttl"
# ttl_file = "oscars_expose_updated4.ttl"
# g = Graph()
# g.parse(ttl_file, format="turtle")

# all_classes = get_all_classes(g)

# cls = URIRef("http://example.org/ontology#Director")
# cls_mapping = {}
# prop_mapping = {}
# defi_table = []
# defi_trigger = []
# defi_fk = []

# for cls in all_classes:
#     table_defi, cls_map, prop_map, foreign_key_defi, trigger_defi = table_definition_class(g, cls, cls_mapping, prop_mapping)
#     if table_defi:
#         defi_table.append(table_defi)
#     if trigger_defi:
#         defi_trigger.extend(trigger_defi)
#     if foreign_key_defi:
#         defi_fk.extend(foreign_key_defi)

# defi_table
# # equi_mapping = build_equivalent_class_map(g)[0]
# # equi_mapping
# # cls_map
# # prop_map
# # sub_class_trigger_defi
# # foreign_key_defi

Erstellt Tabelle zur Darstellung von Instanz-Annotations

In [ ]:
def table_definition_instance_comments():
    """_summary_

    :param definition: _description_
    :type definition: _type_
    """

    columns = []
    column_names = ["id", "annotation_name", "value"]

    for name in column_names:
        columns.append({
            "attributeName": name,
            "datatype": "VARCHAR"
        })

    annotation_table_definition = {
        "tableName": "instance_annotations",
        "columns": columns,
        "tableAnnotations": ["all annotations (rdfs and custom ons)"]
    }
   
    return annotation_table_definition


Für Version 3 von R4 - Erstellt die subClassOf-Metatabelle

In [406]:
def table_definition_sub_class_of():
    """_summary_
    """
    columns = []
    sub_class_key_definition = []
    column_names = ["sub_class", "sub_class_label", "super_class", "super_class_label"]

    for name in column_names:
        column = {
            "attributeName": name,
            "datatype": "VARCHAR",
        }
        if name in ["sub_class", "super_class"]:
            # column["foreignKeyTable"] = "classes"
            # column["foreignKeyAttribute"] = "class_uri"
            column["primaryKey"] = True

            sub_class_key_definition.append({
                "foreign_key_table": "sub_class_table",
                "foreign_key_column": name,
                "target_table": "classes",
                "target_column": "class_uri",
            })

        columns.append(column)

    sub_class_table_definition = {
        "tableName": "sub_class_table",
        "columns": columns,
        "tableAnnotations": ["shows all subClassOf-Relationships"]
    }

    return sub_class_table_definition, sub_class_key_definition

table_definition_sub_class_of()[0]

{'tableName': 'sub_class_table',
 'columns': [{'attributeName': 'sub_class',
   'datatype': 'VARCHAR',
   'primaryKey': True},
  {'attributeName': 'sub_class_label', 'datatype': 'VARCHAR'},
  {'attributeName': 'super_class', 'datatype': 'VARCHAR', 'primaryKey': True},
  {'attributeName': 'super_class_label', 'datatype': 'VARCHAR'}],
 'tableAnnotations': ['shows all subClassOf-Relationships']}

### Einträge

In [423]:
def entry_definition_all_classes(graph, all_classes):
    """_summary_

    :param g: _description_
    :type g: _type_
    :param all_classes: _description_
    :type all_classes: _type_
    """
    g = graph

    all_classes_definition = []
    equivalent_class_map = build_equivalent_class_map(g)[0]

    for cls in all_classes:
        class_info = extract_classes(g, cls)
        cls_info = class_info[cls]
        
        sql_name = cls_info["sqlName"]["sql_name"]
        implemented_as = cls_info["implementedAs"]
        equivalent_class = cls_info["owl:equivalentClass"]

        # Werte-Grundgerüst
        values = {
            "class_uri": remove_prefix(cls),
            "associated_table": sql_name,
            "implemented_as": implemented_as,
        }

        if equivalent_class:
            # values["equivalent_with_class"] = remove_prefix(next(iter(equivalent_class)))
            values["equivalent_with_class"] = remove_prefix(equivalent_class_map[cls])

        # Gesamtstruktur erzeugen
        all_classes_definition.append({
            "table": "classes",
            "values": values
        })
    
    #### NOTE Variante 5 von R5 NOTE ####
    seen_equivalent_class = []
    for equivalent_class in equivalent_class_map.values():
        if equivalent_class not in all_classes and equivalent_class not in seen_equivalent_class:
            values = {
                "class_uri": equivalent_class,
                "associated_table": equivalent_class,
                "implemented_as": "table",
            }
            all_classes_definition.append({
                "table": "classes",
                "values": values
            })
            seen_equivalent_class.append(equivalent_class)
    #### NOTE Variante 5 von R5 NOTE ####

    return all_classes_definition
        
# ttl_file = "oscars_expose_updated4.ttl"
# g = Graph()
# g.parse(ttl_file, format="turtle")

# all_classes = get_all_classes(g)

# entry_definition_all_classes(g, all_classes)

Mapt Individuen an Tabellen und ihre Values an die richtigen Attribute.

In [ ]:
def instance_definition(graph, all_classes, cls_mapping, prop_mapping, id_type):
    """
    :param g: rdflib.Graph
    :param class_to_table_map: Mapping von RDF-Klasse → SQL-Tabelle (z. B. {'http://example.org/ontology#Movie': 'movie'})
    :param prop_to_column_map: Mapping von RDF-Property → SQL-Spaltenname
    :return: Liste von Dicts für INSERT-Statements
    """
    g = graph
    instances = []
    insertet_instances = set()

    for s, o in g.subject_objects(RDF.type):
        if o not in all_classes:
            continue
        if s in insertet_instances:
            continue
        insertet_instances.add(s)

        cls_uris = set(g.objects(s, RDF.type))

        seen_tables = []

        for class_uri in cls_uris:
            if not class_uri or class_uri not in cls_mapping:
                continue  # ungemappt
            
            table = cls_mapping[class_uri]
            
            if table in seen_tables:
                continue
            seen_tables.append(table)

            values = {}

            # ID setzen
            if id_type == "uri":
                values["id"] = remove_prefix(s)

            # Label setzen
            label = g.value(s, RDFS.label)
            if label:
                values["label"] = remove_prefix(label).replace("'", "''")

            # Properties verarbeiten
            for p, o in g.predicate_objects(s):
                if p not in prop_mapping:
                    continue
                if table not in prop_mapping[p]["domain"]:
                    continue

                column_name = prop_mapping[p]["name"]

                if isinstance(o, (Literal, URIRef)):
                    value_clean = remove_prefix(o).replace("'", "''")
                else:
                    value_clean = str(o).replace("'", "''")  # falls BNode oder so

                values[column_name] = value_clean

            # Annotationen hinzufügen
            annotations = get_annotaions(g, s)


            if isinstance(table, URIRef):
                table_name = get_sql_name(g, table)[table]["sql_name"]
            else:
                table_name = table
                
            instance_data = {
                "table": table_name,
                "values": values,
                "annotations": annotations
            }

            instances.append(instance_data)

    return instances


# # ttl_file = "oscars_6_movies2_reasoning.ttl"
# ttl_file = "oscars_expose_updated4.ttl"
# # ttl_file = "test_reasoning.ttl"
# g = Graph()
# g.parse(ttl_file, format="turtle")

# all_classes = get_all_classes(g)

# cls_map = {}
# prop_map = {}
# id_type = get_primary_key(g)

# for cls in all_classes:

#     defi, cls_map, prop_map, defi_fks, defi_trigger = table_definition_class(g, cls, cls_map, prop_map)

# defi = instance_definition(g, all_classes, cls_map, prop_map, id_type)
# defi


In [ ]:
def entry_definition_instance_comments(instance_definitions):
    """_summary_

    :param definition: _description_
    :type definition: _type_
    """
    annotation_instances_definition = []
    seen = set()
    key_tuple = []

    for defi in instance_definitions:
        for instance_uri, annotation_values in defi["annotations"].items():
            clean_id = remove_prefix(instance_uri)
            
            for annotation_prop, value in annotation_values.items():
                if not value:
                    continue

                clean_name = remove_prefix(annotation_prop)

                # Liste absichern
                value_list = value if isinstance(value, list) else [value]

                if all(isinstance(v, dict) for v in value_list):
                    for val_dict in value_list:
                        for lang_key, lang_values in val_dict.items():
                            value_clean = ", ".join(lang_values)
                            key_tuple = (clean_id, remove_prefix(lang_key), value_clean)

                            if key_tuple not in seen:
                                seen.add(key_tuple)
                                annotation_instances_definition.append({
                                    "table": "instance_annotations",
                                    "values": {
                                        "id": clean_id,
                                        "annotation_name": remove_prefix(lang_key),
                                        "value": value_clean
                                    }
                                })

                else:
                    value_clean = ", ".join(str(v) for v in value_list)
                    key_tuple = (clean_id, clean_name, value_clean)

                    if key_tuple not in seen:
                        seen.add(key_tuple)
                        annotation_instances_definition.append({
                            "table": "instance_annotations",
                            "values": {
                                "id": clean_id,
                                "annotation_name": clean_name,
                                "value": value_clean
                            }
                        })

    return annotation_instances_definition


# ttl_file = "oscars_expose_updated4.ttl"
# g = Graph()
# g.parse(ttl_file, format="turtle")

# all_classes = get_all_classes(g)

# cls_map = {}
# prop_map = {}
# id_type = ()

# for cls in all_classes:
#     defi, cls_map, prop_map, id_type = table_definition_class(g, cls, cls_map, prop_map)


# defis = instance_definition(g, cls_map, prop_map, id_type)
# for defi in defis:
#     print(defi)


Instanzen für die subClassOf-Tabelle

In [ ]:
def entry_definition_sub_class_of(graph, all_classes):
    """_summary_

    :param graph: _description_
    :type graph: _type_
    :param all_classes: _description_
    :type all_classes: _type_
    """
    g = graph

    sub_class_of_definition = []

    for cls in all_classes:
        cls_info = extract_classes(g, cls)[cls]

        cls_name = cls_info["sqlName"]["sql_name"]

        sub_class_of = cls_info["rdfs:subClassOf"]
        

        if sub_class_of:
            for super_class in sub_class_of:
                super_class_name = get_sql_name(g, super_class)[super_class]["sql_name"]
                values = {
                    "sub_class": remove_prefix(cls),
                    "sub_class_label": cls_name,
                    "super_class": remove_prefix(super_class),
                    "super_class_label": super_class_name
                }
        
                sub_class_of_definition.append({
                    "table": "sub_class_table",
                    "values": values,
                })
    
    return sub_class_of_definition

ttl_file = "oscars_expose_updated4.ttl"
g = Graph()
g.parse(ttl_file, format="turtle")

all_classes = get_all_classes(g)

entry_definition_sub_class_of(g, all_classes)

[{'table': 'sub_class_table',
  'values': {'sub_class': 'Film3',
   'sub_class_label': 'film3',
   'super_class': 'Regessure',
   'super_class_label': 'regessure'}},
 {'table': 'sub_class_table',
  'values': {'sub_class': 'Director',
   'sub_class_label': 'director',
   'super_class': 'Film',
   'super_class_label': 'film'}},
 {'table': 'sub_class_table',
  'values': {'sub_class': 'Director',
   'sub_class_label': 'director',
   'super_class': 'Movie',
   'super_class_label': 'movie123'}},
 {'table': 'sub_class_table',
  'values': {'sub_class': 'Writer',
   'sub_class_label': 'writer',
   'super_class': 'Movie',
   'super_class_label': 'movie123'}},
 {'table': 'sub_class_table',
  'values': {'sub_class': 'Regessure',
   'sub_class_label': 'regessure',
   'super_class': 'Film',
   'super_class_label': 'film'}},
 {'table': 'sub_class_table',
  'values': {'sub_class': 'Regessure',
   'sub_class_label': 'regessure',
   'super_class': 'Director',
   'super_class_label': 'director'}}]

### Views

Variante 2 für Equivalenzklassen. Eine as Tabelle die andere als View.

In [ ]:
def view_definition_equivalnet_class(graph, cls, cls_map):
    """_summary_

    :param graph: _description_
    :type graph: _type_
    """
    g = graph

    view_definition = []

    cls_info = extract_classes(g, cls)

    view_name = cls_info[cls]["sqlName"]["sql_name"]


    if cls_info[cls]["equivalentClassAnnotations"] == False:
        equivalent_classes = cls_info[cls]["owl:equivalentClass"]
        view_comments = []
        view_comments = annotation_sql(cls_info[cls], "annotations", cls, view_comments)
        equivalent_class_name = None

        if len(equivalent_classes) == 1:
            equivalent_class = next(iter(equivalent_classes))
            equivalent_class_name = cls_map[equivalent_class]
            
            view_comments.append(f"> View {view_name} is equivalent to table {equivalent_class_name}")
            view_comments = "\n".join(view_comments)
        
        else:
            equivalent_class_name = []
            for ec in equivalent_classes:
                equivalent_class_name.append(cls_map[ec])

            view_comments.append(f"> View {view_name} is equivalent to tables {' and '.join(equivalent_class_name)}")
            view_comments = "\n".join(view_comments)

            equivalent_class_name = "\nNATURAL JOIN ".join(equivalent_class_name)


        view_definition = [{
            "viewName": view_name,
            "selectValue": "*",
            "equivalentClassName": equivalent_class_name,
            "viewComments": view_comments,
        }]

    return view_definition


# ttl_file = "oscars_expose_updated4.ttl"
# g = Graph()
# g.parse(ttl_file, format="turtle")

# # # all_classes = get_all_classes(g)

# # cls_map = {}
# # # prop_map = {}
# # # cls = URIRef("http://example.org/ontology#Regessure")
# # # for cls in all_classes:
# # #     defi, cls_map, prop_map, id_type = generate_table_info_for_class(g, cls, cls_map, prop_map)
# cls = URIRef("http://example.org/ontology#Regessure")

# # for cls in all_classes:
# test = view_definition_equivalnet_class(g, cls, cls_map)
# print(test)

### SQL Erstellung

Generiert aus generate_table_info_for_class() SQL-Code

In [ ]:
def sql_generate_datatype_check(column_name, restrictions_definition):
    checks = []

    for defi in restrictions_definition:
        restriction = defi["restriction"]
        restriction_value = defi["value"]

        # Vergleichsoperatoren für Zahlen
        if restriction == XSD.minInclusive:
            checks.append(f"{column_name} >= {restriction_value}")
        elif restriction == XSD.maxInclusive:
            checks.append(f"{column_name} <= {restriction_value}")
        elif restriction == XSD.minExclusive:
            checks.append(f"{column_name} > {restriction_value}")
        elif restriction == XSD.maxExclusive:
            checks.append(f"{column_name} < {restriction_value}")

        # String-Längen (nur sinnvoll für VARCHAR, TEXT etc.)
        elif restriction == XSD.length:
            checks.append(f"char_length({column_name}) = {restriction_value}")
        elif restriction == XSD.minLength:
            checks.append(f"char_length({column_name}) >= {restriction_value}")
        elif restriction == XSD.maxLength:
            checks.append(f"char_length({column_name}) <= {restriction_value}")

        # Regex-Muster
        elif restriction == XSD.pattern:
            pattern = str(restriction_value).replace("'", "''")  # SQL-Escaping
            checks.append(f"{column_name} ~ '^{pattern}$'")

        # Optional: Hinweise auf nicht direkt umsetzbare Facetten
        elif restriction == XSD.totalDigits or restriction == XSD.fractionDigits:
            print(f"⚠️ Hinweis: totalDigits/fractionDigits für '{column_name}' erkannt, aber nicht automatisch umsetzbar.")


    if checks:
        return f"CHECK ({' AND '.join(checks)})"
    else:
        None


In [404]:
def sql_create_table(table_definition):
    """_summary_

    :param definition: _description_
    :type definition: _type_
    :return: _description_
    :rtype: _type_
    """
    sql_table = []
    
    for defi in table_definition:
        if defi:
            table = defi["tableName"]
            table_comments = []
            table_comments_sql = []
            col_defs = []
            col_comments_sql = []
            col_comments = {}
            primary_keys = []
            foreign_key_line = []
            unique_constraints = []

            table_sql = []

            for col in defi["columns"]:
                
    ###### Attribut Name, Datatype und Foreign Key ######
                attribute_name = col["attributeName"]
                col_line = f'{attribute_name} {col["datatype"]}'

                # if not col.get("nullable", True):
                #     col_line += " NOT NULL"
                #          
                restrictions_definition = col.get("check_constraints")
                if restrictions_definition:
                    check_constraints = sql_generate_datatype_check(attribute_name, restrictions_definition)
                    col_line += f" {check_constraints}"

                col_defs.append(col_line)

    ###### Attribut Primary Key 1 ######
                if col.get("primaryKey"):
                    primary_keys.append(attribute_name)

    ###### Attribut Annotations ######
                if col.get("attributeAnnotations"):
                    for annotations in col["attributeAnnotations"]:
                        if col_comments.get(attribute_name):
                            col_comments[attribute_name].update({annotations})
                        else:
                            col_comments[attribute_name] = {annotations}

    ###### Attribut Single Value ######
                if col.get("singleValued"):
                    if col_comments.get(attribute_name) is not None:
                        col_comments[attribute_name].update({col['singleValued']})
                    else:
                        col_comments[attribute_name] = {col['singleValued']}

    ###### Attribut Comments ######            
                if col.get("comment"):
                    col_comments[attribute_name] = {col['comment']}   

                if col_comments:
                    comment_text = "\n".join(col_comments[col["attributeName"]])
                    col_comments_sql.append(
                        f"COMMENT ON COLUMN {table}.{attribute_name} IS '{comment_text}';"
                    )
                
    ###### Table Primary Key 2 ######
            if primary_keys:
                pk_line = f'PRIMARY KEY ({", ".join(primary_keys)})'
                col_defs.append(pk_line)
            
            if foreign_key_line:
                col_defs.append(", ".join(foreign_key_line))

    ###### Table Annotations ######        
            if defi["tableAnnotations"]:
                table_comments = defi["tableAnnotations"]
                # print(table_comments)
                comment_text = "\n".join(table_comments)
                col_comments_sql.append(
                    f"COMMENT ON TABLE {table} IS '{comment_text}';"
                )


            table_sql.extend([
                f"DROP TABLE IF EXISTS {table} CASCADE; \n"
                f"CREATE TABLE {table} (\n    " + ",\n    ".join(col_defs) + "\n);"
            ])

            table_sql.extend(col_comments_sql)
        
        sql_table.append("\n".join(table_sql))

    return "\n".join(sql_table)


ttl_file = "oscars_expose_updated4.ttl"
cls = URIRef("http://example.org/ontology#Director")

defi_table = []
cls_map = {}
prop_map = {}

# for cls in all_classes:
table_defi, cls_map, prop_map, sub_class_defi, trigger_defi = table_definition_class(g, cls, cls_map, prop_map)
if table_defi:
    # sql_table.append(sql_create_table(defi_table))
    defi_table.append(table_defi)

print(defi_table)

defi_sub_class_table, sub_class_key_definition = table_definition_sub_class_of()

sql = sql_create_table(defi_sub_class_table)

print(sql)

[{'tableName': 'director', 'tableAnnotations': ['> rdfs:label: Director', '> rdfs:comment: Macht einen Film', '> otherLabel: Director'], 'columns': [{'attributeName': 'id', 'primaryKey': True, 'datatype': 'VARCHAR', 'comment': 'Primary Key: Unique URI from the original Ontology'}, {'attributeName': 'label', 'datatype': 'VARCHAR', 'comment': 'rdfs:Label'}, {'attributeName': 'directed', 'attributeAnnotations': ['> rdfs:label: directed'], 'singleValued': 'functionalProperty: empirically functional', 'datatype': 'VARCHAR(50)'}, {'attributeName': 'wiki_data_id_of_a_director', 'attributeAnnotations': ['> rdfs:label: WikiData ID of a Director'], 'singleValued': 'functionalProperty: functional', 'datatype': 'VARCHAR(50)'}]}]
DROP TABLE IF EXISTS sub_class_table CASCADE; 
CREATE TABLE sub_class_table (
    sub_class VARCHAR,
    sub_class_label VARCHAR,
    super_class VARCHAR,
    super_class_label VARCHAR,
    PRIMARY KEY (sub_class, super_class)
);
COMMENT ON TABLE sub_class_table IS 'shows 

Generiert aus extract_instances() SQL-Code

In [395]:
def sql_insert_into(instance_definitions):
    """_summary_

    :param definition: _description_
    :type definition: _type_
    """

    sql_lines = []
    annotations = []

    for dict in instance_definitions:
        table = dict["table"]

        attributes = ", ".join(dict["values"].keys())
        values = ", ".join(f"'{str(v)}'" for v in dict["values"].values())
        sql_lines.append(
            f"INSERT INTO {table} ({attributes}) VALUES ({values});"
        )

    return "\n".join(sorted(sql_lines))


ttl_file = "oscars_expose_updated4.ttl"
# ttl_file = "test_reasoning.ttl"
g = Graph()
g.parse(ttl_file, format="turtle")

all_classes = get_all_classes(g)

cls_map = {}
prop_map = {}
id_type = get_primary_key(g)

for cls in all_classes:
    defi, cls_map, prop_map, defi_fks, defi_trigger = table_definition_class(g, cls, cls_map, prop_map)



defis = instance_definition(g, all_classes, cls_map, prop_map, id_type)
defis = entry_definition_sub_class_of(g, all_classes)

sql = sql_insert_into(defis)

print(sql)


INSERT INTO sub_class_table (sub_class, sub_class_label, super_class, super_class_label) VALUES ('Director', 'director', 'Film', 'film');
INSERT INTO sub_class_table (sub_class, sub_class_label, super_class, super_class_label) VALUES ('Director', 'director', 'Movie', 'movie123');
INSERT INTO sub_class_table (sub_class, sub_class_label, super_class, super_class_label) VALUES ('Film3', 'film3', 'Regessure', 'regessure');
INSERT INTO sub_class_table (sub_class, sub_class_label, super_class, super_class_label) VALUES ('Regessure', 'regessure', 'Director', 'director');
INSERT INTO sub_class_table (sub_class, sub_class_label, super_class, super_class_label) VALUES ('Regessure', 'regessure', 'Film', 'film');
INSERT INTO sub_class_table (sub_class, sub_class_label, super_class, super_class_label) VALUES ('Writer', 'writer', 'Movie', 'movie123');


In [383]:
def sql_create_foreign_key(foreign_key_definition):
    """_summary_

    :param foreign_key_definition: _description_
    :type foreign_key_definition: _type_
    """

    foreign_key_sql = []
    # defi = foreign_key_definition

    for defi in foreign_key_definition:

        fk = []
        foreign_key_table = defi["foreign_key_table"]
        foreign_key_column = defi["foreign_key_column"]
        target_table = defi["target_table"]
        target_column = defi["target_column"]

        if foreign_key_table != target_table:
            foreign_key_sql.append(
                f"ALTER TABLE {foreign_key_table}\n"
                f"ADD CONSTRAINT foreign_key_{foreign_key_table}_{foreign_key_column}\n" # WIrd hier nciht gebraucht
                f"FOREIGN KEY ({foreign_key_column})\n"
                f"REFERENCES {target_table}({target_column});"
                )
        else:
            foreign_key_sql.append(
                f"ALTER TABLE {foreign_key_table}\n"
                f"ADD CONSTRAINT foreign_key_{foreign_key_column}\n"
                f"FOREIGN KEY ({foreign_key_column})\n"
                f"REFERENCES {target_table}({target_column})\n"
                f"DEFERRABLE INITIALLY DEFERRED;"
                )
    
    return "\n".join(foreign_key_sql)


# ttl_file = "oscars_6_movies_updated.ttl"
ttl_file = "oscars_expose_updated4.ttl"

g = Graph()
g.parse(ttl_file, format="turtle")

cls = URIRef("http://example.org/ontology#Writer")
cls_mapping = {}
prop_mapping = {}

defi_table, cls_map, prop_map, sub_class_key_defi, sub_class_trigger_defi = table_definition_class(g, cls, cls_mapping, prop_mapping)

# print(sub_class_key_defi)

sub_class_key_defi = [{'foreign_key_table': 'film3', 'foreign_key_column': 'id', 'target_table': 'regessure', 'target_column': 'id'}, {'foreign_key_table': 'director', 'foreign_key_column': 'directed', 'target_table': 'movie123', 'target_column': 'id'}, {'foreign_key_table': 'movie123', 'foreign_key_column': 'directed_by', 'target_table': 'director', 'target_column': 'id'}, {'foreign_key_table': 'movie123', 'foreign_key_column': 'written_by', 'target_table': 'writer', 'target_column': 'id'}, {'foreign_key_table': 'writer', 'foreign_key_column': 'id', 'target_table': 'movie123', 'target_column': 'id'}, {'foreign_key_table': 'writer', 'foreign_key_column': 'wrote', 'target_table': 'movie123', 'target_column': 'id'}]

sql = sql_create_foreign_key(sub_class_key_defi)

print(sql)

ALTER TABLE film3
ADD CONSTRAINT foreign_key_film3_id
FOREIGN KEY (id)
REFERENCES regessure(id);
ALTER TABLE director
ADD CONSTRAINT foreign_key_director_directed
FOREIGN KEY (directed)
REFERENCES movie123(id);
ALTER TABLE movie123
ADD CONSTRAINT foreign_key_movie123_directed_by
FOREIGN KEY (directed_by)
REFERENCES director(id);
ALTER TABLE movie123
ADD CONSTRAINT foreign_key_movie123_written_by
FOREIGN KEY (written_by)
REFERENCES writer(id);
ALTER TABLE writer
ADD CONSTRAINT foreign_key_writer_id
FOREIGN KEY (id)
REFERENCES movie123(id);
ALTER TABLE writer
ADD CONSTRAINT foreign_key_writer_wrote
FOREIGN KEY (wrote)
REFERENCES movie123(id);


In [ ]:
def sql_create_trigger(trigger_definition):
    """_summary_

    :param trigger_definition: _description_
    :type trigger_definition: _type_
    """
    # Was ich brauche:
    #   Erstellung: CREATE TRIGGER triggerName *
    #   Zeitpunkt:	BEFORE, AFTER, INSTEAD OF
    #   Ereignis:	INSERT, UPDATE [OF ...], DELETE, TRUNCATE
    #   Ziel:       ON tabellenname *
    #   Gültigkeit:	FOR EACH ROW, FOR EACH STATEMENT
    #   Bedingung:	WHEN (Bedingung)
    #   Aktion:     EXECUTE FUNCTION trigger_funktion() *
    # * Pflichtfelder

    trigger_sql = []

    for trigger in trigger_definition:
        trigger_ = []
        if trigger.get("triggerName") and trigger.get("target") and trigger.get("function"):
            trigger_name = trigger.get("triggerName")
            trigger_point_time = trigger.get("pointInTime")
            trigger_event = trigger.get("event")
            trigger_target = trigger.get("target")
            trigger_validity = trigger.get("validity")
            trigger_condition = trigger.get("condition")
            trigger_function = trigger.get("function")

            trigger_.append(f"DROP TRIGGER IF EXISTS {trigger_name} ON {trigger_target};")
            trigger_.append(f"CREATE TRIGGER {trigger_name}")
            
            if trigger_point_time and trigger_event:
                trigger_.append(f"{trigger_point_time} {trigger_event} ON {trigger_target}")

            if trigger_validity:
                trigger_.append(f"FOR EACH {trigger_validity}")
            
            if trigger_condition:
                trigger_.append(f"WHEN ({trigger_condition})")
            
            trigger_.append(f"EXECUTE FUNCTION {trigger_function};")
        
        trigger_sql.append("\n".join(trigger_))

    return "\n".join(trigger_sql) 

# sub_class = URIRef("http://example.org/ontology#Writer")
# subcls_info = extract_classes(g, sub_class)[sub_class]
# subcls_name = subcls_info["sqlName"]["sql_name"]
# super_class = subcls_info["rdfs:subClassOf"]

# # trigger_definition = trigger_definition_sub_classes(subcls_name, super_class)
# # print(trigger_definition)
# defi_trigger = []
# cls_mapping = {}
# prop_mapping = {}

# for cls in all_classes:
#     defi_table, cls_map, prop_map, foreign_key_defi, trigger_defi = table_definition_class(g, cls, cls_mapping, prop_mapping)
#     if trigger_defi:
#         defi_trigger.extend(trigger_defi)

# sql = sql_create_trigger(defi_trigger)
# print(sql)

In [ ]:
def sql_create_view(view_definition):
    """_summary_

    :param view_definition: _description_
    :type view_definition: _type_
    """
    view_sql = []

    for defi in view_definition:

        view_name = defi["viewName"]
        select_value = defi["selectValue"]
        equivalent_class_name = defi["equivalentClassName"]
        view_comments = defi["viewComments"]

        view_sql.append(
            f"DROP VIEW IF EXISTS {view_name}; \n"
            f"CREATE VIEW {view_name} AS \n"
            f"SELECT {select_value} \n"
            f"FROM {equivalent_class_name}; \n"
            f"COMMENT ON VIEW {view_name} IS '{view_comments}';"
        ) 
    
    return "\n".join(view_sql)

# view_defi = [{'viewName': 'film2', 'selectValue': '*', 'equivalentClassName': 'director', 'viewComments': '> View film2 is equivalent to table director'}, {'viewName': 'regessure', 'selectValue': '*', 'equivalentClassName': 'writer\nNATURAL JOIN director', 'viewComments': '> annotations - rdfscomment: Regessure wird glaube ich vollkommen anderes geschrieben\n> annotations - other_label: BlaBlaBla\n> View regessure is equivalent to tables writer and director'}, {'viewName': 'film5', 'selectValue': '*', 'equivalentClassName': 'movie', 'viewComments': '> annotations - rdfscomment: EIn Film5 Kommentar\n> View film5 is equivalent to table movie'}]
# for a in view_defi:
#     print(a)
# sql = sql_create_view(view_defi)
# print(sql)

### Ω - Übertragung an PostgreSQL

In [452]:
db_config = {
    "host": "localhost",
    "database": "Oscar-Ontology",
    "user": "ragna",
    "password": "123456789",
    "port": "5434"
}

ttl_file = "Wine_reasoning2.ttl"
# ttl_file = "oscars_6_movies3_reasoning.ttl"
# ttl_file = "oscars_6_movies_updated.ttl"
# ttl_file = "oscars_expose_updated4.ttl"
g = Graph()
g.parse(ttl_file, format="turtle")

conn = psycopg2.connect(**db_config)
cursor = conn.cursor()


all_classes = get_all_classes(g)

cls_map = {}
prop_map = {}
id_type = get_primary_key(g)

sql_table = []
sql_view = []
sql_defi_trigger = []

defi_table = []
defi_trigger = []
defi_sub_class_fk = []
defi_view = []



# CREATE TABLE
for cls in all_classes:
    table_defi, cls_map, prop_map, sub_class_defi, trigger_defi = table_definition_class(g, cls, cls_map, prop_map)
    if table_defi:
        defi_table.append(table_defi)
    if trigger_defi:
        defi_trigger.extend(trigger_defi)
    if sub_class_defi:
        defi_sub_class_fk.extend(sub_class_defi)

# sql_table = sql_create_table(defi_table)
# print(sql_table)

defi_annotation_table = table_definition_instance_comments()
defi_table.append(defi_annotation_table)
# sql_table_annotation = sql_create_table([defi_annotation_table])

defi_table_all_classes, defi_fk = table_definition_all_classes_table()
defi_table.append(defi_table_all_classes)
# sql_table_all_classes = sql_create_table([defi_table_all_classes])
# print(sql_table_all_classes)

defi_table_sub_class, defi_sub_class_key = table_definition_sub_class_of()
defi_table.append(defi_table_sub_class)
# sql_table_sub_class = sql_create_table([defi_table_sub_class])


# sql_all_tables = "\n".join([
#     sql_table, 
    # sql_table_annotation, 
    # sql_table_all_classes, 
    # sql_table_sub_class
# ])


sql_all_tables = sql_create_table(defi_table)
# print(sql_all_tables)

if sql_all_tables:
    cursor.execute(sql_all_tables)
    conn.commit()


# INSERT INTO
defi_instances = instance_definition(g, all_classes, cls_map, prop_map, id_type)
sql_instances = sql_insert_into(defi_instances)

defi_annotation_instances = entry_definition_instance_comments(defi_instances)
sql_annotation_instances = sql_insert_into(defi_annotation_instances)

defi_all_classes_instances = entry_definition_all_classes(g, all_classes)
sql_all_classes_instances = sql_insert_into(defi_all_classes_instances)
# print(sql_all_classes_instances)

defi_sub_class_instances = entry_definition_sub_class_of(g, all_classes)
sql_sub_class_instances = sql_insert_into(defi_sub_class_instances)

sql_entries = "\n".join([
    "BEGIN;", 
    sql_instances, 
    sql_annotation_instances, 
    sql_all_classes_instances,
    sql_sub_class_instances, 
    "COMMIT;"
])

if sql_entries:
    cursor.execute(sql_entries)
    conn.commit()



# CREATE VIEW
# for cls in all_classes:
#     view_defi = view_definition_equivalnet_class(g, cls, cls_map)
#     if view_defi:
#         defi_view.extend(view_defi)

# sql_view = sql_create_view(defi_view)

# if sql_view:
#     cursor.execute(sql_view)
#     conn.commit()



# CREATE TRIGGER
sub_class_functions = sub_class_trigger_functions()
equivalent_class_functiom = equivalent_class_trigger_function()

sql_trigger = sql_create_trigger(defi_trigger)


all_triggers = "\n".join([
    equivalent_class_functiom,
    sub_class_functions,
    sql_trigger
])

if all_triggers:
    cursor.execute(all_triggers)
    conn.commit()



# ALTER FK
sql_fks = []
if defi_fk:
    sql_defi_fk = sql_create_foreign_key(defi_fk)
    sql_fks.append(sql_defi_fk)
if defi_sub_class_fk:
    sql_sub_class_fks = sql_create_foreign_key(defi_sub_class_fk)
    sql_fks.append(sql_sub_class_fks)
if defi_sub_class_key:
    sql_sub_class_table_fks = sql_create_foreign_key(defi_sub_class_key)
    sql_fks.append(sql_sub_class_table_fks)

sql_fks = "\n".join(sql_fks)

# if sql_fks:
#     cursor.execute(sql_fks)
#     conn.commit()

conn.close()
print("fertig")

UniqueViolation: FEHLER:  doppelter Schlüsselwert verletzt Unique-Constraint »wine_body_pkey«
DETAIL:  Schlüssel »(id)=(Full)« existiert bereits.


## Oscar-Ontology Erstellung


In [ ]:
def fetch_and_save_wikidata(file_type, output_file):
    """
    Sendet eine SPARQL CONSTRUCT-Abfrage an Wikidata, erhält RDF-Daten 
    und speichert sie sowohl im TURTLE- als auch RDF-Format.

    :param output_ttl: Dateiname für die TURTLE-Datei
    :param output_rdf: Dateiname für die RDF-Datei
    """
    # **1️⃣ Wikidata Query Service als Endpunkt**
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

    # **2️⃣ User-Agent setzen, um HTTP 403 Fehler zu vermeiden**
    sparql.addCustomHttpHeader("User-Agent", "MyWikidataBot/1.0 (mailto:me@example.com)")

    # **3️⃣ SPARQL CONSTRUCT-Abfrage definieren**
    sparql.setQuery("""                 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology#>

CONSTRUCT {
######### Individuen ######### 
  ?film a ex:Movie ;
          ex:movieTitel ?filmLabel ;
          rdfs:label ?filmLabel ;
          ex:movieID ?film ;
          ex:releaseYear ?releaseYear ;
          ex:hasGenre ?genre ;
          ex:hasDirector ?director ;
          ex:hasCastMember ?castMember ;
          ex:hasWriter ?writer ;
          ex:hasProducer ?producer ;
          ex:receivedAward  ?award ; 
          ex:wonAwardInYear ?awardYearMovie ;
          ex:runTime ?runTime .         
  ?genre a ex:Genre ;
           ex:genreName ?genreLabel ;
           rdfs:label ?genreLabel ;
           ex:genreID ?genre ;
           ex:genreOf ?film .
  ?director a ex:Director;
              ex:name ?directorLabel ;
              rdfs:label ?directorLabel ;
              ex:receivedAward ?awardDirector ;
              ex:directorID ?director ;
              ex:wonAwardInYear ?awardYearDirector ;
              ex:wonAwardFor ?awardFilmDirectorLabel ;
              ex:directed ?filmLabel .
  ?castMember a ex:CastMember ;
                ex:name ?castMemberLabel ;
                rdfs:label ?castMemberLabel ;
                ex:castID ?castMember ;
                ex:receivedAward ?awardCast ;
                ex:wonAwardFor ?awardFilmCast ;
                ex:wonAwardInYear ?awardYearCast ;
                ex:actedIn ?film .
  ?writer a ex:Writer ;
            ex:name ?writerLabel ;
            rdfs:label ?writerLabel ;
            ex:writerID ?writer ;
            ex:receivedAward ?awardWriter ;
            ex:wonAwardFor ?awardFilmWriterLabel ;
            ex:wonAwardInYear ?awardYearWriter ;
            ex:wrote ?filmLabel .
  ?producer a ex:Producer ;
              ex:name ?producerLabel ;
              rdfs:label ?producerLabel;
              ex:producerID ?producer;
              ex:receivedAward ?awardProducer ;
              ex:wonAwardFor ?awardFilmProducer;
              ex:wonAwardInYear ?awardYearProducer ;
              ex:produced ?film .
  ?award a ex:AwardCategory ;
           ex:awardName ?awardLabel ;
           rdfs:label ?awardLabel ;
           ex:awardID ?award;
           ex:awardedTo ?film ;
           ex:awardedTo ?winner .
          #  ex:wonAwardInYear ?awardYearMovie .           
  ?winner a ex:Winner ;
            ex:name ?winnerLabel ;
            rdfs:label ?winnerLabel ;
            ex:winnerID ?winner;
            ex:receivedAward ?award ;
            ex:wonAwardInYear ?awardYearMovie ;
            ex:wonAwardFor ?film .


                      
######### Properties #########
 
  ### Movie ###
  ex:hasCastMember a owl:ObjectProperty ;
                    rdfs:label "has cast" ;
                    rdfs:domain ex:Movie ;
                    rdfs:range ex:CastMember ;
                    rdfs:subPropertyOf ex:hasCastAndCrew .
  ex:hasWriter a owl:ObjectProperty ;
                 rdfs:label "written by" ;
                 rdfs:domain ex:Movie ;
                 rdfs:range ex:Writer ; 
                 rdfs:subPropertyOf ex:hasCastAndCrew .
  ex:hasProducer a owl:ObjectProperty ;
                  rdfs:label "produced by" ;
                  rdfs:domain ex:Movie ;
                  rdfs:range ex:Producer ;
                  rdfs:subPropertyOf ex:hasCastAndCrew .
  ex:hasDirector a owl:ObjectProperty ;
                  rdfs:label "directed by" ;
                  rdfs:domain ex:Movie ;
                  rdfs:range ex:Director ; 
                  rdfs:subPropertyOf ex:hasCastAndCrew .
  ex:hasGenre a owl:ObjectProperty ;
                rdfs:label "has gerne" ;
                rdfs:domain ex:Movie ;
                rdfs:range ex:Genre .
  ex:releaseYear a owl:DatatypeProperty ;
                  rdfs:domain ex:Movie ;
                  rdfs:range xsd:gYear .
  ex:runTime a owl:DatatypeProperty ;
              rdfs:domain ex:Movie ;
              rdfs:range xsd:decimal .
  ex:movieID a owl:DatatypeProperty ;
              rdfs:domain ex:Movie ;
              rdfs:range xsd:string ;
              rdfs:label "WikiData ID of a Movie" ;
              rdfs:subPropertyOf ex:wikiID .
  
  
  ### Genre ###
  ex:genreOf a owl:ObjectProperty ;
              rdfs:label "gerne of" ;
              rdfs:domain ex:Genre ;
              rdfs:range ex:Movie .
  ex:genreID a owl:DatatypeProperty ;
              rdfs:domain ex:Genre ;
              rdfs:range xsd:string ;
              rdfs:label "WikiData ID of a Genre" ;
              rdfs:subPropertyOf ex:wikiID .

  
  ## Director ###
  ex:directed a owl:ObjectProperty ;
                rdfs:label "directed" ;
                rdfs:domain ex:Director ;
                rdfs:range ex:Movie ; 
                rdfs:subPropertyOf ex:participatedIn .
  ex:directorID a owl:DatatypeProperty ;
              rdfs:domain ex:Director ;
              rdfs:range xsd:string ;
              rdfs:label "WikiData ID of a Director" ;
              rdfs:subPropertyOf ex:wikiID .


  ### CastMember ###
  ex:actedIn a owl:ObjectProperty ;
                rdfs:label "acted in" ;
                rdfs:domain ex:CastMember ;
                rdfs:range ex:Movie ;
                rdfs:subPropertyOf ex:participatedIn .
  ex:castID a owl:DatatypeProperty ;
              rdfs:domain ex:CastMember ;
              rdfs:range xsd:string ;
              rdfs:label "WikiData ID of a CastMember" ;
              rdfs:subPropertyOf ex:wikiID .


  ### Writer ###
  ex:wrote a owl:ObjectProperty ;
              rdfs:label "wrote" ;
              rdfs:domain ex:Writer ;
              rdfs:range ex:Movie ; 
              rdfs:subPropertyOf ex:participatedIn .
  ex:writerID a owl:DatatypeProperty ;
                rdfs:domain ex:Writer ;
                rdfs:range xsd:string ;
                rdfs:label "WikiData ID of a Writer" ;
                rdfs:subPropertyOf ex:wikiID .


  ### Producer ###
  ex:produced a owl:ObjectProperty ;
                rdfs:label "produced" ;
                rdfs:domain ex:Producer ;
                rdfs:range ex:Movie ;
                rdfs:subPropertyOf ex:participatedIn .
  ex:producerID a owl:DatatypeProperty ;
                  rdfs:domain ex:Producer ;
                  rdfs:range xsd:string ;
                  rdfs:label "WikiData ID of a Producer" ;
                  rdfs:subPropertyOf ex:wikiID .


  ### AwardCategory ###
  ex:awardedTo a owl:ObjectProperty ;
                        rdfs:label "awarded to" ;
                        rdfs:domain ex:AwardCategory ;
                        rdfs:range ex:AwardWinner .
  ex:awardID a owl:DatatypeProperty ;
                rdfs:domain ex:AwardCategory ;
                rdfs:range xsd:string ;
                rdfs:label "WikiData ID of a Oscar" ;
                rdfs:subPropertyOf ex:wikiID . 


  ### gemeinsame Properties ###
  ex:wonAwardFor a owl:ObjectProperty ;
                    rdfs:label "won Award for Movie" ;
                    rdfs:domain ex:AwardWinner ;
                    rdfs:range ex:Movie .
  ex:wonAwardInYear a owl:DatatypeProperty ;
                      rdfs:domain ex:AwardWinner ;
                      rdfs:range xsd:gYear .
  ex:receivedAward a owl:ObjectProperty ;
                      rdfs:label "won Award" ;
                      rdfs:domain ex:AwardWinner ;
                      rdfs:range ex:AwardCategory .


  ### Abstrakte Properties ###
  ex:participatedIn a owl:ObjectProperty ;
                      rdfs:label "participated in" ;
                      rdfs:domain ex:CastAndCrew ;
                      rdfs:range ex:Movie .
  ex:hasCastAndCrew a owl:ObjectProperty ;
                      rdfs:label "has Cast and Crew" ;
                      rdfs:domain ex:Movie ;
                      rdfs:range ex:CastAndCrew .
    ex:wikiID a owl:DatatypeProperty ;
                rdfs:label "WikiData Identifier" ;
                rdfs:domain ex:MovieProduction ; 
                rdfs:range xsd:string .

                    
# ######## Klassen #########
  ex:Movie a rdfs:Class ; 
            rdfs:subClassOf ex:MovieProduction ; 
            rdfs:comment "Movie enthält alle Filme die einen Oscar in mindestens einer diesieser Kategorien gewonnen haben: Best Pictur, Best Director, Best Original Screenplay, Best Adapted Screenplay, Best Actor in rdf:type Leading Role, Best Actress in a Leading Role, Best Actor in a Supporting Role und Best Actress in a Supporting Role." .
                    
  ex:Director a rdfs:Class ; 
                rdfs:subClassOf ex:CastAndCrew .
  ex:Writer a rdfs:Class ; 
              rdfs:subClassOf ex:CastAndCrew .
  ex:CastMember a rdfs:Class ; 
                  rdfs:subClassOf ex:CastAndCrew .
  ex:Producer a rdfs:Class ; 
                rdfs:subClassOf ex:CastAndCrew .
                    
  ex:DirectorWinner a rdfs:Class ; 
                      rdfs:subClassOf ex:Director, ex:Winner .
  ex:CastWinner a rdfs:Class ; 
                  rdfs:subClassOf ex:CastMember, ex:Winner .
  ex:WriterWinner a rdfs:Class ; 
                    rdfs:subClassOf ex:Writer, ex:Winner .
  ex:ProducerWinner a rdfs:Class ; 
                      rdfs:subClassOf ex:Producer, ex:Winner .
  ex:MovieWinner a rdfs:Class ; 
                   rdfs:subClassOf ex:Movie, ex:AwardWinner .
  ex:Winner a rdfs:Class ; 
              rdfs:subClassOf ex:AwardWinner .
  ex:AwardWinner a rdfs:Class .
  
  ex:Genre a rdfs:Class ; 
             rdfs:subClassOf ex:MovieProduction .
  ex:CastAndCrew a rdfs:Class ; 
                   rdfs:subClassOf ex:MovieProduction .
  ex:MovieProduction a rdfs:Class .                 
      
}                    
WHERE {
  
### Movie ###
  ?film wdt:P31 wd:Q11424.
  ?film rdfs:label ?filmLabel.
  FILTER(LANG(?filmLabel) = "en") 
  
  VALUES ?film {wd:Q44578 wd:Q208632 wd:Q171861 wd:Q133654 wd:Q13255497 wd:Q18002795}
  ?film p:P166 ?awardStat.
  ?awardStat ps:P166 ?award
             VALUES ?award {wd:Q102427 wd:Q103360 wd:Q103916 wd:Q103618 wd:Q41417 wd:Q107258 wd:Q106291 wd:Q106301}
  ?awardStat pq:P585 ?awardYear.
  
  BIND(YEAR(?awardYear) AS ?awardYearMovie)
  OPTIONAL { ?award rdfs:label ?awardLabel. FILTER(LANG(?awardLabel) = "en") }
  
  BIND(?awardYearMovie - 1 AS ?releaseYear) 
  
  ?film p:P2047 ?statement .
  ?statement ps:P2047 ?runTime .
  FILTER NOT EXISTS {
    ?film p:P2047 ?otherStatement .
    ?otherStatement ps:P2047 ?otherRuntime .
    FILTER(?otherRuntime < ?runTime)  # Behalte nur den kleinsten
  }

  
### Genre ###
    ?film wdt:P136 ?genre.
    ?genre rdfs:label ?genreLabel.    
    FILTER(LANG(?genreLabel) = "en") 

  
### Winner ###
  OPTIONAL {
    ?awardStat pq:P1346 ?winner .  # Gewinner (z. B. Produzenten, Schauspieler)
    ?winner rdfs:label ?winnerLabel .
    FILTER(LANG(?winnerLabel) = "en")
  }
 
  
### Cast Member ###
  OPTIONAL {
    ?film wdt:P161 ?castMember .  # Cast Member des Films
    ?castMember rdfs:label ?castMemberLabel .    
    FILTER(LANG(?castMemberLabel) = "en")
    
    # 🔹 Oscar-Gewinn überprüfen
    ?castMember p:P166 ?awardStatement. 
    ?awardStatement ps:P166 ?awardCast2
       VALUES ?awardCast2 {wd:Q103618 wd:Q103916 wd:Q106291 wd:Q106301}
  } 
 
  
  OPTIONAL{
    ?film p:P166 ?awardStat .
    ?film wdt:P161 ?castMember .
    ?awardStat ps:P166 ?awardCa .
    VALUES ?awardCa {wd:Q103618 wd:Q103916 wd:Q106291 wd:Q106301 }
    ?awardStat pq:P1346 ?winnerCa .  # Wer hat den Preis erhalten?

    FILTER(?winnerCa = ?castMember)  # Gewinner ist der Writer
    BIND(IF(?awardCa = wd:Q103916 || ?awardCa = wd:Q103618 || ?awardCa = wd:Q106291 || ?awardCa = wd:Q106301, ?awardCa, "" ) AS ?awardCast)
    BIND(?film AS ?awardFilmCast)
    ?awardStat pq:P585 ?awardYear.
    BIND(YEAR(?awardYear) AS ?awardYearCast)
    ?awardCast rdfs:label ?awardCastLabel. FILTER(LANG(?awardCastLabel) = "en")
    ?awardFilmCast rdfs:label ?awardFilmCastLabel. FILTER(LANG(?awardFilmCastLabel) = "en")
  }
   
### Director ###
    ?film wdt:P57 ?director. 
    ?director rdfs:label ?directorLabel.  
    FILTER(LANG(?directorLabel) = "en")

  OPTIONAL {
    ?film p:P166 ?awardStat .
    ?film wdt:P162 ?director.
    ?awardStat ps:P166 wd:Q103360 . 
    ?awardStat pq:P1346 ?winnerDi .  # Wer hat den Preis erhalten?

    # Prüfen, ob der Gewinner gleich dem Produzenten ist
    FILTER(?winnerDi = ?director)
    BIND(wd:Q103360 AS ?awardDirector)  # Award "Best Picture" zuordnen
    BIND(?film AS ?awardFilmDirector)
    ?awardStat pq:P585 ?awardYear.
    BIND(YEAR(?awardYear) AS ?awardYearDirector)
    ?awardDirector rdfs:label ?awardDirectorLabel. FILTER(LANG(?awardDirectorLabel) = "en")
    ?awardFilmDirector rdfs:label ?awardFilmDirectorLabel. FILTER(LANG(?awardFilmDirectorLabel) = "en")
  }
  
  
### Writer ### 
    ?film wdt:P58 ?writer.
    ?writer rdfs:label ?writerLabel.
    FILTER(LANG(?writerLabel) = "en")
  
  OPTIONAL {
    ?film p:P166 ?awardStat .
    ?film wdt:P58 ?writer .
    ?awardStat ps:P166 ?awardWr .
    VALUES ?awardWr {wd:Q41417 wd:Q107258}
    ?awardStat pq:P1346 ?winnerWr .  # Wer hat den Preis erhalten?

    FILTER(?winnerWr = ?writer)  # Gewinner ist der Writer
    BIND(IF(?awardWr = wd:Q41417 || ?awardWr = wd:Q107258, ?awardWr, "" ) AS ?awardWriter)
    BIND(?film AS ?awardFilmWriter)
    ?awardStat pq:P585 ?awardYear.
    BIND(YEAR(?awardYear) AS ?awardYearWriter)
    ?awardWriter rdfs:label ?awardWriterLabel. FILTER(LANG(?awardWriterLabel) = "en")
    ?awardFilmWriter rdfs:label ?awardFilmWriterLabel. FILTER(LANG(?awardFilmWriterLabel) = "en")
  }

  
### Producer ###
    ?film wdt:P162 ?producer.
    ?producer rdfs:label ?producerLabel.
    FILTER(LANG(?producerLabel) = "en")
  
  OPTIONAL {
    ?film p:P166 ?awardStat .
    ?film wdt:P162 ?producer.
    ?awardStat ps:P166 wd:Q102427 .  # Der Award ist "Best Picture"
    ?awardStat pq:P1346 ?winnerPr .  # Wer hat den Preis erhalten?

    FILTER(?winnerPr = ?producer)
    BIND(wd:Q102427 AS ?awardProducer)  # Award "Best Picture" zuordnen
    BIND(?film AS ?awardFilmProducer)
    ?awardStat pq:P585 ?awardYear.
    BIND(YEAR(?awardYear) AS ?awardYearProducer)
    ?awardProducer rdfs:label ?awardProducerLabel. FILTER(LANG(?awardProducerLabel) = "en")
    ?awardFilmProducer rdfs:label ?awardFilmProducerLabel. FILTER(LANG(?awardFilmProducerLabel) = "en")
  }

  
  # Zeitraum der letzten 10 Jahre (anpassen falls nötig)
  # FILTER(year(?awardYear) = 1998 )
  # FILTER(year(?awardYear) <= 2021 )
}
    """)

    # **4️⃣ TURTLE-Daten abrufen und speichern**
    sparql.setMethod("POST")  # Setze den Request auf POST, um große Abfragen zu unterstützen

    if file_type == 'turtle':    
      sparql.setReturnFormat(TURTLE)
      results = sparql.query().convert()  # RDF-Graph-Objekt erhalten

      # **Speicherung als TURTLE**
      with open(output_file, "w", encoding="utf-8") as f:
          f.write(results.serialize(format="turtle"))
      print(f"✅ TURTLE-Daten erfolgreich gespeichert in: oscars.ttl")
    
    else:
      # **5️⃣ RDF/XML-Daten abrufen und speichern**
      sparql.setReturnFormat(RDFXML)
      results = sparql.query().convert()

      # **Speicherung als RDF/XML**
      with open(output_file, "w", encoding="utf-8") as f:
          f.write(results.serialize(format="xml"))
      print(f"✅ RDF-Daten erfolgreich gespeichert in: oscars.rdf")

# **📌 Funktion aufrufen**
# fetch_and_save_wikidata("turtle", "oscars_6_movies3.ttl")
# fetch_and_save_wikidata("turtle", "oscars_example.ttl" )

c:\Users\ilove\AppData\Local\Programs\Python\Python310\lib\site-packages\SPARQLWrapper\Wrapper.py:1179: RuntimeWarning: Format requested was TURTLE, but RDF/XML (application/rdf+xml;charset=utf-8) has been returned by the endpoint
  warnings.warn(


✅ TURTLE-Daten erfolgreich gespeichert in: oscars.ttl


In der Ontology-Datei fehlen ein paar Klassenzuordnungen. Werden hiermit aber aller ergänzt.

In [ ]:
def add_missing_types(ttl_file, output_file):
    """
    Überprüft die RDF-Datei und ergänzt fehlende rdf:type-Einträge basierend auf bestimmten Attributen.

    :param ttl_file: Pfad zur RDF-Datei (XML oder Turtle)
    :param output_file: Pfad zur neuen, korrigierten RDF-Datei
    """
    # **1️⃣ RDF-Datei laden**
    g = Graph()
    g.parse(ttl_file, format="turtle")  # Falls Turtle: format="turtle"

    # RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")
    # subject = URIRef("http://www.wikidata.org/entity/Q150943")  # Der Eintrag, dem das Label hinzugefügt werden soll
    # label_value = Literal("Gene Hackman", lang="en")   # Das Label, das hinzugefügt werden soll
    # g.add((subject, RDFS.label, label_value))
    # print(f"✅ `rdfs:name` '{label_value}' wurde für {subject} hinzugefügt.")

    # **2️⃣ Wichtige URIs definieren**
    SCHEMA = URIRef("http://example.org/ontology#")
    
    # **Dictionary mit Attribut → erwarteter rdf:type**
    ATTR_TO_TYPE = {
        "directed": URIRef("http://example.org/ontology#Director"),
        "actedIn": URIRef("http://example.org/ontology#CastMember"),
        "produced": URIRef("http://example.org/ontology#Producer"),
        "wrote": URIRef("http://example.org/ontology#Writer"),
        # "receivedAward": URIRef("http://example.org/ontology#AwardWinner"),
        # "wonAwardFor": URIRef("http://example.org/ontology#AwardWinner")
    }

    # **3️⃣ Alle Individuals prüfen, die diese Attribute haben**
    for attribute, rdf_type in ATTR_TO_TYPE.items():
        attribute_uri = URIRef(f"http://example.org/ontology#{attribute}")  # Umwandlung in URIRef
        
        individuals_to_check = set()
        for s, p, o in g.triples((None, attribute_uri, None)):  # Alle, die dieses Attribut haben
            individuals_to_check.add(s)

        # **4️⃣ Überprüfen, ob diese Individuals bereits den erwarteten Typ haben**
        for individual in individuals_to_check:
            has_correct_type = any(o == rdf_type for _, _, o in g.triples((individual, RDF.type, None)))

            # **Falls der Typ fehlt, füge ihn hinzu**
            if not has_correct_type:
                print(f"➕ Füge rdf:type {rdf_type} für {individual} hinzu")
                g.add((individual, RDF.type, rdf_type))

    # **5️⃣ Geänderte RDF-Datei speichern**
    g.serialize(output_file, format="turtle", encoding="utf-8")

    print(f"✅ Datei gespeichert als '{output_file}'. Fehlende rdf:type-Einträge wurden ergänzt!")

# # **📌 Skript ausführen**
ttl_file = "oscars_6_movies3.ttl"  # Originaldatei
output_file = "oscars_6_movies_updated3.ttl"  # Geänderte Datei speichern


# add_missing_types(ttl_file, output_file)


✅ Datei gespeichert als 'oscars_6_movies_updated3.ttl'. Fehlende rdf:type-Einträge wurden ergänzt!


Überprüft ob für ein bestimmtes Attribut innerhalb einer KLasse mehrere Werte existieren

In [ ]:
def get_movies_with_multiple_award_years(ttl_file):
    """
    Findet alle Filme mit mehr als einem 'wonAwardInYear' Eintrag.

    :param ttl_file: Pfad zur RDF-Datei (XML oder Turtle)
    :return: Dictionary {Film: Set(AwardYears)}
    """
    # RDF-Datei laden
    g = Graph()
    g.parse(ttl_file, format="turtle")  # Falls Turtle: format="turtle"

    # Namespace für Schema.org
    MOVIE_CLASS = URIRef("http://schema.org/Movie")  # Klasse für Filme
    AWARD_YEAR = URIRef("http://schema.org/runTime")  # Attribut für gewonnenes Jahr

    # **1️⃣ Finde alle Filme in der RDF-Datei**
    movies = {s for s, _, o in g.triples((None, RDF.type, MOVIE_CLASS))}

    # **2️⃣ Finde alle Filme mit mehr als einem 'wonAwardInYear'**
    movies_with_award_years = {}

    for movie in movies:
        award_years = {o for _, _, o in g.triples((movie, AWARD_YEAR, None))}
        
        if len(award_years) > 1:  # Falls es mehrere AwardYears gibt
            movies_with_award_years[movie] = award_years

    return movies_with_award_years

# # **📌 Funktion aufrufen**
# ttl_file = "oscars.ttl"
# movies_with_awards = get_movies_with_multiple_award_years(ttl_file)

# # **📌 Ergebnisse ausgeben**
# print("\n📌 Filme mit mehr als einem Award-Jahr:")
# print(len(movies_with_awards))
# if movies_with_awards:
#     for movie, award_years in movies_with_awards.items():
#         print(f"🎬 Film: {movie}")
#         print(f"   🏆 AwardYears: {', '.join(map(str, award_years))}")
# else:
#     print("✅ Keine Filme mit mehreren Award-Jahren gefunden.")


Vergleicht zwei .ttl Dateien miteinander

In [ ]:
def load_rdf(file_path):
    """Lädt eine RDF-Datei und gibt sie als Graph zurück."""
    g = Graph()
    g.parse(file_path, format="turtle")  # Falls Turtle: format="turtle"
    return g

def compare_ttl_files(file1, file2):
    """Vergleicht zwei RDF-Dateien und gibt Unterschiede aus."""
    g1 = load_rdf(file1)
    g2 = load_rdf(file2)

    # Unterschiede finden
    only_in_g1 = set(g1) - set(g2)
    only_in_g2 = set(g2) - set(g1)

    if not only_in_g1 and not only_in_g2:
        print("✅ Die RDF-Dateien sind identisch!")
    else:
        print("⚠️ Unterschiede gefunden:")

        if only_in_g1:
            print("\n📌 In Datei 1, aber nicht in Datei 2:")
            for triple in only_in_g1:
                print(triple)

        if only_in_g2:
            print("\n📌 In Datei 2, aber nicht in Datei 1:")
            for triple in only_in_g2:
                print(triple)

# # Dateien zum Vergleich
# ttl_file_1 = "oscars_updated.rdf"
# ttl_file_2 = "oscars_updated2.rdf"

# compare_ttl_files(ttl_file_1, ttl_file_2)

### SPARQL-Code

alle Filme ohne Subproperties

In [ ]:
# PREFIX schema: <http://schema.org/>
# PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

# CONSTRUCT {
#   ?film a schema:Movie ;
#           rdfs:label ?filmLabel ;
#           schema:wikiID ?film ;
#           schema:releaseYear ?releaseYear;
#           schema:hasGenre ?genre ;
#           schema:hasDirector ?director ;
#           schema:hasCastMember ?castMember ;
#           schema:hasWriter ?writer ;
#           schema:hasProducer ?producer;
#           schema:receivedAwardFilm  ?award; 
#           schema:wonAwardInYear ?awardYearMovie ;
#           schema:runTime ?runTime .
#   ?genre a schema:Genre ;
#            rdfs:label ?genreLabel ;
#            schema:wikiID ?genre ;
#            schema:genreOf ?film .
#   ?director a schema:Director;
#               rdfs:label ?directorLabel ;
#               schema:receivedAwardDirector ?awardDirector ;
#               schema:wikiID ?director ;
#               schema:awardYear ?awardYearDirector ;
#               schema:wonAwardFor ?awardFilmDirector;
#               schema:directed ?film .
#   ?castMember a schema:CastMember ;
#                 rdfs:label ?castMemberLabel ;
#                 schema:wikiID ?castMember ;
#                 schema:receivedAwardCast ?awardCast ;
#                 schema:wonAwardFor ?awardFilmCast;
#                 schema:awardYear ?awardYearCast ;
#                 schema:actedIn ?film .
#   ?writer a schema:Writer ;
#             rdfs:label ?writerLabel ;
#             schema:wikiID ?writer ;
#             schema:receivedAwardWriter ?awardWriter ;
#             schema:wonAwardFor ?awardFilmWriter;
#             schema:awardYear ?awardYearWriter ;
#             schema:wrote ?film .
#   ?producer a schema:Producer ;
#               rdfs:label ?producerLabel;
#               schema:wikiID ?producer;
#               schema:receivedAwardProducer ?awardProducer ;
#               schema:wonAwardFor ?awardFilmProducer;
#               schema:awardYear ?awardYearProducer ;
#               schema:produced ?film .
#   ?award a schema:AwardCategory ;
#            rdfs:label ?awardLabel ;
#            schema:wikiID ?award;
#            schema:awardedToFilm ?film ;
#            schema:awardedToPerson ?winner ;
#            schema:awardYear ?awardYearMovie .           
#   ?winner a schema:AwardWinner ;
#             rdfs:label ?winnerLabel ;
#             schema:wikiID ?winner;
#             schema:receivedAward ?award ;
#             schema:awardYear ?awardYearMovie ;
#             schema:wonAwardFor ?film .


# # Klasse: Movie 
#   schema:hasCastMember a rdf:Property ;
#                          rdfs:label "has cast" ;
#                          rdfs:domain schema:Movie ;
#                          rdfs:range schema:CastMember .
#   schema:hasWriter a rdf:Property ;
#                      rdfs:label "written by" ;
#                      rdfs:domain schema:Movie ;
#                      rdfs:range schema:Writer .
#   schema:hasProducer a rdf:Property ;
#                        rdfs:label "produced by" ;
#                        rdfs:domain schema:Movie ;
#                        rdfs:range schema:Producer .
#   schema:hasDirector a rdf:Property ;
#                        rdfs:label "directed by" ;
#                        rdfs:domain schema:Movie ;
#                        rdfs:range schema:Director .
#   schema:hasGenre a rdf:Property ;
#                     rdfs:label "has gerne" ;
#                     rdfs:domain schema:Movie ;
#                     rdfs:range schema:Genre .
#   schema:receivedAwardFilm a rdf:Property;
#                           rdfs:label "won Awards" ;
#                           rdfs:domain schema:Movie ;
#                           rdfs:range schema:AwardCategory .

# # Klasse: Genre
#   schema:genreOf a rdf:Property ;
#                    rdfs:label "gerne of" ;
#                    rdfs:domain schema:Genre ;
#                    rdfs:range schema:Movie .

# # Klasse: Director
#   schema:directed a rdf:Property ;
#                     rdfs:label "directed" ;
#                     rdfs:domain schema:Director ;
#                     rdfs:range schema:Movie .
#   schema:receivedAwardDirector a rdf:Property ;
#                                  rdfs:label "won Award" ;
#                                  rdfs:domain schema:Director ;
#                                  rdfs:range schema:AwardCategory .
#   schema:wonAwardFor a rdf:Property ;
#                        rdfs:label "won Award for Movie" ;
#                        rdfs:domain schema:Director ;
#                        rdfs:range schema:Movie .

# # Klasse: CastMember  
#   schema:actedIn a rdf:Property ;
#                    rdfs:label "acted in" ;
#                    rdfs:domain schema:CastMember ;
#                    rdfs:range schema:Movie .
#   schema:receivedAwardCast a rdf:Property ;
#                              rdfs:label "won Award" ;
#                              rdfs:domain schema:CastMember ;
#                              rdfs:range schema:AwardCategory .
#   schema:wonAwardFor a rdf:Property ;
#                        rdfs:label "won Award for Movie" ;
#                        rdfs:domain schema:CastMember ;
#                        rdfs:range schema:Movie .

# # Klasse: Writer
#   schema:wrote a rdf:Property ;
#                  rdfs:label "wrote" ;
#                  rdfs:domain schema:Writer ;
#                  rdfs:range schema:Movie .
#   schema:receivedAwardWriter a rdf:Property ;
#                           rdfs:label "won Award" ;
#                           rdfs:domain schema:Writer ;
#                           rdfs:range schema:AwardCategory .
#   schema:wonAwardFor a rdf:Property ;
#                        rdfs:label "won Award for Movie" ;
#                        rdfs:domain schema:Writer ;
#                        rdfs:range schema:Movie .

# # Klasse: Producer
#   schema:produced a rdf:Property ;
#                     rdfs:label "produced" ;
#                     rdfs:domain schema:Producer ;
#                     rdfs:range schema:Movie .
#   schema:receivedAwardProducer a rdf:Property ;
#                           rdfs:label "won Award" ;
#                           rdfs:domain schema:Producer ;
#                           rdfs:range schema:AwardCategory .
#   schema:wonAwardFor a rdf:Property ;
#                        rdfs:label "won Award for Movie" ;
#                        rdfs:domain schema:Producer ;
#                        rdfs:range schema:Movie .

# # Klasse: AwardedCategory
#   schema:awardedToFilm a rdf:Property ;
#                          rdfs:label "awarded to" ;
#                          rdfs:domain schema:AwardCategory ;
#                          rdfs:range schema:Movie .
#   schema:awardedToPerson a rdf:Property ;
#                            rdfs:label "awarded to" ;
#                            rdfs:domain schema:AwardCategory ;
#                            rdfs:range schema:AwardWinner .
  
    

#   schema:Director rdfs:subClassOf schema:CastAndCrew .
#   schema:CastMember rdfs:subClassOf schema:CastAndCrew . 
#   schema:Writer rdfs:subClassOf schema:CastAndCrew .
#   schema:Producer rdfs:subClassOf schema:CastAndCrew .
#   schema:awardWinner rdfs:subClassOf schema:CastAndCrew.
  
#   schema:Movie rdfs:subClassOf schema:MovieProduction.
#   schema:Genre rdfs:subClassOf schema:MovieProduction.
#   schema:CastAndCrew rdfs:subClassOf schema:MovieProduction.
 
# }
                    
# WHERE {

# ### Movie ### 
#   ?film wdt:P31 wd:Q11424.
#   ?film rdfs:label ?filmLabel.
#   FILTER(LANG(?filmLabel) = "en") 
  
# #   VALUES ?film { wd:Q500672} #wd:Q208632 } # wd:Q44578 wd:Q160060 wd:Q125076 wd:Q134773 wd:Q20856802 }
#   ?film p:P166 ?awardStat.
#   ?awardStat ps:P166 ?award
#              VALUES ?award { wd:Q102427 wd:Q103360 wd:Q103916 wd:Q103618 wd:Q41417 wd:Q107258 wd:Q106291 wd:Q106301}
#   ?awardStat pq:P585 ?awardYear.
  
#   BIND(YEAR(?awardYear) AS ?awardYearMovie)
#   OPTIONAL { ?award rdfs:label ?awardLabel. FILTER(LANG(?awardLabel) = "en") }
  
#   BIND(?awardYearMovie - 1 AS ?releaseYear) 
  
#   ?film p:P2047 ?statement .
#   ?statement ps:P2047 ?runTime .
#   FILTER NOT EXISTS {
#     ?film p:P2047 ?otherStatement .
#     ?otherStatement ps:P2047 ?otherRuntime .
#     FILTER(?otherRuntime < ?runTime)  # Behalte nur den kleinsten
#   }
  

# ### Genre ###
#     ?film wdt:P136 ?genre.
#     ?genre rdfs:label ?genreLabel.    
#     FILTER(LANG(?genreLabel) = "en") 


# ### WInner ### 
#   OPTIONAL {
#     ?awardStat pq:P1346 ?winner .  # Gewinner (z. B. Produzenten, Schauspieler)
#     ?winner rdfs:label ?winnerLabel .
#     FILTER(LANG(?winnerLabel) = "en")
#   }
  

# ### CastMember ###
#   OPTIONAL {
#     ?film wdt:P161 ?castMember .  # Cast Member des Films
#     ?castMember rdfs:label ?castMemberLabel .    
#     FILTER(LANG(?castMemberLabel) = "en")
    
#     # 🔹 Oscar-Gewinn überprüfen
#     ?castMember p:P166 ?awardStatement. 
#     ?awardStatement ps:P166 ?awardCast2
#        VALUES ?awardCast2 {wd:Q103618 wd:Q103916 wd:Q106291 wd:Q106301}
#   } 
 
#   OPTIONAL{
#     ?film p:P166 ?awardStat .
#     ?film wdt:P161 ?castMember .
#     ?awardStat ps:P166 ?awardCa .
#     VALUES ?awardCa {wd:Q103618 wd:Q103916 wd:Q106291 wd:Q106301 }
#     ?awardStat pq:P1346 ?winnerCa .  # Wer hat den Preis erhalten?

#     FILTER(?winnerCa = ?castMember)  # Gewinner ist der Writer
#     BIND(IF(?awardCa = wd:Q103916 || ?awardCa = wd:Q103618 || ?awardCa = wd:Q106291 || ?awardCa = wd:Q106301, ?awardCa, "" ) AS ?awardCast)
#     BIND(?film AS ?awardFilmCast)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearCast)
#     ?awardCast rdfs:label ?awardCastLabel. FILTER(LANG(?awardCastLabel) = "en")
#     ?awardFilmCast rdfs:label ?awardFilmCastLabel. FILTER(LANG(?awardFilmCastLabel) = "en")
#   }
   

# ### Director ###
#     ?film wdt:P57 ?director. 
#     ?director rdfs:label ?directorLabel.  
#     FILTER(LANG(?directorLabel) = "en")

#   OPTIONAL {
#     ?film p:P166 ?awardStat .
#     ?film wdt:P162 ?director.
#     ?awardStat ps:P166 wd:Q103360 . 
#     ?awardStat pq:P1346 ?winnerDi .  # Wer hat den Preis erhalten?

#     FILTER(?winnerDi = ?director)
#     BIND(wd:Q103360 AS ?awardDirector)  # Award "Best Picture" zuordnen
#     BIND(?film AS ?awardFilmDirector)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearDirector)
#     ?awardDirector rdfs:label ?awardDirectorLabel. FILTER(LANG(?awardDirectorLabel) = "en")
#     ?awardFilmDirector rdfs:label ?awardFilmDirectorLabel. FILTER(LANG(?awardFilmDirectorLabel) = "en")
#   }
  

# ### Writer ### 
#     ?film wdt:P58 ?writer.
#     ?writer rdfs:label ?writerLabel.
#     FILTER(LANG(?writerLabel) = "en")
  
#   OPTIONAL {
#     ?film p:P166 ?awardStat .
#     ?film wdt:P58 ?writer .
#     ?awardStat ps:P166 ?awardWr .
#     VALUES ?awardWr {wd:Q41417 wd:Q107258}
#     ?awardStat pq:P1346 ?winnerWr .  # Wer hat den Preis erhalten?

#     FILTER(?winnerWr = ?writer)  # Gewinner ist der Writer
#     BIND(IF(?awardWr = wd:Q41417 || ?awardWr = wd:Q107258, ?awardWr, "" ) AS ?awardWriter)
#     BIND(?film AS ?awardFilmWriter)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearWriter)
#     ?awardWriter rdfs:label ?awardWriterLabel. FILTER(LANG(?awardWriterLabel) = "en")
#     ?awardFilmWriter rdfs:label ?awardFilmWriterLabel. FILTER(LANG(?awardFilmWriterLabel) = "en")
#   }


# ### Producer ###  
#     ?film wdt:P162 ?producer.
#     ?producer rdfs:label ?producerLabel.
#     FILTER(LANG(?producerLabel) = "en")
  
#   OPTIONAL {
#     ?film p:P166 ?awardStat .
#     ?film wdt:P162 ?producer.
#     ?awardStat ps:P166 wd:Q102427 .  # Der Award ist "Best Picture"
#     ?awardStat pq:P1346 ?winnerPr .  # Wer hat den Preis erhalten?

#     FILTER(?winnerPr = ?producer)
#     BIND(wd:Q102427 AS ?awardProducer)  # Award "Best Picture" zuordnen
#     BIND(?film AS ?awardFilmProducer)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearProducer)
#     ?awardProducer rdfs:label ?awardProducerLabel. FILTER(LANG(?awardProducerLabel) = "en")
#     ?awardFilmProducer rdfs:label ?awardFilmProducerLabel. FILTER(LANG(?awardFilmProducerLabel) = "en")
#   }

  
#   # Zeitraum der letzten 10 Jahre (anpassen falls nötig)
# #   FILTER(year(?awardYear) >= 2020 )
# #   FILTER(year(?awardYear) <= 2021 )
# }

alle Filme mit abstrakten Subproperties

In [ ]:
# PREFIX schema: <http://schema.org/>
# PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

# CONSTRUCT {
# ######### Klassen #########
  
#   ?film a schema:Movie ;
#           schema:movieTitel ?filmLabel ;
#           rdfs:label ?filmLabel ;
#           schema:wikiID ?film ;
#           schema:releaseYear ?releaseYear;
#           schema:hasGenre ?genre ;
#           schema:hasDirector ?director ;
#           schema:hasCastMember ?castMember ;
#           schema:hasWriter ?writer ;
#           schema:hasProducer ?producer;
#           schema:receivedAward  ?award; 
#           schema:wonAwardInYear ?awardYearMovie ;
#           schema:runTime ?runTime .
#   ?genre a schema:Genre ;
#            schema:genreName ?genreLabel ;
#            rdfs:label ?genreLabel ;
#            schema:wikiID ?genre ;
#            schema:genreOf ?film .
#   ?director a schema:Director;
#               schema:name ?directorLabel ;
#               rdfs:label ?directorLabel ;
#               schema:receivedAward ?awardDirector ;
#               schema:wikiID ?director ;
#               schema:wonAwardInYear ?awardYearDirector ;
#               schema:wonAwardFor ?awardFilmDirector;
#               schema:directed ?film .
#   ?castMember a schema:CastMember ;
#                 schema:name ?castMemberLabel ;
#                 rdfs:label ?castMemberLabel ;
#                 schema:wikiID ?castMember ;
#                 schema:receivedAward ?awardCast ;
#                 schema:wonAwardFor ?awardFilmCast;
#                 schema:wonAwardInYear ?awardYearCast ;
#                 schema:actedIn ?film .
#   ?writer a schema:Writer ;
#             schema:name ?writerLabel ;
#             rdfs:label ?writerLabel ;
#             schema:wikiID ?writer ;
#             schema:receivedAward ?awardWriter ;
#             schema:wonAwardFor ?awardFilmWriter;
#             schema:wonAwardInYear ?awardYearWriter ;
#             schema:wrote ?film .
#   ?producer a schema:Producer ;
#               schema:name ?producerLabel ;
#               rdfs:label ?producerLabel;
#               schema:wikiID ?producer;
#               schema:receivedAward ?awardProducer ;
#               schema:wonAwardFor ?awardFilmProducer;
#               schema:wonAwardInYear ?awardYearProducer ;
#               schema:produced ?film .
#   ?award a schema:AwardCategory ;
#            schema:awardName ?awardLabel ;
#            rdfs:label ?awardLabel ;
#            schema:wikiID ?award;
#            schema:awardedTo ?film ;
#            schema:awardedTo ?winner .
# #            schema:wonAwardInYear ?awardYearMovie .           
#   ?winner a schema:Winner ;
#             schema:name ?winnerLabel ;
#             rdfs:label ?winnerLabel ;
#             schema:wikiID ?winner;
#             schema:receivedAward ?award ;
#             schema:wonAwardInYear ?awardYearMovie ;
#             schema:wonAwardFor ?film .

  
  
# ######### Properties #########
  
#   ### Movie ###
#   schema:hasCastMember a rdf:Property ;
#                          rdfs:label "has cast" ;
#                          rdfs:domain schema:Movie ;
#                          rdfs:range schema:CastMember ;
#                          rdfs:subPropertyOf schema:hasCastAndCrew .
#   schema:hasWriter a rdf:Property ;
#                      rdfs:label "written by" ;
#                      rdfs:domain schema:Movie ;
#                      rdfs:range schema:Writer ;
#                      rdfs:subPropertyOf schema:hasCastAndCrew .
#   schema:hasProducer a rdf:Property ;
#                        rdfs:label "produced by" ;
#                        rdfs:domain schema:Movie ;
#                        rdfs:range schema:Producer ;
#                        rdfs:subPropertyOf schema:hasCastAndCrew .
#   schema:hasDirector a rdf:Property ;
#                        rdfs:label "directed by" ;
#                        rdfs:domain schema:Movie ;
#                        rdfs:range schema:Director ;
#                        rdfs:subPropertyOf schema:hasCastAndCrew .
#   schema:hasGenre a rdf:Property ;
#                     rdfs:label "has gerne" ;
#                     rdfs:domain schema:Movie ;
#                     rdfs:range schema:Genre .
#   schema:releaseYear a rdf:Property ;
#                        rdfs:domain schema:Movie ;
#                        rdfs:range xsd:gYear .
#   schema:runTime a rdf:Property ;
#                        rdfs:domain schema:Movie ;
#                        rdfs:range xsd:Integer .
  
  
#   ### Genre ###
#   schema:genreOf a rdf:Property ;
#                    rdfs:label "gerne of" ;
#                    rdfs:domain schema:Genre ;
#                    rdfs:range schema:Movie .

  
#   ## Director ###
#   schema:directed a rdf:Property ;
#                     rdfs:label "directed" ;
#                     rdfs:domain schema:Director ;
#                     rdfs:range schema:Movie ;
#                     rdfs:subPropertyOf schema:participatedIn .
  
  
#   ### CastMember ###
#   schema:actedIn a rdf:Property ;
#                    rdfs:label "acted in" ;
#                    rdfs:domain schema:CastMember ;
#                    rdfs:range schema:Movie ;
#                    rdfs:subPropertyOf schema:participatedIn .

  
#   ### Writer ###
#   schema:wrote a rdf:Property ;
#                  rdfs:label "wrote" ;
#                  rdfs:domain schema:Writer ;
#                  rdfs:range schema:Movie ;
#                  rdfs:subPropertyOf schema:participatedIn .

  
#   ### Producer ###
#   schema:produced a rdf:Property ;
#                     rdfs:label "produced" ;
#                     rdfs:domain schema:Producer ;
#                     rdfs:range schema:Movie ;
#                     rdfs:subPropertyOf schema:participatedIn .
   
  
#   ### AwardCategory ###
#   schema:awardedTo a rdf:Property ;
#                            rdfs:label "awarded to" ;
#                            rdfs:domain schema:AwardCategory ;
#                            rdfs:range schema:AwardWinner . 
  
 
#   ### gemeinsame Properties ###
#   schema:wonAwardFor a rdf:Property ;
#                        rdfs:label "won Award for Movie" ;
#                        rdfs:domain schema:AwardWinner ;
#                        rdfs:range schema:Movie .
#   schema:wonAwardInYear a rdf:Property ;
#                           rdfs:domain schema:AwardWinner ;
#                           rdfs:range xsd:gYear .
#   schema:receivedAward a rdf:Property ;
#                          rdfs:label "won Award" ;
#                          rdfs:domain schema:AwardWinner ;
#                          rdfs:range schema:AwardCategory .
  
  
#   ### Abstrakte Properties ###
#   schema:participatedIn a rdf:Property ;
#                           rdfs:label "participated in" ;
#                           rdfs:domain schema:CastAndCrew ;
#                           rdfs:range schema:Movie .
#   schema:hasCastAndCrew a rdf:Property ;
#                           rdfs:label "has Cast and Crew" ;
#                           rdfs:domain schema:Movie ;
#                           rdfs:range schema:CastAndCrew .
  
      
# ######### Oberklassen #########
  
#   schema:Director rdfs:subClassOf schema:CastAndCrew .
#   schema:CastMember rdfs:subClassOf schema:CastAndCrew . 
#   schema:Writer rdfs:subClassOf schema:CastAndCrew .
#   schema:Producer rdfs:subClassOf schema:CastAndCrew .
#   schema:Winner rdfs:subClassOf schema:CastAndCrew.
  
#   schema:DirectorWinner rdfs:subClassOf schema:Director.
#   schema:CastWinner rdfs:subClassOf schema:CastMember.
#   schema:WriterWinner rdfs:subClassOf schema:Writer.
#   schema:ProducerWinner rdfs:subClassOf schema:Producer.
#   schema:MovieWinner rdfs:subClassOf schema:Movie.
  
#   schema:DirectorWinner rdfs:subClassOf schema:Winner.
#   schema:CastWinner rdfs:subClassOf schema:Winner.
#   schema:WriterWinner rdfs:subClassOf schema:Winner.
#   schema:ProducerWinner rdfs:subClassOf schema:Winner.
  
#   schema:MovieWinner rdfs:subClassOf schema:AwardWinner.
#   schema:Winner rdfs:subClassOf schema:AwardWinner.
  
#   schema:AwardWinner rdfs:subClassOf schema:MovieProduction .
#   schema:Movie rdfs:subClassOf schema:MovieProduction.
#   schema:Genre rdfs:subClassOf schema:MovieProduction.
#   schema:CastAndCrew rdfs:subClassOf schema:MovieProduction.
# }                    
# WHERE {
  
# ### Movie ###
#   ?film wdt:P31 wd:Q11424.
#   ?film rdfs:label ?filmLabel.
#   FILTER(LANG(?filmLabel) = "en") 
  
#   VALUES ?film { wd:Q44578 } # wd:Q208632 wd:Q160060 wd:Q125076 wd:Q134773 wd:Q20856802 }
#   ?film p:P166 ?awardStat.
#   ?awardStat ps:P166 ?award
#              VALUES ?award { wd:Q102427 wd:Q103360 wd:Q103916 wd:Q103618 wd:Q41417 wd:Q107258 wd:Q106291 wd:Q106301}
#   ?awardStat pq:P585 ?awardYear.
  
#   BIND(YEAR(?awardYear) AS ?awardYearMovie)
#   OPTIONAL { ?award rdfs:label ?awardLabel. FILTER(LANG(?awardLabel) = "en") }
  
#   BIND(?awardYearMovie - 1 AS ?releaseYear) 
  
#   ?film p:P2047 ?statement .
#   ?statement ps:P2047 ?runTime .
#   FILTER NOT EXISTS {
#     ?film p:P2047 ?otherStatement .
#     ?otherStatement ps:P2047 ?otherRuntime .
#     FILTER(?otherRuntime < ?runTime)  # Behalte nur den kleinsten
#   }

  
# ### Genre ###
#     ?film wdt:P136 ?genre.
#     ?genre rdfs:label ?genreLabel.    
#     FILTER(LANG(?genreLabel) = "en") 

  
# ### Winner ###
#   OPTIONAL {
#     ?awardStat pq:P1346 ?winner .  # Gewinner (z. B. Produzenten, Schauspieler)
#     ?winner rdfs:label ?winnerLabel .
#     FILTER(LANG(?winnerLabel) = "en")
#   }
 
  
# ### Cast Member ###
#   OPTIONAL {
#     ?film wdt:P161 ?castMember .  # Cast Member des Films
#     ?castMember rdfs:label ?castMemberLabel .    
#     FILTER(LANG(?castMemberLabel) = "en")
    
#     # 🔹 Oscar-Gewinn überprüfen
#     ?castMember p:P166 ?awardStatement. 
#     ?awardStatement ps:P166 ?awardCast2
#        VALUES ?awardCast2 {wd:Q103618 wd:Q103916 wd:Q106291 wd:Q106301}
#   } 
 
  
#   OPTIONAL{
#     ?film p:P166 ?awardStat .
#     ?film wdt:P161 ?castMember .
#     ?awardStat ps:P166 ?awardCa .
#     VALUES ?awardCa {wd:Q103618 wd:Q103916 wd:Q106291 wd:Q106301 }
#     ?awardStat pq:P1346 ?winnerCa .  # Wer hat den Preis erhalten?

#     FILTER(?winnerCa = ?castMember)  # Gewinner ist der Writer
#     BIND(IF(?awardCa = wd:Q103916 || ?awardCa = wd:Q103618 || ?awardCa = wd:Q106291 || ?awardCa = wd:Q106301, ?awardCa, "" ) AS ?awardCast)
#     BIND(?film AS ?awardFilmCast)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearCast)
#     ?awardCast rdfs:label ?awardCastLabel. FILTER(LANG(?awardCastLabel) = "en")
#     ?awardFilmCast rdfs:label ?awardFilmCastLabel. FILTER(LANG(?awardFilmCastLabel) = "en")
#   }
   
# ### Director ###
#     ?film wdt:P57 ?director. 
#     ?director rdfs:label ?directorLabel.  
#     FILTER(LANG(?directorLabel) = "en")

#   OPTIONAL {
#     ?film p:P166 ?awardStat .
#     ?film wdt:P162 ?director.
#     ?awardStat ps:P166 wd:Q103360 . 
#     ?awardStat pq:P1346 ?winnerDi .  # Wer hat den Preis erhalten?

#     # Prüfen, ob der Gewinner gleich dem Produzenten ist
#     FILTER(?winnerDi = ?director)
#     BIND(wd:Q103360 AS ?awardDirector)  # Award "Best Picture" zuordnen
#     BIND(?film AS ?awardFilmDirector)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearDirector)
#     ?awardDirector rdfs:label ?awardDirectorLabel. FILTER(LANG(?awardDirectorLabel) = "en")
#     ?awardFilmDirector rdfs:label ?awardFilmDirectorLabel. FILTER(LANG(?awardFilmDirectorLabel) = "en")
#   }
  
  
# ### Writer ### 
#     ?film wdt:P58 ?writer.
#     ?writer rdfs:label ?writerLabel.
#     FILTER(LANG(?writerLabel) = "en")
  
#   OPTIONAL {
#     ?film p:P166 ?awardStat .
#     ?film wdt:P58 ?writer .
#     ?awardStat ps:P166 ?awardWr .
#     VALUES ?awardWr {wd:Q41417 wd:Q107258}
#     ?awardStat pq:P1346 ?winnerWr .  # Wer hat den Preis erhalten?

#     FILTER(?winnerWr = ?writer)  # Gewinner ist der Writer
#     BIND(IF(?awardWr = wd:Q41417 || ?awardWr = wd:Q107258, ?awardWr, "" ) AS ?awardWriter)
#     BIND(?film AS ?awardFilmWriter)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearWriter)
#     ?awardWriter rdfs:label ?awardWriterLabel. FILTER(LANG(?awardWriterLabel) = "en")
#     ?awardFilmWriter rdfs:label ?awardFilmWriterLabel. FILTER(LANG(?awardFilmWriterLabel) = "en")
#   }

  
# ### Producer ###
#     ?film wdt:P162 ?producer.
#     ?producer rdfs:label ?producerLabel.
#     FILTER(LANG(?producerLabel) = "en")
  
#   OPTIONAL {
#     ?film p:P166 ?awardStat .
#     ?film wdt:P162 ?producer.
#     ?awardStat ps:P166 wd:Q102427 .  # Der Award ist "Best Picture"
#     ?awardStat pq:P1346 ?winnerPr .  # Wer hat den Preis erhalten?

#     FILTER(?winnerPr = ?producer)
#     BIND(wd:Q102427 AS ?awardProducer)  # Award "Best Picture" zuordnen
#     BIND(?film AS ?awardFilmProducer)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearProducer)
#     ?awardProducer rdfs:label ?awardProducerLabel. FILTER(LANG(?awardProducerLabel) = "en")
#     ?awardFilmProducer rdfs:label ?awardFilmProducerLabel. FILTER(LANG(?awardFilmProducerLabel) = "en")
#   }

  
#   # Zeitraum der letzten 10 Jahre (anpassen falls nötig)
# #   FILTER(year(?awardYear) >= 2020 )
# #   FILTER(year(?awardYear) <= 2021 )
# }

Filme zwischen 2010 und 2025 mit echten Subproperties. 

In [ ]:
# PREFIX schema: <http://schema.org/>
# PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>


# CONSTRUCT {
#   ?film a schema:Movie ;
#           rdfs:label ?filmLabel ;
#           schema:wikiID ?film ;
#           schema:releaseYear ?releaseYear;
#           schema:hasGenre ?genre ;
#           schema:hasDirector ?director ;
#           schema:hasCastMember ?castMember ;
#           schema:hasWriter ?writer ;
#           schema:hasProducer ?producer;
#           schema:receivedAwardFilm  ?award; 
#           schema:wonAwardInYear ?awardYearMovie ;
#           schema:runTime ?runTime ;
#           schema:hasCastAndCrew ?castAndCrew ;
#           schema:hasParticipant ?castAndCrew .
#   ?genre a schema:Genre ;
#            rdfs:label ?genreLabel ;
#            schema:wikiID ?genre ;
#            schema:genreOf ?film .
#   ?director a schema:Director;
#               rdfs:label ?directorLabel ;
#               schema:receivedAwardDirector ?awardDirector ;
#               schema:wikiID ?director ;
#               schema:awardYear ?awardYearDirector ;
#               schema:wonAwardFor ?awardFilmDirector;
#               schema:directed ?film .
#   ?castMember a schema:CastMember ;
#                 rdfs:label ?castMemberLabel ;
#                 schema:wikiID ?castMember ;
#                 schema:receivedAwardCast ?awardCast ;
#                 schema:wonAwardFor ?awardFilmCast;
#                 schema:awardYear ?awardYearCast ;
#                 schema:actedIn ?film .
#   ?writer a schema:Writer ;
#             rdfs:label ?writerLabel ;
#             schema:wikiID ?writer ;
#             schema:receivedAwardWriter ?awardWriter ;
#             schema:wonAwardFor ?awardFilmWriter;
#             schema:awardYear ?awardYearWriter ;
#             schema:wrote ?film .
#   ?producer a schema:Producer ;
#               rdfs:label ?producerLabel;
#               schema:wikiID ?producer;
#               schema:receivedAwardProducer ?awardProducer ;
#               schema:wonAwardFor ?awardFilmProducer;
#               schema:awardYear ?awardYearProducer ;
#               schema:produced ?film .
#   ?award a schema:AwardCategory ;
#            rdfs:label ?awardLabel ;
#            schema:wikiID ?award;
#            schema:awardedToFilm ?film ;
#            schema:awardedToPerson ?winner ;
#            schema:awardYear ?awardYearMovie .           
#   ?winner a schema:AwardWinner ;
#             rdfs:label ?winnerLabel ;
#             schema:wikiID ?winner;
#             schema:receivedAward ?award ;
#             schema:awardYear ?awardYearMovie ;
#             schema:wonAwardFor ?film .
#   ?castAndCrew a schema:CastAndCrew ;
#                 rdfs:label ?castAndCrewLabel ;
#                 schema:wikiID ?castAndCrew ;
#                 schema:memberOfFilm ?film .



# # Klasse: Movie 
#   schema:hasCastMember a rdf:Property ;
#                          rdfs:label "has cast" ;
#                          rdfs:domain schema:Movie ;
#                          rdfs:range schema:CastMember ;
#                          rdfs:subPropertyOf schema:hasCastAndCrew .
#   schema:hasCastAndCrew a rdf:Property ;
#                          rdfs:label "has crew member" ;
#                          rdfs:domain schema:Movie ;
#                          rdfs:range schema:CastAndCrew ;
#                          rdfs:subPropertyOf schema:hasParticipant .
#   schema:hasParticipant a rdf:Property ;
#                           rdfs:label "has participant" ;
#                           rdfs:domain schema:Movie ;
#                           rdfs:range schema:MovieProduction .
#   schema:hasWriter a rdf:Property ;
#                      rdfs:label "written by" ;
#                      rdfs:domain schema:Movie ;
#                      rdfs:range schema:Writer ;
#                      rdfs:subPropertyOf schema:hasCastAndCrew .
#   schema:hasProducer a rdf:Property ;
#                        rdfs:label "produced by" ;
#                        rdfs:domain schema:Movie ;
#                        rdfs:range schema:Producer ;
#                        rdfs:subPropertyOf schema:hasCastAndCrew .
#   schema:hasDirector a rdf:Property ;
#                        rdfs:label "directed by" ;
#                        rdfs:domain schema:Movie ;
#                        rdfs:range schema:Director ;
#                        rdfs:subPropertyOf schema:hasCastAndCrew .
#   schema:hasGenre a rdf:Property ;
#                     rdfs:label "has gerne" ;
#                     rdfs:domain schema:Movie ;
#                     rdfs:range schema:Genre .
#   schema:receivedAwardFilm a rdf:Property;
#                           rdfs:label "won Awards" ;
#                           rdfs:domain schema:Movie ;
#                           rdfs:range schema:AwardCategory .


# # Klasse: Genre
#   schema:genreOf a rdf:Property ;
#                    rdfs:label "gerne of" ;
#                    rdfs:domain schema:Genre ;
#                    rdfs:range schema:Movie .

# # Klasse: Director
#   schema:directed a rdf:Property ;
#                     rdfs:label "directed" ;
#                     rdfs:domain schema:Director ;
#                     rdfs:range schema:Movie ;
#                     rdfs:subPropertyOf schema:participatedIn .
#   schema:receivedAwardDirector a rdf:Property ;
#                                  rdfs:label "won Award" ;
#                                  rdfs:domain schema:Director ;
#                                  rdfs:range schema:AwardCategory .
#   schema:wonAwardFor a rdf:Property ;
#                        rdfs:label "won Award for Movie" ;
#                        rdfs:domain schema:Director ;
#                        rdfs:range schema:Movie .

# # Klasse: CastMember  
#   schema:actedIn a rdf:Property ;
#                    rdfs:label "acted in" ;
#                    rdfs:domain schema:CastMember ;
#                    rdfs:range schema:Movie ;
#                    rdfs:subPropertyOf schema:participatedIn .
#   schema:receivedAwardCast a rdf:Property ;
#                              rdfs:label "won Award" ;
#                              rdfs:domain schema:CastMember ;
#                              rdfs:range schema:AwardCategory .
#   schema:wonAwardFor a rdf:Property ;
#                        rdfs:label "won Award for Movie" ;
#                        rdfs:domain schema:CastMember ;
#                        rdfs:range schema:Movie .

# # Klasse: Writer 
#   schema:wrote a rdf:Property ;
#                  rdfs:label "wrote" ;
#                  rdfs:domain schema:Writer ;
#                  rdfs:range schema:Movie ;
#                  rdfs:subPropertyOf schema:participatedIn .
#   schema:receivedAwardWriter a rdf:Property ;
#                           rdfs:label "won Award" ;
#                           rdfs:domain schema:Writer ;
#                           rdfs:range schema:AwardCategory .
#   schema:wonAwardFor a rdf:Property ;
#                        rdfs:label "won Award for Movie" ;
#                        rdfs:domain schema:Writer ;
#                        rdfs:range schema:Movie .

# # Klasse: Producer
#   schema:produced a rdf:Property ;
#                     rdfs:label "produced" ;
#                     rdfs:domain schema:Producer ;
#                     rdfs:range schema:Movie ;
#                     rdfs:subPropertyOf schema:participatedIn .
#   schema:receivedAwardProducer a rdf:Property ;
#                           rdfs:label "won Award" ;
#                           rdfs:domain schema:Producer ;
#                           rdfs:range schema:AwardCategory .
#   schema:wonAwardFor a rdf:Property ;
#                        rdfs:label "won Award for Movie" ;
#                        rdfs:domain schema:Producer ;
#                        rdfs:range schema:Movie .

# # Klasse: AwardCategory  
#   schema:awardedToFilm a rdf:Property ;
#                          rdfs:label "awarded to" ;
#                          rdfs:domain schema:AwardCategory ;
#                          rdfs:range schema:Movie .
#   schema:awardedToPerson a rdf:Property ;
#                            rdfs:label "awarded to" ;
#                            rdfs:domain schema:AwardCategory ;
#                            rdfs:range schema:AwardWinner .
  
# # Klasse: ohne Zuordnung
#   schema:participatedIn a rdf:Property ;
#                           rdfs:label "participated in" ;
#                           rdfs:domain schema:CastAndCrew ;
#                           rdfs:range schema:Movie .   

#   schema:Director rdfs:subClassOf schema:CastAndCrew .
#   schema:CastMember rdfs:subClassOf schema:CastAndCrew . 
#   schema:Writer rdfs:subClassOf schema:CastAndCrew .
#   schema:Producer rdfs:subClassOf schema:CastAndCrew .
#   schema:awardWinner rdfs:subClassOf schema:CastAndCrew.
  
#   schema:CastAndCrew rdfs:subClassOf schema:MovieProduction.
#   schema:Movie rdfs:subClassOf schema:MovieProduction.
#   schema:Genre rdfs:subClassOf schema:MovieProduction.
  
# }
# WHERE {

# ### Movie ###
#   ?film wdt:P31 wd:Q11424.
#   ?film rdfs:label ?filmLabel.
#   FILTER(LANG(?filmLabel) = "en") 
  
  
# #   VALUES ?film { wd:Q208632} # wd:Q44578 wd:Q160060 wd:Q125076 wd:Q134773 wd:Q20856802 }
#   ?film p:P166 ?awardStat.
#   ?awardStat ps:P166 ?award
#              VALUES ?award { wd:Q102427 wd:Q103360 wd:Q103916 wd:Q103618 wd:Q41417 wd:Q107258 wd:Q106291 wd:Q106301}
#   ?awardStat pq:P585 ?awardYear.
  
#   BIND(YEAR(?awardYear) AS ?awardYearMovie)
#   OPTIONAL { ?award rdfs:label ?awardLabel. FILTER(LANG(?awardLabel) = "en") }
  
#   BIND(?awardYearMovie - 1 AS ?releaseYear)
  
  
#   ?film p:P2047 ?statement .
#   ?statement ps:P2047 ?runTime .
#   FILTER NOT EXISTS {
#     ?film p:P2047 ?otherStatement .
#     ?otherStatement ps:P2047 ?otherRuntime .
#     FILTER(?otherRuntime < ?runTime)  # Behalte nur den kleinsten
#   }


# ### Genre ###
#     ?film wdt:P136 ?genre.
#     ?genre rdfs:label ?genreLabel.    
#     FILTER(LANG(?genreLabel) = "en") 


# ### Winner ###  
#   OPTIONAL {
#     ?awardStat pq:P1346 ?winner .  # Gewinner (z. B. Produzenten, Schauspieler)
#     ?winner rdfs:label ?winnerLabel .
#     FILTER(LANG(?winnerLabel) = "en")
#   }
 

# ### CastMember ###
#   OPTIONAL {
#     ?film wdt:P161 ?castMember .  # Cast Member des Films
#     ?castMember rdfs:label ?castMemberLabel .    
#     FILTER(LANG(?castMemberLabel) = "en")
    
#     # 🔹 Oscar-Gewinn überprüfen
#     ?castMember p:P166 ?awardStatement. 
#     ?awardStatement ps:P166 ?awardCast2
#        VALUES ?awardCast2 {wd:Q103618 wd:Q103916 wd:Q106291 wd:Q106301}
#   } 
 
#   OPTIONAL{
#     ?film p:P166 ?awardStat .
#     ?film wdt:P161 ?castMember .
#     ?awardStat ps:P166 ?awardCa .
#     VALUES ?awardCa {wd:Q103618 wd:Q103916 wd:Q106291 wd:Q106301 }
#     ?awardStat pq:P1346 ?winnerCa .  # Wer hat den Preis erhalten?

#     FILTER(?winnerCa = ?castMember)  # Gewinner ist der Writer
#     BIND(IF(?awardCa = wd:Q103916 || ?awardCa = wd:Q103618 || ?awardCa = wd:Q106291 || ?awardCa = wd:Q106301, ?awardCa, "" ) AS ?awardCast)
#     BIND(?film AS ?awardFilmCast)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearCast)
#     ?awardCast rdfs:label ?awardCastLabel. FILTER(LANG(?awardCastLabel) = "en")
#     ?awardFilmCast rdfs:label ?awardFilmCastLabel. FILTER(LANG(?awardFilmCastLabel) = "en")
#   }
   

# ### Director ###
#     ?film wdt:P57 ?director. 
#     ?director rdfs:label ?directorLabel.  
#     FILTER(LANG(?directorLabel) = "en")    

#   OPTIONAL {
#     ?film p:P166 ?awardStat .
#     ?film wdt:P162 ?director.
#     ?awardStat ps:P166 wd:Q103360 . 
#     ?awardStat pq:P1346 ?winnerDi .  # Wer hat den Preis erhalten?

#     FILTER(?winnerDi = ?director)
#     BIND(wd:Q103360 AS ?awardDirector)  # Award "Best Picture" zuordnen
#     BIND(?film AS ?awardFilmDirector)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearDirector)
#     ?awardDirector rdfs:label ?awardDirectorLabel. FILTER(LANG(?awardDirectorLabel) = "en")
#     ?awardFilmDirector rdfs:label ?awardFilmDirectorLabel. FILTER(LANG(?awardFilmDirectorLabel) = "en")
#   }
  

# ### Writer ### 
#     ?film wdt:P58 ?writer.
#     ?writer rdfs:label ?writerLabel.
#     FILTER(LANG(?writerLabel) = "en")
   
  
#   OPTIONAL {
#     ?film p:P166 ?awardStat .
#     ?film wdt:P58 ?writer .
#     ?awardStat ps:P166 ?awardWr .
#     VALUES ?awardWr {wd:Q41417 wd:Q107258}
#     ?awardStat pq:P1346 ?winnerWr .  # Wer hat den Preis erhalten?

#     FILTER(?winnerWr = ?writer)  # Gewinner ist der Writer
#     BIND(IF(?awardWr = wd:Q41417 || ?awardWr = wd:Q107258, ?awardWr, "" ) AS ?awardWriter)
#     BIND(?film AS ?awardFilmWriter)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearWriter)
#     ?awardWriter rdfs:label ?awardWriterLabel. FILTER(LANG(?awardWriterLabel) = "en")
#     ?awardFilmWriter rdfs:label ?awardFilmWriterLabel. FILTER(LANG(?awardFilmWriterLabel) = "en")
#   }


# ### Producer ###
#     ?film wdt:P162 ?producer.
#     ?producer rdfs:label ?producerLabel.
#     FILTER(LANG(?producerLabel) = "en")
    
  
#   OPTIONAL {
#     ?film p:P166 ?awardStat .
#     ?film wdt:P162 ?producer.
#     ?awardStat ps:P166 wd:Q102427 .  # Der Award ist "Best Picture"
#     ?awardStat pq:P1346 ?winnerPr .  # Wer hat den Preis erhalten?

#     FILTER(?winnerPr = ?producer)
#     BIND(wd:Q102427 AS ?awardProducer)  # Award "Best Picture" zuordnen
#     BIND(?film AS ?awardFilmProducer)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearProducer)
#     ?awardProducer rdfs:label ?awardProducerLabel. FILTER(LANG(?awardProducerLabel) = "en")
#     ?awardFilmProducer rdfs:label ?awardFilmProducerLabel. FILTER(LANG(?awardFilmProducerLabel) = "en")
#   }
  

# ### CastAndCrew ### 
#   # Alle CastAndCrew-Rollen als UNION gesammelt
#   {
#     ?film wdt:P57 ?person .   # Regisseur
#     BIND(?person AS ?castAndCrew)
#     BIND("Director" AS ?role)
#   }
#   UNION {
#     ?film wdt:P58 ?person .   # Drehbuchautor
#     BIND(?person AS ?castAndCrew)
#     BIND("Writer" AS ?role)
#   }
#   UNION {
#     ?film wdt:P162 ?person .  # Produzent
#     BIND(?person AS ?castAndCrew)
#     BIND("Producer" AS ?role)
#   }
#   UNION {
#     ?film wdt:P161 ?person .  # Cast Member des Films  
#     # 🔹 Oscar-Gewinn überprüfen
#     ?person p:P166 ?awardStatement. 
#     ?awardStatement ps:P166 ?awardCast2
#        VALUES ?awardCast2 {wd:Q103618 wd:Q103916 wd:Q106291 wd:Q106301}
    
#     BIND(?person AS ?castAndCrew)
#     BIND("Cast" AS ?role)
#   }

#   SERVICE wikibase:label {
#     bd:serviceParam wikibase:language "en,de" .
#     ?film rdfs:label ?filmLabel .
#     ?castAndCrew rdfs:label ?CastAndCrewLabel .
#   }
  
 
#   # Zeitraum der letzten 10 Jahre (anpassen falls nötig)
#   FILTER(year(?awardYear) <= 2025 )
#   FILTER(year(?awardYear) >= 2010 )
# }

Alle Filme mit Länderzuordnungen. Kann allerdins nicht ausgeführt werden, da es zu lange dauert.

In [ ]:
# PREFIX schema: <http://schema.org/>
# PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

# CONSTRUCT {
#   ?film a schema:Movie ;
#           rdfs:label ?filmLabel ;
#           schema:wikiID ?film ;
#           schema:releaseYear ?releaseYear;
#           schema:hasGenre ?genre ;
#           schema:hasDirector ?director ;
#           schema:hasCastMember ?castMember ;
#           schema:hasWriter ?writer ;
#           schema:hasProducer ?producer;
#           schema:receivedAwardFilm  ?award; 
#           schema:wonAwardInYear ?awardYearMovie ;
#           schema:runTime ?runTime .
# #           schema:originCountry ?originCountry ; 
# #           schema:filmedIn ?filmingCountry ;
# #           schema:takesPlaceIn ?narrativeCountry .
#   ?genre a schema:Genre ;
#            rdfs:label ?genreLabel ;
#            schema:wikiID ?genre ;
#            schema:genreOf ?film .
#   ?director a schema:Director;
#               rdfs:label ?directorLabel ;
#               schema:receivedAwardDirector ?awardDirector ;
#               schema:wikiID ?director ;
#               schema:awardYear ?awardYearDirector ;
#               schema:wonAwardFor ?awardFilmDirector;
#               schema:directed ?film .
#   ?castMember a schema:CastMember ;
#                 rdfs:label ?castMemberLabel ;
#                 schema:wikiID ?castMember ;
#                 schema:receivedAwardCast ?awardCast ;
#                 schema:wonAwardFor ?awardFilmCast;
#                 schema:awardYear ?awardYearCast ;
#                 schema:actedIn ?film .
#   ?writer a schema:Writer ;
#             rdfs:label ?writerLabel ;
#             schema:wikiID ?writer ;
#             schema:receivedAwardWriter ?awardWriter ;
#             schema:wonAwardFor ?awardFilmWriter;
#             schema:awardYear ?awardYearWriter ;
#             schema:wrote ?film .
#   ?producer a schema:Producer ;
#               rdfs:label ?producerLabel;
#               schema:wikiID ?producer;
#               schema:receivedAwardProducer ?awardProducer ;
#               schema:wonAwardFor ?awardFilmProducer;
#               schema:awardYear ?awardYearProducer ;
#               schema:produced ?film .
#   ?award a schema:AwardCategory ;
#            rdfs:label ?awardLabel ;
#            schema:wikiID ?award;
#            schema:awardedToFilm ?film ;
#            schema:awardedToPerson ?winner ;
#            schema:awardYear ?awardYearMovie .           
#   ?winner a schema:AwardWinner ;
#             rdfs:label ?winnerLabel ;
#             schema:wikiID ?winner;
#             schema:receivedAward ?award ;
#             schema:awardYear ?awardYearMovie ;
#             schema:wonAwardFor ?film .
# #   ?originCountry a schema:OriginCountry ;
# #                    rdfs:label ?originCountryLabel ;
# #                    schema:wikiID ?originCountry ;
# #                    schema:continent ?originContinent ;
# #                    schema:originOf ?film .
# #   ?filmingCountry a schema:FilmingCountry ;
# #                     rdfs:label ?filmingCountryLabel ;
# #                     schema:wikiID ?filmingCountry;
# #                     schema:continent ?filmingContinent ;
# #                     schema:filmLocation ?film.
# #   ?narrativeCountry a schema:NarrativeLocation ;
# #                       rdfs:label ?narrativeCountryLabel ;
# #                       schema:wikiID ?narrativeCountry ;
# #                       schema:continent ?narrativeContinent;
# #                       schema:placeOfStory ?film .

  
#   schema:hasCastMember a rdf:Property ;
#                          rdfs:label "has cast" ;
#                          rdfs:domain schema:Movie ;
#                          rdfs:range schema:CastMember .
#   schema:hasWriter a rdf:Property ;
#                      rdfs:label "written by" ;
#                      rdfs:domain schema:Movie ;
#                      rdfs:range schema:Writer .
#   schema:hasProducer a rdf:Property ;
#                        rdfs:label "produced by" ;
#                        rdfs:domain schema:Movie ;
#                        rdfs:range schema:Producer .
#   schema:hasDirector a rdf:Property ;
#                        rdfs:label "directed by" ;
#                        rdfs:domain schema:Movie ;
#                        rdfs:range schema:Director .
#   schema:hasGenre a rdf:Property ;
#                     rdfs:label "has gerne" ;
#                     rdfs:domain schema:Movie ;
#                     rdfs:range schema:Genre .
#   schema:receivedAwardFilm a rdf:Property;
#                           rdfs:label "won Awards" ;
#                           rdfs:domain schema:Movie ;
#                           rdfs:range schema:AwardCategory .

#   schema:genreOf a rdf:Property ;
#                    rdfs:label "gerne of" ;
#                    rdfs:domain schema:Genre ;
#                    rdfs:range schema:Movie .

#   schema:directed a rdf:Property ;
#                     rdfs:label "directed" ;
#                     rdfs:domain schema:Director ;
#                     rdfs:range schema:Movie .
#   schema:receivedAwardDirector a rdf:Property ;
#                                  rdfs:label "won Award" ;
#                                  rdfs:domain schema:Director ;
#                                  rdfs:range schema:AwardCategory .
#   schema:wonAwardFor a rdf:Property ;
#                        rdfs:label "won Award for Movie" ;
#                        rdfs:domain schema:Director ;
#                        rdfs:range schema:Movie .
  
#   schema:actedIn a rdf:Property ;
#                    rdfs:label "acted in" ;
#                    rdfs:domain schema:CastMember ;
#                    rdfs:range schema:Movie .
#   schema:receivedAwardCast a rdf:Property ;
#                              rdfs:label "won Award" ;
#                              rdfs:domain schema:CastMember ;
#                              rdfs:range schema:AwardCategory .
#   schema:wonAwardFor a rdf:Property ;
#                        rdfs:label "won Award for Movie" ;
#                        rdfs:domain schema:CastMember ;
#                        rdfs:range schema:Movie .

#   schema:wrote a rdf:Property ;
#                  rdfs:label "wrote" ;
#                  rdfs:domain schema:Writer ;
#                  rdfs:range schema:Movie .
#   schema:receivedAwardWriter a rdf:Property ;
#                           rdfs:label "won Award" ;
#                           rdfs:domain schema:Writer ;
#                           rdfs:range schema:AwardCategory .
#   schema:wonAwardFor a rdf:Property ;
#                        rdfs:label "won Award for Movie" ;
#                        rdfs:domain schema:Writer ;
#                        rdfs:range schema:Movie .

#   schema:produced a rdf:Property ;
#                     rdfs:label "produced" ;
#                     rdfs:domain schema:Producer ;
#                     rdfs:range schema:Movie .
#   schema:receivedAwardProducer a rdf:Property ;
#                           rdfs:label "won Award" ;
#                           rdfs:domain schema:Producer ;
#                           rdfs:range schema:AwardCategory .
#   schema:wonAwardFor a rdf:Property ;
#                        rdfs:label "won Award for Movie" ;
#                        rdfs:domain schema:Producer ;
#                        rdfs:range schema:Movie .
   
#   schema:awardedToFilm a rdf:Property ;
#                          rdfs:label "awarded to" ;
#                          rdfs:domain schema:AwardCategory ;
#                          rdfs:range schema:Movie .
#   schema:awardedToPerson a rdf:Property ;
#                            rdfs:label "awarded to" ;
#                            rdfs:domain schema:AwardCategory ;
#                            rdfs:range schema:AwardWinner .
 
# #   schema:originCountry a rdf:Property ;
# #                          rdfs:label "Country of Origin" ;
# #                          rdfs:domain schema:Movie ;
# #                          rdfs:range schema:OriginCountry .  
# #   schema:filmedIn a rdf:Property ;
# #                     rdfs:label "filmed in" ;
# #                     rdfs:domain schema:Movie ;
# #                     rdfs:range schema:FilmingCountry .  
# #   schema:takesPlaceIn a rdf:Property ;
# #                         rdfs:label "Story takes place in" ;
# #                         rdfs:domain schema:Movie ;
# #                         rdfs:range schema:NarrativeLocation .    

  
    

#   schema:Director rdfs:subClassOf schema:CastAndCrew .
#   schema:CastMember rdfs:subClassOf schema:CastAndCrew . 
#   schema:Writer rdfs:subClassOf schema:CastAndCrew .
#   schema:Producer rdfs:subClassOf schema:CastAndCrew .
#   schema:awardWinner rdfs:subClassOf schema:CastAndCrew.
  
#   schema:Movie rdfs:subClassOf schema:MovieProduction.
#   schema:Genre rdfs:subClassOf schema:MovieProduction.
#   schema:CastAndCrew rdfs:subClassOf schema:MovieProduction.
# #   schema:Country rdfs:subClassOf schema:MovieProduction.
# #   
# #   schema:OriginCountry rdfs:subClassOf schema:Country.
# #   schema:FilmingCountry rdfs:subClassOf schema:Country.
# #   schema:NarrativeLocation rdfs:subClassOf schema:Country.
# }
                    
# WHERE {
# # 
#   ?film wdt:P31 wd:Q11424.
#   ?film rdfs:label ?filmLabel.
#   FILTER(LANG(?filmLabel) = "en") 
# #   BIND(IF(?film = wd:Q1197729, "Titanic (1952)", ?filmLabel2) AS ?filmLabel)
  
#   # Hole das Original-Label, wenn verfügbar
# #   OPTIONAL {
# #     ?film rdfs:label ?originalLabel .
# #     FILTER(LANG(?originalLabel) = "en")  
# #   }
  
# #   VALUES ?film { wd:Q500672} #wd:Q208632 } # wd:Q44578 wd:Q160060 wd:Q125076 wd:Q134773 wd:Q20856802 }
#   ?film p:P166 ?awardStat.
#   ?awardStat ps:P166 ?award
#              VALUES ?award { wd:Q102427 wd:Q103360 wd:Q103916 wd:Q103618 wd:Q41417 wd:Q107258 wd:Q106291 wd:Q106301}
#   ?awardStat pq:P585 ?awardYear.
  
#   BIND(YEAR(?awardYear) AS ?awardYearMovie)
#   OPTIONAL { ?award rdfs:label ?awardLabel. FILTER(LANG(?awardLabel) = "en") }
  
#   BIND(?awardYearMovie - 1 AS ?releaseYear) 
  
#   ?film p:P2047 ?statement .
#   ?statement ps:P2047 ?runTime .
#   FILTER NOT EXISTS {
#     ?film p:P2047 ?otherStatement .
#     ?otherStatement ps:P2047 ?otherRuntime .
#     FILTER(?otherRuntime < ?runTime)  # Behalte nur den kleinsten
#   }
#   #   ?film wdt:P2047 ?runTime.
  
# #   OPTIONAL{
#     ?film wdt:P136 ?genre.
#     ?genre rdfs:label ?genreLabel.    
#     FILTER(LANG(?genreLabel) = "en") 
# #   }
  
#   OPTIONAL {
#     ?awardStat pq:P1346 ?winner .  # Gewinner (z. B. Produzenten, Schauspieler)
#     ?winner rdfs:label ?winnerLabel .
#     FILTER(LANG(?winnerLabel) = "en")
#   }
  
#   OPTIONAL {
#     ?film wdt:P161 ?castMember .  # Cast Member des Films
#     ?castMember rdfs:label ?castMemberLabel .    
#     FILTER(LANG(?castMemberLabel) = "en")
    
#     # 🔹 Oscar-Gewinn überprüfen
#     ?castMember p:P166 ?awardStatement. 
#     ?awardStatement ps:P166 ?awardCast2
#        VALUES ?awardCast2 {wd:Q103618 wd:Q103916 wd:Q106291 wd:Q106301}
#   } 
 
#   OPTIONAL{
#     ?film p:P166 ?awardStat .
#     ?film wdt:P161 ?castMember .
#     ?awardStat ps:P166 ?awardCa .
#     VALUES ?awardCa {wd:Q103618 wd:Q103916 wd:Q106291 wd:Q106301 }
#     ?awardStat pq:P1346 ?winnerCa .  # Wer hat den Preis erhalten?

#     FILTER(?winnerCa = ?castMember)  # Gewinner ist der Writer
#     BIND(IF(?awardCa = wd:Q103916 || ?awardCa = wd:Q103618 || ?awardCa = wd:Q106291 || ?awardCa = wd:Q106301, ?awardCa, "" ) AS ?awardCast)
#     BIND(?film AS ?awardFilmCast)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearCast)
#     ?awardCast rdfs:label ?awardCastLabel. FILTER(LANG(?awardCastLabel) = "en")
#     ?awardFilmCast rdfs:label ?awardFilmCastLabel. FILTER(LANG(?awardFilmCastLabel) = "en")
#   }
   

#     ?film wdt:P57 ?director. 
#     ?director rdfs:label ?directorLabel.  
#     FILTER(LANG(?directorLabel) = "en")

#   OPTIONAL {
#     ?film p:P166 ?awardStat .
#     ?film wdt:P162 ?director.
#     ?awardStat ps:P166 wd:Q103360 . 
#     ?awardStat pq:P1346 ?winnerDi .  # Wer hat den Preis erhalten?

#     # Prüfen, ob der Gewinner gleich dem Produzenten ist
#     FILTER(?winnerDi = ?director)
#     BIND(wd:Q103360 AS ?awardDirector)  # Award "Best Picture" zuordnen
#     BIND(?film AS ?awardFilmDirector)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearDirector)
#     ?awardDirector rdfs:label ?awardDirectorLabel. FILTER(LANG(?awardDirectorLabel) = "en")
#     ?awardFilmDirector rdfs:label ?awardFilmDirectorLabel. FILTER(LANG(?awardFilmDirectorLabel) = "en")
#   }
  
 
#     ?film wdt:P58 ?writer.
#     ?writer rdfs:label ?writerLabel.
#     FILTER(LANG(?writerLabel) = "en")
  
#   OPTIONAL {
#     ?film p:P166 ?awardStat .
#     ?film wdt:P58 ?writer .
#     ?awardStat ps:P166 ?awardWr .
#     VALUES ?awardWr {wd:Q41417 wd:Q107258}
#     ?awardStat pq:P1346 ?winnerWr .  # Wer hat den Preis erhalten?

#     FILTER(?winnerWr = ?writer)  # Gewinner ist der Writer
#     BIND(IF(?awardWr = wd:Q41417 || ?awardWr = wd:Q107258, ?awardWr, "" ) AS ?awardWriter)
#     BIND(?film AS ?awardFilmWriter)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearWriter)
#     ?awardWriter rdfs:label ?awardWriterLabel. FILTER(LANG(?awardWriterLabel) = "en")
#     ?awardFilmWriter rdfs:label ?awardFilmWriterLabel. FILTER(LANG(?awardFilmWriterLabel) = "en")
#   }

# #   OPTIONAL{  
#     ?film wdt:P162 ?producer.
#     ?producer rdfs:label ?producerLabel.
#     FILTER(LANG(?producerLabel) = "en")
#   OPTIONAL {
#     ?film p:P166 ?awardStat .
#     ?film wdt:P162 ?producer.
#     ?awardStat ps:P166 wd:Q102427 .  # Der Award ist "Best Picture"
#     ?awardStat pq:P1346 ?winnerPr .  # Wer hat den Preis erhalten?

#     # Prüfen, ob der Gewinner gleich dem Produzenten ist
#     FILTER(?winnerPr = ?producer)
#     BIND(wd:Q102427 AS ?awardProducer)  # Award "Best Picture" zuordnen
#     BIND(?film AS ?awardFilmProducer)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearProducer)
#     ?awardProducer rdfs:label ?awardProducerLabel. FILTER(LANG(?awardProducerLabel) = "en")
#     ?awardFilmProducer rdfs:label ?awardFilmProducerLabel. FILTER(LANG(?awardFilmProducerLabel) = "en")
#   }
  
# #   OPTIONAL {
# #     ?film wdt:P915/wdt:P17 ?filmingCountry .  # Hole Drehort (z. B. Stadt, Land oder Ort)
# #     ?filmingCountry rdfs:label ?filmingCountryLabel .
# #     FILTER(LANG(?filmingCountryLabel) = "en")
# #     
# #     ?filmingCountry wdt:P30 ?filmingContinent .
# #     ?filmingContinent rdfs:label ?filmingContinentLabel .
# #     FILTER(LANG(?filmingContinentLabel) = "en")
# #   }
# # 
# #   OPTIONAL {
# #     ?film wdt:P840/wdt:P17 ?narrativeCountry .
# #     ?narrativeCountry rdfs:label ?narrativeCountryLabel .
# #     FILTER(LANG(?narrativeCountryLabel) = "en")
# # 
# #     ?narrativeCountry wdt:P30 ?narrativeContinent .
# #     ?narrativeContinent rdfs:label ?narrativeContinentLabel .
# #     FILTER(LANG(?narrativeContinentLabel) = "en")
# #   }
# #   
# #   OPTIONAL {
# #     ?film wdt:P495 ?originCountry.
# #     ?originCountry rdfs:label ?originCountryLabel.
# #     FILTER(LANG(?originCountryLabel) = "en")
# #     
# #     ?originCountry wdt:P30 ?originContinent .
# #     ?originContinent rdfs:label ?originContinentLabel .
# #     FILTER(LANG(?originContinentLabel) = "en")
# #   }

  
#   # Zeitraum der letzten 10 Jahre (anpassen falls nötig)
# #   FILTER(year(?awardYear) >= 2020 )
# #   FILTER(year(?awardYear) <= 2021 )

#   # Setzt Label auf englisch
# #   FILTER(LANG(?filmLabel) = "en")
# }